<a href="https://colab.research.google.com/github/Necrotox/Necrotox/blob/main/bot_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q --no-warn-script-location --index-url https://download.pytorch.org/whl/cpu \
  torch==2.7.1+cpu > /dev/null 2>&1

!pip install -q --no-warn-script-location --index-url https://download.pytorch.org/whl/cpu \
  torchvision==0.22.0+cpu > /dev/null 2>&1
!pip install -q --no-warn-script-location --index-url https://download.pytorch.org/whl/cpu \
  torchaudio==2.7.1+cpu > /dev/null 2>&1

!pip install -q --no-warn-script-location \
  pytorch-lightning==2.5.2 pytorch-forecasting==1.4.0 > /dev/null 2>&1

!pip install -q tinkoff-investments dill telebot --upgrade mplfinance > /dev/null 2>&1

import asyncio
import contextlib
import functools
import statsmodels.api as sm
import inspect
import zipfile
import pickle
import json
import gc
import re
import time
import pytz
import dill
import lzma
import pandas as pd
import nest_asyncio
import hashlib
import random
import joblib
import math
import psutil

import scipy.sparse as sp
import scipy.sparse.linalg as spla


from numba import jit, njit
from grpc import StatusCode
from grpc.aio import AioRpcError
from contextlib import redirect_stderr, redirect_stdout
from tinkoff.invest import AsyncClient, CandleInterval
from tinkoff.invest.exceptions import RequestError
from tinkoff.invest import (
    AsyncClient,
    CandleInstrument,
    MarketDataRequest,
    SubscribeCandlesRequest,
    SubscriptionAction,
    SubscriptionInterval,
)
from tinkoff.invest.schemas import Candle
from statsmodels.tsa.seasonal import STL
from scipy.stats import norm
from sklearn.ensemble import IsolationForest
from collections import defaultdict, deque
from multiprocessing import Process
from multiprocessing import Process
from datetime import datetime as dt
from datetime import timedelta
import numpy as np
import mplfinance as mpf
import telebot
import datetime
import matplotlib
from sklearn.neighbors import NearestNeighbors
from scipy.signal import lfilter

from scipy import stats
from scipy.ndimage import gaussian_filter1d
from scipy.signal import savgol_filter
from scipy.stats import norm, pearsonr
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from sklearn.model_selection import BaseCrossValidator

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, PowerTransformer, QuantileTransformer, RobustScaler, StandardScaler

from sklearn.neighbors import BallTree

from pandas.tseries.frequencies import to_offset
from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data.encoders import EncoderNormalizer
from pytorch_forecasting.metrics import Metric, QuantileLoss
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.callbacks.progress import RichProgressBar, TQDMProgressBar
from pytorch_lightning.loggers import TensorBoardLogger

import torch
import torch.nn as nn
import torch.nn.functional as F

from typing import Any, Dict, Iterable, Iterator, List, Literal, Mapping, Optional, Tuple
import os, gc, warnings, typing as tp

from lightgbm import LGBMRegressor

matplotlib.use("agg")
import warnings

warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings('ignore')

import psutil
import os.path

nest_asyncio.apply()

print('Загруженны библиотеки', f"Память: {psutil.Process().memory_info().rss / 1024 ** 2:.2f} МБ")

# nest_asyncio.apply()

#-1002681271607 Чат с общей статой
#-1002619839070 Чат по прошедшим проверку алертам
#-1002555215009 Чат со статистикой
#-1002596481496 Чат с ошибками


warnings.simplefilter("ignore", UserWarning)
path_to_save = '/content/drive/MyDrive/t_ml/data/'
tg_bot_token = "7734846848:AAHjM7r9ZInzC59GYkUfQUZr7a6ynIaTRAg"
bot = telebot.TeleBot(tg_bot_token)

#with open('/content/drive/MyDrive/t_ml/ful_tickers_params.txt', 'r') as file:
#    full_ticker_params = json.load(file)


MAX_CANDLES = 5000

TOKEN = 't.bWjxPlU6vz77qoqS754OKy0QorLDaZP-CE091dhGl56v7GHrqgF-mQAdWaeRg2kDRJmmxzvaaOwKUTxW6dnOKg'
sectors = ['consumer', 'ecomaterials', 'electrocars', 'energy', 'financial', 'green_buildings',
           'green_energy', 'health_care', 'industrials', 'it', 'materials',
           'other', 'real_estate', 'telecom', 'utilities']

ban_list = ['KLVZ', 'APTK', 'OKEY', 'VSEH', 'KRKNP', 'UDMN', 'YAKG', 'AFKS', 'CARM', 'MGKL', 'MBNK', 'ZAYM', 'GLTR@GS',
            'DELI', 'DIAS', 'IVAT', 'SOFL', 'AKRN', 'GMKN', 'KAZT', 'KAZTP',
            'KZOS', 'KZOSP', 'NKNCP', 'OBNE', 'OBNEP', 'PHOR', 'RUAL', 'UFOSP', 'UNKL', 'VSMO', 'LEAS', 'CNTL', 'MGTSP',
            'HYDR', 'IRAO', 'MRKY', 'OGKB']

countrys = ['AR', 'AU', 'BE', 'BM', 'BR', 'CA', 'CH', 'CN',
            'DE', 'FI', 'FR', 'GB', 'HK', 'IE', 'IL', 'IN', 'IT', 'JP',
            'KR', 'KZ', 'NL', 'PE', 'RU', 'SE', 'SG', 'TW', 'US', 'UY']

#TICKERS = ['LIFE','UNAC','VKCO','MGNT','SVCB','ETLN','WUSH','HHRU','LNZL','SELG',
# 'TATNP','PRFN','MAGN','VTBR','NKHP','ALRS','MRKP','FLOT','TATN','SPBE','DVEC','RTKM','MTSS',
# 'TGKN','TRNFP','FEES','RBCM','ZILLP','MSRS','NSVZ','GCHE','SNGSP','NVTK','NKNC',
# 'AQUA','VRSB','MOEX','ROLO','SNGS','CBOM','TGKBP','ABRD','PIKK','ROSN','TRMK','MRKU',
# 'CHMF','ENPG','MRKV','LSRG','SVAV','RTKMP','KMAZ','VEON-RX','BANE','AFLT','SBER',
# 'MVID','MSTT','MTLR','GAZP','SBERP','LENT','BSPB','RKKE','LSNG','MRKC','POSI','KROT','SIBN',
# 'TGKB','KLSB','RNFT','MRKZ','QIWI','GTRK','NLMK','BELU','LNZLP','SFIN','RASP','SGZH',
# 'UPRO','MRKS','MSNG','PLZL','LKOH','NMTP','TGKA','MTLRP']

TICKERS =  ['RAGR', 'MGNT', 'MSTT', 'VRSB', 'PRFN', 'MTLR', 'LIFE', 'UPRO', 'GECO',
       'BANE', 'MTLRP', 'GEMC', 'NVTK', 'TRNFP', 'TRMK', 'LSNGP', 'OBNEP',
       'SNGSP', 'UWGN', 'MRKP', 'KZOSP', 'YDEX', 'NLMK', 'IRKT', 'CNTLP',
       'LENT', 'KLSB', 'SELG', 'NMTP', 'UNAC', 'VKCO', 'MRKU', 'UGLD', 'NTZL',
       'BANEP', 'FLOT', 'TGKJ', 'MAGN', 'ROSN', 'TGKB', 'AFKS', 'TTLK', 'HEAD',
       'KZIZ', 'NOMP', 'OKEY', 'ABRD', 'NSVZ', 'MRKV', 'LSNG', 'MRKC', 'SVAV',
       'ETLN', 'MRKZ', 'LNZL', 'CNRU', 'BSPB', 'RBCM', 'PMSB', 'LSRG', 'RNFT',
       'MOEX', 'GTRK', 'NKHP', 'LKOH', 'SBERP', 'SBER', 'PLZL', 'RENI', 'MDMG',
       'AFLT', 'FESH', 'OBNE', 'X5', 'MGTSP', 'DVEC', 'KROT', 'TATNP', 'OZPH',
       'TGKN', 'TATN', 'PMSBP', 'TGKBP', 'SPBE', 'LNZLP', 'CHMK', 'KZIZP',
       'RKKE', 'FRHC']

#TICKERS = ['NKNC', 'PIKK']


SECTOR = sectors[4]
COUNTRY = countrys[22]

try:
    with open(f'{path_to_save}open_price.txt', 'r') as f:
        open_price = json.loads(f.read())

    with open(f'{path_to_save}close_price.txt', 'r') as f:
        close_price = json.loads(f.read())

    with open(f'{path_to_save}high_price.txt', 'r') as f:
        high_price = json.loads(f.read())

    with open(f'{path_to_save}low_price.txt', 'r') as f:
        low_price = json.loads(f.read())

    with open(f'{path_to_save}volume.txt', 'r') as f:
        volume = json.loads(f.read())

    print("Данные 1 успешно загружены.")
except Exception as e:
    print(f"Ошибка при загрузке данных: {e}")
    open_price, close_price, high_price, low_price, volume = {}, {}, {}, {}, {}
    await asyncio.sleep(60)

try:
    with open(f'{path_to_save}time_last_kline_start.txt', 'r') as f:
        time_last_kline_start = json.loads(f.read())

    with open(f'{path_to_save}time_last_kline_end.txt', 'r') as f:
        time_last_kline_end = json.loads(f.read())

    with open(f'{path_to_save}ma.txt', 'r') as f:
        ma = json.loads(f.read())

    with open(f'{path_to_save}open_trades.txt', 'r') as f:
        open_trades = json.loads(f.read())

    with open(f'{path_to_save}trading_data.txt', 'r') as f:
        trading_data = json.loads(f.read())

    print("Данные 2 успешно загружены.")
except Exception as e:
    print(f"Ошибка при загрузке данных: {e}")
    time_last_kline_start, time_last_kline_end, ma, open_trades, trading_data = {}, {}, {}, {}, {}
    await asyncio.sleep(60)

try:

    with open(f'{path_to_save}pmax.txt', 'r') as f:
        pmax = json.loads(f.read())

    with open(f'{path_to_save}signals.txt', 'r') as f:
        signals = json.loads(f.read())

    print("Данные 3 успешно загружены.")
except Exception as e:
    print(f"Ошибка при загрузке данных: {e}")
    pmax, signals = {}, {}
    await asyncio.sleep(60)

print('Загрузка Моделей', f"Память: {psutil.Process().memory_info().rss / 1024 ** 2:.2f} МБ")

try:
    kmeans_global = joblib.load(f'{path_to_save}models/kmeans_global.pkl')
    print('Модель MiniBatchKMeans успешно загруженна', f"Память: {psutil.Process().memory_info().rss / 1024 ** 2:.2f} МБ")

except Exception as e:
    print(f"Ошибка при загрузке модели MiniBatchKMeans: {e}")
    kmeans_global = joblib.load(f'{path_to_save}models/kmeans_global.pkl')
    print('Модель MiniBatchKMeans загруженна со второй попытки', f"Память: {psutil.Process().memory_info().rss / 1024 ** 2:.2f} МБ")
    await asyncio.sleep(60)

try:
    scaler_global = joblib.load(f'{path_to_save}models/scaler_global.pkl')
    print('Скалер успешно загружен', f"Память: {psutil.Process().memory_info().rss / 1024 ** 2:.2f} МБ")

except Exception as e:
    print(f"Ошибка при загрузке модели MiniBatchKMeans: {e}")
    scaler_global = joblib.load(f'{path_to_save}models/scaler_global.pkl')
    print('Скалер загружен со второй попытки', f"Память: {psutil.Process().memory_info().rss / 1024 ** 2:.2f} МБ")
    await asyncio.sleep(60)
'''
try:
    with open(f'{path_to_save}/models/regression_model_general.dill', 'rb') as file:
        regression_model = dill.load(file)
        print('Модель выхода из сделки загруженна', f"Память: {psutil.Process().memory_info().rss / 1024 ** 2:.2f} МБ")
except Exception as e:
    print(f"Ошибка при загрузке модели регрессии: {e}")
    with open(f'{path_to_save}/models/regression_model_general.dill', 'rb') as file:
        regression_model = dill.load(file)
        print('Модель выхода из сделки загруженна с второйпопытки',
              f"Память: {psutil.Process().memory_info().rss / 1024 ** 2:.2f} МБ")
    await asyncio.sleep(60)
try:
    with open(f'{path_to_save}/models/global_model.dill', 'rb') as file:
        classifier_model = dill.load(file)
        print('Глобальная модель поиска входа для всех кластеров заргуженна',
              f"Память: {psutil.Process().memory_info().rss / 1024 ** 2:.2f} МБ")
except Exception as e:
    print(f"Ошибка при загрузке модели поиска входа для 0 кластера: {e}")
    with open(f'{path_to_save}/ful_tickers_params.txt', 'rb') as file:
        classifier_model = dill.load(file)
        print('Глобальная модель поиска входа для всех кластеров загруженна с второй попытки',
              f"Память: {psutil.Process().memory_info().rss / 1024 ** 2:.2f} МБ")
    await asyncio.sleep(60)
'''

try:
    with open(f'{path_to_save}/ful_tickers_params_new.txt', 'r') as file:
      ticker_params = json.load(file)
      print('Параметры для тикетов загруженны',
            f"Память: {psutil.Process().memory_info().rss / 1024 ** 2:.2f} МБ")
except Exception as e:
    print(f"Ошибка при загрузке параметров тикетов: {e}")
    with open(f'{path_to_save}/ful_tickers_params_new.txt', 'r') as file:
        ticker_params = json.load(file)
        print('Параметры для тикетов загруженны со второй попытки',
              f"Память: {psutil.Process().memory_info().rss / 1024 ** 2:.2f} МБ")
    await asyncio.sleep(60)

try:
    transformers_path = f'{path_to_save}/models/neuros/'
    models_path = f'{path_to_save}/models/final_models/'

    with open(f'{path_to_save}/models/final_cols.pkl', 'rb') as f:
        final_cols = pickle.load(f)

    with open(f'{path_to_save}/models/final_params.pkl', 'rb') as f:
        final_params = pickle.load(f)

    with open(f'{path_to_save}/models/best_methods.pkl', 'rb') as f:
        methods = pickle.load(f)

except Exception as e:
    print(f"Ошибка при определении пути на модели и трансформеры: {e}")
    await asyncio.sleep(60)

try:
    with open(f'{path_to_save}/phase_ful_tickers_params.txt', 'r') as file:
      phase_ticker_params = json.load(file)
      print('Параметры для тикетов с фазами загруженны',
            f"Память: {psutil.Process().memory_info().rss / 1024 ** 2:.2f} МБ")
except Exception as e:
    print(f"Ошибка при загрузке параметров тикетов с фазами: {e}")
    with open(f'{path_to_save}/phase_ful_tickers_params.txt', 'r') as file:
        phase_ticker_params = json.load(file)
        print('Параметры для тикетов с фазами загруженны со второй попытки',
              f"Память: {psutil.Process().memory_info().rss / 1024 ** 2:.2f} МБ")
    await asyncio.sleep(60)

print('Память после загрузки всех модулей', f"Память: {psutil.Process().memory_info().rss / 1024 ** 2:.2f} МБ")


# open_price, close_price, high_price, low_price, volume, time_last_kline_start, time_last_kline_end, atr, pmax, signals, var = {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}

def to_dense(X):
    """Преобразует разреженную матрицу в плотную."""
    if issparse(X):
        return X.toarray()
    return X

class ToDenseTransformer(BaseEstimator, TransformerMixin):
    """Преобразует разреженную матрицу в плотную numpy-массив."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        if issparse(X):
            return X.toarray()
        return X
    def get_feature_names_out(self, input_features: tp.Sequence[str] | None = None):
        return np.asarray(input_features) if input_features is not None else np.array([])

def deep_elbow(imp: np.ndarray, win: int = 5, eps: float = 0.02) -> int:
    """
    Берём окно длиной win, считаем средний относительный спад.
    Первое место, где спад < eps, считаем плато.
    """
    if len(imp) <= win:
        return len(imp)
    dif = np.abs(np.diff(imp) / (imp[:-1] + 1e-9))
    # скользящее среднее
    m = np.convolve(dif, np.ones(win) / win, mode="valid")
    flat = np.nonzero(m < eps)[0]
    return int(flat[0] + win) if flat.size else len(imp)

# ───────────────────────────────────────────────────────
# 2.  корреляционная чистка (быстрая, исправленная)
# ───────────────────────────────────────────────────────
def corr_prune(df: pd.DataFrame, feats: list[str], thr=.95) -> list[str]:
    if len(feats) < 2 or thr >= 1:
        return feats
    X  = df[feats].apply(pd.to_numeric, errors='ignore')
    C  = X.corr().abs().to_numpy()
    keep = []
    for i in range(len(feats)):
        if not keep or C[i, keep].max() < thr:
            keep.append(i)
    return [feats[i] for i in keep]

# ───────────────────────────────────────────────────────
# 3.  универсальный быстрый селектор
# ───────────────────────────────────────────────────────
def fast_feature_select(
        res           : pd.DataFrame,      # feature / importance
        df_full       : pd.DataFrame,      # датасет для corr-prune
        target_col    : str = "target",
        *,
        method        : str = "elbow",     # elbow | deep_elbow | percentile | quantile | top_k
        top_k         : int = 150,         # для method="top_k"
        perc_limit    : float = .90,       # для method="percentile"
        quantile_q    : float = .10,       # для method="quantile"
        elbow_eps     : float = .05,       # (> flat %) для (shallow) elbow
        deep_win      : int = 5,           # окно для deep_elbow
        deep_eps      : float = .02,       # порог для deep_elbow
        corr_thr      : float = .95        # корреляционный порог
) -> list[str]:

    ranked = res.sort_values("importance", ascending=False).reset_index(drop=True)
    feats  = ranked.feature.to_numpy()
    imps   = ranked.importance.to_numpy()

    # ---------- 1) сколько оставить  ----------
    if method == "elbow":                 # одношаговое колено
        k = np.argmax(np.abs(np.diff(imps) / (imps[:-1] + 1e-9)) < elbow_eps) + 1
        if k == 1:        # колено не найдено
            k = len(imps)
    elif method == "deep_elbow":
        k = deep_elbow(imps, win=deep_win, eps=deep_eps)
    elif method == "percentile":          # кумулятивная доля
        cum = np.cumsum(imps)
        k   = np.searchsorted(cum / cum[-1], perc_limit) + 1
    elif method == "quantile":
        thr = np.quantile(imps, 1 - quantile_q)
        k   = int((imps >= thr).sum())
    elif method == "top_k":
        k = min(top_k, len(feats))
    else:
        raise ValueError("unknown method")

    selected = feats[:k].tolist()

    # ---------- 2) корреляционная чистка ----------
    selected = corr_prune(
        df_full.drop(columns=[target_col], errors='ignore'),
        selected,
        thr=corr_thr
    )

    return selected

def patch_feature_timings(cls):
    """
    Оборачивает все методы cls, начинающиеся на _feat_,
    и складывает затраченное время в self._timings[method_name].
    Вызывать сразу после объявления класса.
    """
    def timed(func):
        def wrapper(self, *args, **kwargs):
            t0 = time.perf_counter()
            result = func(self, *args, **kwargs)
            dt = time.perf_counter() - t0
            # заводим словарь при первом же вызове
            if not hasattr(self, "_timings"):
                self._timings = {}
            self._timings[func.__name__] = dt
            return result
        return wrapper

    for name, method in inspect.getmembers(cls, inspect.isfunction):
        if name.startswith("_feat_"):             # ← только расчётные функции
            setattr(cls, name, timed(method))

    return cls

def _slope(y):
        x = np.arange(len(y))
        # линейная регрессия «по формуле»
        xm, ym = x.mean(), y.mean()
        beta = ((x - xm) * (y - ym)).sum() / ((x - xm)**2).sum()
        return beta

@njit
def _rolling_entropy_exact_numba(x, window):
    """ Точная реализация rolling entropy с bins='auto' для каждого окна. """
    n = len(x)
    res = np.empty(n, dtype=np.float32)
    res[:] = np.nan
    if window < 2:
        return res
    for end in range(window - 1, n):
        win = x[end - window + 1 : end + 1]
        a_min = np.min(win)
        a_max = np.max(win)
        if a_min == a_max:
            res[end] = 0.0
            continue
        sorted_win = np.sort(win)
        idx25 = int(0.25 * window)
        idx75 = int(0.75 * window)
        q25 = sorted_win[idx25]
        q75 = sorted_win[idx75]
        iqr = q75 - q25
        sturges = int(np.ceil(np.log2(window) + 1))
        if iqr > 0:
            bin_width = 2.0 * iqr / (window ** (1.0 / 3.0))
            fd = int(np.ceil((a_max - a_min) / bin_width))
        else:
            fd = 1
        nbins = max(sturges, fd, 1)
        edges = np.empty(nbins + 1, dtype=np.float32)
        step = (a_max - a_min) / nbins
        edges[0] = a_min
        for i in range(1, nbins):
            edges[i] = a_min + i * step
        edges[nbins] = a_max
        counts = np.zeros(nbins, dtype=np.int32)  # int32 достаточно для window<=1e9
        for val in win:
            idx = np.searchsorted(edges, val, side='right') - 1
            if 0 <= idx < nbins:
                counts[idx] += 1
        ent = 0.0
        total = float(window)
        for c in counts:
            if c > 0:
                p = c / total
                ent -= p * np.log(p + 1e-10)
        res[end] = ent
    return res

@jit(nopython=True)
def rolling_autocorr(arr, window):
    n = len(arr)
    result = np.full(n, 0.0)
    for i in range(n):
        start = max(0, i - window + 1)
        w = i - start + 1
        if w <= 1:
            continue
        s = arr[start: i + 1]
        a = s[1:]
        b = s[:-1]
        n_pts = w - 1
        mean_a = np.sum(a) / n_pts
        mean_b = np.sum(b) / n_pts
        cov = np.dot(a, b) / n_pts - mean_a * mean_b
        var_a = np.dot(a, a) / n_pts - mean_a * mean_a
        var_b = np.dot(b, b) / n_pts - mean_b * mean_b
        if var_a <= 0 or var_b <= 0:
            result[i] = np.nan
        else:
            std_a = np.sqrt(var_a)
            std_b = np.sqrt(var_b)
            result[i] = cov / (std_a * std_b)
    return result


# ------------------------------------------------------------------------------
@patch_feature_timings
class FeatureCalculatorForRegression:
    """
    df  -- исходный OHLCV-DataFrame.
    required_features -- список имён колонок, которые нужны модели.
    params -- { primitive_name: {... гиперпараметры ...}, 'stat_window': int }.
    """

    _PRIMITIVES = {
        "MEDPRICE":               "_feat_base",
        "MACD":                   "_feat_macd",
        "MACD_Hist":              "_feat_macd",
        "Overbought_Oversold":    "_feat_overbought",
        "Overbought_Oversold_Index_mean": "_feat_overbought",
        "Price_MADist%":          "_feat_madist",
        "Mean_Reversion":         "_feat_mean_reversion",
        "Fear_Greed":             "_feat_fear_greed",
        "perc_var_open_close":    "_feat_price_variation",
        "pmax_norm":              "_feat_pmax_ma",
        "ma_norm":                "_feat_pmax_ma",
        "ma_pmax_norm_rage":      "_feat_pmax_ma",
        "ma_pmax_norm_rage_pct":  "_feat_pmax_ma",
        "slope_trend":            "_feat_slope",
        "ema_trend":              "_feat_ema_trend",
        "hp_trend":               "_feat_hp_trend",
        "trade_bars_counter":     "_feat_trade_duration",
        "ROC":                    "_feat_roc",
        "ATR_norm":               "_feat_atr",
        "BB_Width":               "_feat_bb_width",
        "Asset_Growth":           "_feat_asset_growth",
        "ema_acceleration":       "_feat_ema_acceleration",
        "price_change":           "_feat_price_change",
        "Asset_To_Equity_Ratio":  "_feat_asset_to_equity_ratio",
        "volume_ratio":           "_feat_fear_greed_index",
        "WILLR":                  "_feat_willr",
        "kf_trend":               "_feat_kf_trend",
        "Fractal_Dim":            "_feat_fractal_dim",
        "Peak_Exhaustion_Score":  "_feat_peak_exhaustion",
        "%B_BB":                  "_feat_bb_percent",
        "Kurtosis_roll":          "_feat_kurtosis_roll",
        "OBV_div":                "_feat_obv_div",
        "RSI_slope":              "_feat_rsi_slope",
        "Vol_Decay":              "_feat_vol_decay",
        "Accel_Decay":            "_feat_accel_decay",
        "Entropy_roll":           "_feat_entropy_roll",
        "Wavelet_Var_Ratio":      "_feat_wavelet_var",
        "Autocorr_Lag1":          "_feat_autocorr",
        "Beta_Market":            "_feat_beta",
        "PSC":                    "_feat_peak_squeeze_curvature",
        "PSC_raw":                "_feat_peak_squeeze_curvature",
        "PSC_z":                  "_feat_peak_squeeze_curvature",
        "PSC_sigmoid":            "_feat_peak_squeeze_curvature",
    }

    # СТАРАЯ: r"^ago_(\d+)_"
    # НОВАЯ: умеет и "ago50_", и "ago_50_"
    _LAG_RE  = re.compile(r"^ago_?(\d+)_")
    _STAT_RE = re.compile(r"_(mean|min|max|std|skew|kurt|quantile(\d{2}))$")
    _LOGSF   = "_logsf"

    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()
        f64 = self.df.select_dtypes("float64").columns
        self.df[f64] = self.df[f64].astype(np.float32)
        if "time" in self.df:
            ts = pd.to_datetime(self.df["time"], utc=True, errors="coerce")
            self.df["hour"]        = ts.dt.hour.astype("int8")
            self.df["day_of_week"] = ts.dt.day_of_week.astype("int8")

    def calculate_features(
        self,
        required_features: Iterable[str],
        params: Mapping[str, Mapping[str, Any]] | None = None
    ) -> pd.DataFrame:
        saved_cols = ['regime', 'normalized_target', 'batch', 'time', 'open', 'close', 'high', 'low', 'volume', 'buy_signal',
                      'sell_signal', 'event_sell_time', 'event_sell_price', 'event_time', 'event_price', 'event_sell_time',
                      'event_sell_price', 'target', 'pnl', 'ma', 'pmax']
        self._params      = defaultdict(dict, params or {})
        self._stat_window = self._params.get("stat_window", 50)
        for col in required_features:
            self._ensure_column(col)
        out = self.df[list(required_features)].copy()
        f64 = out.select_dtypes("float64").columns
        out[f64] = out[f64].astype(np.float32)

        for mandatory_col in saved_cols:
            if mandatory_col in self.df.columns:
                out[mandatory_col] = self.df[mandatory_col]

        return out

    def calculate_all_possible_features(self, params: Mapping[str, Mapping[str, Any]] | None = None) -> pd.DataFrame:
        """
        Вычисляет все возможные фичи, исключая lag-версии для указанных колонок.
        Все бесконечные значения (np.inf/-np.inf) заменяются на 0.
        Порядок вычислений:
        1. Все базовые примитивы
        2. Lag-версии фич (кроме исключенных)
        3. Статистики для всех фич
        """
        # Инициализация параметров
        if not hasattr(self, '_params'):
            self._params = defaultdict(dict, params or {})
        self._stat_window = self._params.get("stat_window", 50)

        # Колонки, для которых не нужно создавать lag-версии
        EXCLUDE_FROM_LAGS = {
            'time', 'open', 'close', 'high', 'low', 'volume',
            'ma', 'pmax', 'buy_signal', 'sell_signal', 'regime',
            'event_time', 'event_price', 'event_sell_time',
            'event_sell_price', 'pnl', 'target', 'normalized_target',
            'batch', 'hour', 'day_of_week', 'trade_bars_counter'
        }

        # 1. Вычисляем все базовые примитивы
        all_primitives = list(self._PRIMITIVES.keys())
        for primitive in all_primitives:
            method_name = self._PRIMITIVES[primitive]
            primitive_params = self._params.get(primitive, {})
            try:
                getattr(self, method_name)(**primitive_params)
            except Exception as e:
                print(f"Ошибка при вычислении примитива {primitive}: {str(e)}")

        # 2. Добавляем lag-версии только для разрешенных фич
        numeric_cols = [
            col for col in self.df.select_dtypes(include=['float32', 'float64', 'int32', 'int64']).columns
            if col not in EXCLUDE_FROM_LAGS and  # Исключаем указанные колонки
            not self._LAG_RE.match(col) and      # Исключаем уже lag-фичи
            not col.endswith(self._LOGSF) and    # Исключаем logsf-фичи
            not self._STAT_RE.search(col)        # Исключаем статистики
        ]

        lag_periods = [1, 2, 3, 5, 10, 20, 50]  # Стандартные лаги

        for col in numeric_cols:
            for lag in lag_periods:
                lag_col = f"ago_{lag}_{col}"
                if lag_col not in self.df.columns:
                    self.df[lag_col] = self.df[col].shift(lag)

        # 3. Добавляем статистики для всех фич (кроме исключенных)
        all_cols_for_stats = [
            col for col in self.df.columns
            if col not in EXCLUDE_FROM_LAGS and
            not col.endswith(self._LOGSF) and
            not self._STAT_RE.search(col)
        ]

        stats = ['mean', 'std', 'min', 'max', 'skew', 'kurt']

        for col in all_cols_for_stats:
            for stat in stats:
                stat_col = f"{col}_{stat}"
                if stat_col not in self.df.columns:
                    try:
                        self._add_stat(col, stat)
                    except Exception as e:
                        print(f"Ошибка при вычислении статистики {stat} для {col}: {str(e)}")

        # 4. Добавляем logsf-версии только для разрешенных фич
        main_cols_for_logsf = [
            col for col in numeric_cols
            if not col.startswith('ago_') and
            not col.endswith(self._LOGSF) and
            col not in EXCLUDE_FROM_LAGS
        ]

        for col in main_cols_for_logsf:
            logsf_col = f"{col}{self._LOGSF}"
            if logsf_col not in self.df.columns:
                try:
                    self.df[logsf_col] = norm.logsf(self.df[col])
                except Exception as e:
                    print(f"Ошибка при вычислении logsf для {col}: {str(e)}")

        # 5. Заменяем бесконечные значения на 0
        numeric_cols_all = self.df.select_dtypes(include=['float32', 'float64', 'int32', 'int64']).columns
        self.df[numeric_cols_all] = self.df[numeric_cols_all].replace([np.inf, -np.inf], 0)

        # Сохраняем все оригинальные колонки
        for col in EXCLUDE_FROM_LAGS:
            if col in self.df.columns and col not in self.df:
                self.df[col] = self.df[col]

        return self.df.copy()

    def _ensure_column(self, name: str):
        if name in self.df:
            return

        # 1) lag-префикс "ago50_" или "ago_50_"
        m = self._LAG_RE.match(name)
        if m:
            lag  = int(m.group(1))
            base = name[m.end():]
            self._ensure_column(base)
            self.df[name] = self.df[base].shift(lag)
            return

        # 2) _logsf
        if name.endswith(self._LOGSF):
            base = name[:-len(self._LOGSF)]
            self._ensure_column(base)
            self.df[name] = norm.logsf(self.df[base])
            return

        # 3) статистический суффикс
        m = self._STAT_RE.search(name)
        if m:
            stat = m.group(1)
            base = name[:m.start()]
            self._ensure_column(base)
            self._add_stat(base, stat)
            return

        # 4) примитив
        prim = name
        if prim.startswith("Overbought_Oversold"):
            prim = "Overbought_Oversold"
        if prim.startswith("Fear_Greed"):
            prim = "Fear_Greed"
        if prim not in self._PRIMITIVES:
            raise KeyError(f"Не знаю, как получить примитив «{prim}» для «{name}»")
        getattr(self, self._PRIMITIVES[prim])(**self._params.get(prim, {}))
        if name not in self.df:
            raise RuntimeError(f"После _feat_{prim}() нет колонки «{name}»")

    def _add_stat(self, base: str, stat: str):
        col = f"{base}_{stat}"
        if col in self.df:
            return
        s = self.df[base]; w = self._stat_window
        if stat == "mean":
            self.df[col] = s.rolling(w).mean()
        elif stat == "std":
            self.df[col] = s.rolling(w).std()
        elif stat == "min":
            self.df[col] = s.rolling(w).min()
        elif stat == "max":
            self.df[col] = s.rolling(w).max()
        elif stat == "skew":
            self.df[col] = s.rolling(w).skew()
        elif stat == "kurt":
            self.df[col] = s.rolling(w).kurt()
        elif stat.startswith("quantile"):
            q = int(stat[-2:]) / 100
            self.df[col] = s.rolling(w).quantile(q)
        else:
            raise ValueError(f"Неизвестная stat «{stat}»")

    # ---------------------- ПРИМИТИВЫ ----------------------

    def _feat_base(self, medprice: int = 50):
        if "MEDPRICE" in self.df:
            return
        self.df["MEDPRICE"]      = (self.df["high"] + self.df["low"]) / 2
        self.df["MEDPRICE_std"] = self.df["MEDPRICE"].rolling(medprice).std()

    def _feat_macd(self, fast: int = 12, slow: int = 26, signal: int = 9):
        """
        Быстрый расчет нормализованного MACD с использованием векторизованных операций
        """
        if {"MACD","MACD_Hist"}.issubset(self.df.columns):
            return

        close = self.df['close']
        # Создаем множества для уникальных периодов
        ema_cache_fp = close.ewm(span=fast, adjust=False).mean()

        ema_cache_sp = close.ewm(span=slow, adjust=False).mean()
        rolling_cache = close.rolling(window=slow).mean()

        # Основной цикл вычислений
        ema_fast = ema_cache_fp
        ema_slow = ema_cache_sp
        rolling_mean = rolling_cache
        macd = ema_fast - ema_slow
        macd_norm = macd / rolling_mean
        self.df[f'MACD'] = macd_norm
        signal = macd.ewm(span=signal, adjust=False).mean()
        signal_norm = signal / rolling_mean

        # Сохраняем результаты
        self.df[f'MACD_Hist'] = macd_norm - signal_norm

    '''def _feat_macd(self, fast: int = 12, slow: int = 26, signal: int = 9):
        if {"MACD","MACD_Hist"}.issubset(self.df.columns):
            return
        c  = self.df["close"]
        ef = c.ewm(span=fast, adjust=False).mean()
        es = c.ewm(span=slow, adjust=False).mean()
        macd = (ef - es) / (c.rolling(slow).mean().add(1e-10))
        sig  = macd.ewm(span=signal, adjust=False).mean()
        self.df["MACD"]      = macd
        self.df["MACD_Hist"] = macd - sig'''

    def _feat_overbought(self, rsi_p: int = 14, stoch_p: int = 14):
        name = "Overbought_Oversold_Index"
        if name in self.df:
            return
        c   = self.df["close"]; d = c.diff()
        g   = d.clip(lower=0); l = (-d).clip(lower=0)
        rs  = g.rolling(rsi_p).mean() / (l.rolling(rsi_p).mean().add(1e-10))
        rsi = 100 - (100 / (1 + rs))
        lo  = self.df["low"].rolling(stoch_p).min()
        hi  = self.df["high"].rolling(stoch_p).max()
        st  = 100*(c - lo)/(hi - lo + 1e-10)
        self.df[name] = (rsi + st)/2

    def _feat_madist(self, span_lenght: int = 200):
        name = "Price_MADist%"
        if name in self.df:
            return
        ema = self.df["close"].ewm(span=span_lenght, adjust=False).mean()
        self.df[name] = (self.df["close"]/ema - 1)*100

    def _feat_mean_reversion(self, window: int = 20):
        name = "Mean_Reversion"
        if name in self.df:
            return
        ma = self.df["close"].rolling(window).mean()
        self.df[name] = self.df["close"] - ma

    def _feat_fear_greed(self, window: int = 14):
        name = "Fear_Greed_Index"
        if name in self.df:
            return
        v  = self.df["close"].pct_change().rolling(window).std()
        vc = self.df["volume"].pct_change().rolling(window).mean()
        tr = self.df["close"]/self.df["close"].rolling(window).mean()
        self.df[name] = (v + vc + tr)/3*100

    def _feat_price_variation(self):
        name = "perc_var_open_close"
        if name in self.df:
            return
        eps = 1e-10
        self.df[name] = (self.df["close"]-self.df["open"])/(self.df["open"]+eps)*100

    def _feat_pmax_ma(self,
        pmax_ma_length: int = 10,
        pmax_ma_length_roll: int = 50,
        pct_window: int = 5
    ):
        need = {
            "pmax_norm", "ma_norm",
            "ma_pmax_norm_rage", "ma_pmax_norm_rage_pct"
        }
        if need.issubset(self.df.columns):
            return
        if {"pmax","ma"}.difference(self.df.columns):
            raise ValueError("Нужны 'pmax' и 'ma'")
        c = self.df["close"]
        self.df["pmax_norm"]             = (c-self.df["pmax"])/self.df["pmax"]
        self.df["ma_norm"]               = (c-self.df["ma"])/self.df["ma"]
        self.df["ma_pmax_norm_rage"]     = self.df["ma_norm"] - self.df["pmax_norm"]
        # новый примитив — pct-динамика
        self.df["ma_pmax_norm_rage_pct"] = \
          self.df["ma_pmax_norm_rage"].pct_change(pct_window).fillna(0)

    def _feat_slope(self, slope_lag: int = 300, pct_window: int = 6):
        name = "slope_trend"
        if name in self.df:
            return
        r = self.df["close"].pct_change(pct_window).fillna(0)
        self.df[name] = r.rolling(slope_lag, min_periods=slope_lag)\
                         .apply(_slope, raw=True)

    def _feat_ema_trend(self, span: int = 300, pct_window: int = 6):
        name = "ema_trend"
        if name in self.df:
            return
        r = self.df["close"].pct_change(pct_window).fillna(0)
        e = r.ewm(span=span, adjust=False).mean()
        self.df[name] = e.diff().fillna(0)

    def _feat_asset_to_equity_ratio(self):
        """
        Вычисление коэффициента соотношения активов и собственного капитала.
        """
        name = "Asset_To_Equity_Ratio"
        asset = self.df['close']
        equity = self.df['low']
        # Добавляем в DataFrame
        self.df[name] = asset / (equity + 1e-10)

    def _feat_hp_trend(self, lamb: float = 1600):
        name = "hp_trend"
        if name in self.df:
            return
        y    = np.log(self.df["close"]).fillna(method="ffill")
        coef = lamb/(1+lamb)
        tr   = np.empty(len(y), dtype=float)
        tr[0] = y.iloc[0]
        for i in range(1, len(y)):
            tr[i] = coef*y.iloc[i] + (1-coef)*tr[i-1]
        self.df[name] = np.append([0], np.diff(tr))

    def _feat_kf_trend(self,
        pct_window: int = 6,
        obs_var: float = 1e-4, # σ² ε_t (шум наблюдения)
        level_var: float = 1e-5 # σ² η_t (шум уровня)
        ) -> pd.DataFrame:
        """
        Добавляет к DataFrame колонки:
        kf_trend — one-sided Калман-оценка тренда доходностей
        kf_trend_logsf — лог-survival-function (z-score) тренда
        Полностью каузально, обновляется тик-за-тиком.
        """

        name = 'kf_trend'
        # 1. Доходности
        r = self.df['close'].pct_change(pct_window).fillna(0)
        # 2. Local-level модель: y_t = μ_t + ε_t ;  μ_t = μ_{t-1} + η_t
        mod = sm.tsa.UnobservedComponents(r, level='llevel')

        # 3. Параметры модели в log-шкале (требование statsmodels)
        params = np.log([obs_var, level_var])

        # 4. Только forward-filter → нет look-ahead bias
        res = mod.filter(params)                       # <— односторонний Калман
        trend = pd.Series(res.filtered_state[0], index=self.df.index)

        # 5. Запись результата
        self.df['kf_trend'] = trend

    def _feat_willr(self, window=14):
        """
        Вычисление %R по методу Уильямса (WILLR).
        """
        name = 'WILLR'
        high = self.df['high']
        low = self.df['low']
        close = self.df['close']

        highest_high = high.rolling(window).max()
        lowest_low = low.rolling(window).min()

        willr = ((highest_high - close) / (highest_high - lowest_low)) * -100

        # Добавляем в DataFrame
        self.df[name] = willr

    def _feat_fear_greed_index(self, window: int = 14):
        """
        Расчет объема как отношение последнего объема к скользящему среднему.
        """
        name = "volume_ratio"
        if name in self.df:
            return
        s = self.df["volume"]
        self.df[name] = s / s.rolling(window).mean()

    def _feat_trade_duration(self) -> pd.DataFrame:
        """
        Добавляет в self.df колонку 'trade_bars_counter' — количество прошедших баров
        с момента входа в текущую открытую сделку до каждой строки self.df (включая строку).
        Если открытой сделки нет — NaN.

        Требования к self.df:
          - self.df['time'] — время начала свечи (строки в ISO формате, совпадают со шкалой time_last_kline_start[ticker])
          - self.df отсортирован по времени по возрастанию
        """
        def _parse_to_timestamp(value):
            # Используем pd.to_datetime для унифицированного парсинга ISO строк в UTC Timestamp
            return pd.to_datetime(value, utc=True)

        name = "trade_bars_counter"
        if name in self.df.columns:
            return self.df

        n = len(self.df)
        if n == 0:
            self.df[name] = np.empty(0, dtype=np.float32)
            return self.df

        # Получаем текущую открытую сделку (предполагаем open_trades и ticker доступны)
        od = open_trades.get(ticker)
        has_open = bool(od and od.get("status") == "open" and od.get("buy_time"))
        if not has_open:
            self.df[name] = np.full(n, np.nan, dtype=np.float32)
            return self.df

        # Парсим buy_time в Timestamp
        try:
            buy_ts = _parse_to_timestamp(od["buy_time"])
        except ValueError:
            # Если ошибка парсинга, установить NaN
            self.df[name] = np.full(n, np.nan, dtype=np.float32)
            return self.df

        # Получаем эталонную шкалу времени (ISO строки)
        times_iso = time_last_kline_start.get(ticker, [])
        if not times_iso:
            self.df[name] = np.full(n, np.nan, dtype=np.float32)
            return self.df

        # Парсим шкалу в список Timestamp
        try:
            dt_scale = [_parse_to_timestamp(s) for s in times_iso]
        except ValueError:
            self.df[name] = np.full(n, np.nan, dtype=np.float32)
            return self.df

        # Найти индекс входа: первая свеча с ts >= buy_ts (bisect_left работает с Timestamp)
        entry_idx_scale = bisect_left(dt_scale, buy_ts)

        if entry_idx_scale >= len(dt_scale):
            self.df[name] = np.full(n, np.nan, dtype=np.float32)
            return self.df

        # Парсим времена из df (строки ISO)
        try:
            df_times = [_parse_to_timestamp(t) for t in self.df['time']]
        except ValueError:
            self.df[name] = np.full(n, np.nan, dtype=np.float32)
            return self.df

        # Построим маппинг: время -> индекс в общей шкале
        scale_index_by_time = {dt_scale[i]: i for i in range(len(dt_scale))}

        # Для каждой строки df найдём её индекс в шкале и вычислим счётчик
        out = np.full(n, np.nan, dtype=np.float32)

        for i, t in enumerate(df_times):
            idx_in_scale = scale_index_by_time.get(t)
            if idx_in_scale is None:
                continue
            delta = idx_in_scale - entry_idx_scale + 1
            if delta >= 1:
                out[i] = float(delta)

        self.df[name] = out

    def _feat_roc(self, window: int = 5):
        name = "ROC"
        if name in self.df:
            return
        self.df[name] = self.df["close"].pct_change(window)

    def _feat_atr(self, atr_window: int = 14):
        name = "ATR_norm"
        if name in self.df:
            return
        h,l,c = self.df["high"], self.df["low"], self.df["close"]
        tr1 = h-l
        tr2 = (h-c.shift()).abs()
        tr3 = (l-c.shift()).abs()
        tr  = pd.concat([tr1,tr2,tr3], axis=1).max(axis=1)
        atr = tr.rolling(atr_window).mean()
        self.df[name] = atr/c

    def _feat_bb_width(self, bb_window: int = 20):
        name = "BB_Width"
        if name in self.df:
            return
        c   = self.df["close"]
        ma  = c.rolling(bb_window).mean()
        std = c.rolling(bb_window).std()
        self.df[name] = 2*std/ma

    def _feat_asset_growth(self, window: int = 3):
        name = "Asset_Growth"
        if name in self.df:
            return
        self.df[name] = self.df["close"].pct_change(window).fillna(0)*100

    def _feat_ema_acceleration(self, pct_window: int = 3, ema_window: int = 300):
        name = "ema_acceleration"
        if name in self.df:
            return
        r = self.df["close"].pct_change(pct_window).fillna(0)
        e = r.ewm(span=ema_window).mean()
        self.df[name] = e.diff(4)

    def _feat_price_change(self, window: int = 1):
        name = "price_change"
        if name in self.df:
            return
        self.df[name] = self.df["close"].pct_change(window).fillna(0)

    def _feat_peak_exhaustion(
        self,
        price_win: int = 60,    # окно "локального максимума"
        mom_win:   int = 10,    # окно для momentum
        vol_win:   int = 20,
        atr_win:   int = 14,
        z_win:     int = 100    # z-score нормализация
    ):
        """
        Peak-Exhaustion Score  ~ 0…1
        1 → почти наверху, импульс затух, объём падает, ATR высок.
        """
        name = "Peak_Exhaustion_Score"
        c = self.df["close"]

        # 1) расстояние до локального max
        roll_max = c.rolling(price_win).max()
        dist_max = (roll_max - c) / roll_max          # 0 — на max, >0 — ниже

        # 2) ослабевающий импульс
        roc_now  = c.pct_change(mom_win)
        roc_hist = roc_now.rolling(price_win).max()   # max импульса в окне
        momentum_div = 1 - (roc_now / (roc_hist + 1e-12))   # 0 → свежий high

        # 3) сушащийся объём
        vol_ratio = self.df["volume"] / \
            self.df["volume"].rolling(vol_win).mean()

        # 4) расширенный спред (ATR/price)
        tr  = pd.concat([
                self.df["high"]  - self.df["low"],
                (self.df["high"] - c.shift()).abs(),
                (self.df["low"]  - c.shift()).abs()
            ], axis=1).max(axis=1)
        atr = tr.rolling(atr_win).mean()
        atr_norm = atr / c

        # 5) агрегируем, переводим в z-score, squash σ → 0…1
        raw = (dist_max + momentum_div + (1/vol_ratio) + atr_norm) / 4
        z   = (raw - raw.rolling(z_win).mean()) / (raw.rolling(z_win).std() + 1e-9)
        self.df[name] = 1 / (1 + np.exp(-z))   # σ(z)

    def _feat_fractal_dim(self, short_win=20, long_win=40):
        """Вычисляет фрактальную размерность на основе отношения ATR разных периодов"""
        name = "Fractal_Dim"
        if name in self.df:
            return

        # Вычисляем ATR для короткого периода
        h, l, c = self.df['high'], self.df['low'], self.df['close']
        tr1 = h - l
        tr2 = (h - c.shift()).abs()
        tr3 = (l - c.shift()).abs()
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        atr_short = tr.rolling(short_win).mean()

        # Вычисляем ATR для длинного периода
        atr_long = tr.rolling(long_win).mean()

        # Вычисляем фрактальную размерность
        ratio = atr_long / (atr_short + 1e-10)  # Добавляем небольшое значение для избежания деления на 0
        self.df[name] = np.log(ratio) / np.log(2)

    def _feat_bb_percent(self, window=20, std_mult=2):
        name = "%B_BB"
        if name in self.df: return
        ma = self.df["close"].rolling(window).mean()
        std = self.df["close"].rolling(window).std()
        self.df[name] = (self.df["close"] - (ma - std_mult * std)) / (4 * std)

    def _feat_kurtosis_roll(self, window=50):
        name = "Kurtosis_roll"
        if name in self.df: return
        ret = self.df["close"].pct_change().fillna(0)
        self.df[name] = ret.rolling(window).kurt()

    def _feat_obv_div(self, window=10):
        name = "OBV_div"
        if name in self.df: return
        sign = np.sign(self.df["close"].diff())
        obv = (sign * self.df["volume"]).cumsum()
        price_chg = self.df["close"].pct_change(window)
        obv_chg = obv.pct_change(window)
        self.df[name] = price_chg - obv_chg

    def _feat_rsi_slope(self, rsi_p=14, diff_win=5):
        name = "RSI_slope"
        if name in self.df: return
        delta = self.df["close"].diff()
        gain = delta.clip(lower=0).rolling(rsi_p).mean()
        loss = -delta.clip(lower=0).rolling(rsi_p).mean()
        rsi = 100 - 100 / (1 + gain / (loss + 1e-10))
        self.df[name] = rsi.diff(diff_win)

    def _feat_vol_decay(self, window=20):
        name = "Vol_Decay"
        if name in self.df: return
        vol_ema = self.df["volume"].ewm(span=window).mean()
        self.df[name] = self.df["volume"] / vol_ema - 1

    def _feat_accel_decay(self, window=10):
        name = "Accel_Decay"
        if name in self.df: return
        vel = self.df["close"].diff(window)
        accel = vel.diff(window)
        self.df[name] = accel / (vel.abs() + 1e-10)

    '''def _feat_entropy_roll(self, window=50):
        name = "Entropy_roll"
        if name in self.df:
            return

        ret = self.df["close"].pct_change().fillna(0)

        def _entropy(x):
            if len(x) < 2:
                return 0
            hist = np.histogram(x, bins='auto')[0]
            hist = hist / hist.sum()  # Нормализуем
            return -np.sum(hist * np.log(hist + 1e-10))

        self.df[name] = ret.rolling(window).apply(_entropy, raw=True)'''

    def _feat_entropy_roll(self, window: int = 50):
        name = "Entropy_roll"
        if name in self.df:
            return
        ret = self.df["close"].pct_change().fillna(0.0).to_numpy(dtype=np.float32)
        ent = _rolling_entropy_exact_numba(ret, window)
        self.df[name] = ent

    def _feat_wavelet_var(self, short_win=10, long_win=50):
        name = "Wavelet_Var_Ratio"
        if name in self.df:
            return
        ret = self.df["close"].pct_change().fillna(0)
        var_short = ret.rolling(short_win).var()
        var_long = ret.rolling(long_win).var()
        self.df[name] = var_short / (var_long + 1e-10)

    '''def _feat_autocorr(self, window=50):
        name = "Autocorr_Lag1"
        if name in self.df:
            return
        ret = self.df["close"].pct_change().fillna(0)
        self.df[name] = ret.rolling(window).apply(lambda x: x.autocorr(lag=1) if len(x) > 1 else 0, raw=False)'''

    def _feat_autocorr(self, window=50):
        name = "Autocorr_Lag1"
        if name in self.df:
            return
        ret = self.df["close"].pct_change().fillna(0)
        arr = ret.to_numpy()  # Use to_numpy() for compatibility
        autocorrs = rolling_autocorr(arr, window)
        self.df[name] = autocorrs

    def _feat_beta(self, window=50):
        name = "Beta_Market"
        if name in self.df or "market_close" not in self.df:
            return
        ret_stock = self.df["close"].pct_change().fillna(0)
        ret_market = self.df["market_close"].pct_change().fillna(0)
        cov = ret_stock.rolling(window).cov(ret_market)
        var_market = ret_market.rolling(window).var()
        self.df[name] = cov / (var_market + 1e-10)

    def _feat_peak_squeeze_curvature(self,
                                vel_win: int = 5,
                                acc_win: int = 5,
                                vol_win: int = 20,
                                atr_win: int = 14,
                                z_win : int = 60):
        """
        Возвращает 3 колонки:
        PSC_raw, PSC_z, PSC_sigmoid ∈ [0,1]
        """
        name = "PSC"
        cols_need = {"PSC_raw","PSC_z","PSC_sigmoid"}
        if cols_need.issubset(self.df.columns): return
        c = self.df['close']

        # 1) speed & accel
        speed  = c.pct_change(vel_win).fillna(0)
        accel  = speed.diff(acc_win).fillna(0)
        curvature = accel / (speed.abs() + 1e-10)

        # 2) squeeze = ATR_norm ↘ & HV_norm ↘
        h,l = self.df['high'], self.df['low']
        tr = pd.concat([h-l, (h-c.shift()).abs(), (l-c.shift()).abs()], axis=1).max(axis=1)
        atr = tr.rolling(atr_win).mean()
        hv  = c.pct_change().rolling(vol_win).std()
        squeeze = - (atr / (c+1e-10)).diff().clip(upper=0)   # падение ATR
        squeeze += - hv.diff().clip(upper=0)                 # падение HV
        squeeze /= 2

        # 3) агрегируем
        raw = 0.6*curvature + 0.4*squeeze

        # 4) z-score + σ(z)
        mu  = raw.rolling(z_win).mean()
        std = raw.rolling(z_win).std()
        z = (raw - mu)/(std + 1e-9)
        sigm = 1/(1+np.exp(-z))

        self.df[f'{name}_raw']     = raw
        self.df[f'{name}_z']       = z.clip(-5, 5)
        self.df[f'{name}_sigmoid'] = sigm


def calculate_metrics(test_data, y_test, y_pred, target_column='normalized_target'):
    """
    Функция для расчета метрик по данным теста и предсказаниям модели.

    Параметры:
    - test_data: pd.DataFrame — тестовые данные с колонками batch, high, close и другими.
    - y_test: pd.Series или np.array — фактические значения целевой переменной.
    - y_pred: pd.Series или np.array — предсказанные моделью значения.
    - target_column: str — название колонки целевой переменной в test_data.

    Возвращает:
    - avg_mse: float — среднеквадратическая ошибка.
    - avg_r2: float — средняя R².
    - std_r2: float — стандартное отклонение R².
    - corr_mean: float — средняя корреляция.
    - corr_std: float — стандартное отклонение корреляции.
    - avg_missed: float — средний процент упущенной прибыли.
    """
    # Инициализация метрик
    mse_scores, r2_scores, corr_scores, missed_pnl = [], [], [], []

    # Расчет корреляции целевой переменной и предсказаний
    y_pred_series = pd.Series(y_pred, index=y_test.index)
    corr_score = test_data[target_column].corr(y_pred_series)
    corr_scores.append(corr_score)

    # MSE и R2
    mse_scores.append(mean_squared_error(y_test, y_pred))
    r2_scores.append(r2_score(y_test, y_pred))

    # Расчет missed_pnl для каждого batch
    """for batch in test_data['batch'].unique():
        mask = test_data['batch'] == batch
        max_high = test_data.loc[mask, 'high'].max()
        pred = y_pred[mask]  # предполагается, что y_pred соответствует normalized_target
        sell_idx = np.argmin(pred)  # продажа на минимальном предсказанном значении
        sell_price = test_data.loc[mask].iloc[sell_idx]['close']
        missed = (max_high - sell_price) / (max_high - test_data.loc[mask].iloc[0]['close'])  # % упущенной прибыли
        missed_pnl.append(missed)"""

    # Усреднение метрик
    avg_mse = float(np.mean(mse_scores))
    avg_r2 = float(np.mean(r2_scores))
    std_r2 = float(np.std(r2_scores))
    corr_mean = float(np.mean(corr_scores))
    corr_std = float(np.std(corr_scores))
    #avg_missed = float(np.mean(missed_pnl))

    # Проверка на корректность результатов
    if np.isfinite([avg_mse, avg_r2, std_r2, corr_mean, corr_std]).all(): #avg_missed
        return avg_mse, avg_r2, std_r2, corr_mean, corr_std, #avg_missed
    else:
        return float('inf'), float('inf'), float('inf'), float('inf'), float('inf'), float('inf')

def prepare_data(df, target_col):
    """
    Подготавливает данные: разделяет на числовые и категориальные признаки, создает конвейер преобразования.
    """
    if type(target_col) == str:
        df.dropna(inplace=True)
        X = df.drop([target_col, 'batch'], axis=1)
        y = df[target_col]
    elif type(target_col) == list:
        df.dropna(inplace=True)
        X = df.drop(target_col+['batch'], axis=1)
        y = df[target_col]

    # Разделение на числовые и категориальные признаки
    numeric_features = X.select_dtypes(include=['int64', 'float64', 'float32', 'int32']).columns
    categorical_features = X.select_dtypes(include=['object']).columns

    # Создание конвейера преобразования
    preprocessing = ColumnTransformer(
        transformers=[
            ('num', Pipeline([
                ('scaler', RobustScaler()),
                ('normalize', PowerTransformer(method='yeo-johnson')),
            ]), numeric_features),
            ('cat', Pipeline([
                ('onehot', OneHotEncoder(handle_unknown='ignore')),
            ]), categorical_features)
        ]
    )

    return X, y, preprocessing


def calculate_indicators(df, features, params=None, mode=None, multy=False):

    fc = FeatureCalculatorForRegression(df)
    if mode==None:
        df_features = fc.calculate_features(params=params, required_features=features)
    else:
        df_features = fc.calculate_all_possible_features()
    if multy == False:
        df1 = df_features[df_features['normalized_target'].notna()]
    else:
        df1 = df_features[df_features['multi_target_5'].notna()]
    features = ['open', 'close', 'high', 'low', 'volume', 'buy_signal', 'sell_signal','event_sell_time','event_sell_price',
                'event_time','event_price','event_sell_time','event_sell_price','target', 'pnl', 'ma', 'pmax'] #time
    df1['regime'] = df1['regime'].astype('object')
    df1[['batch','trade_bars_counter']] = df1[['batch', 'trade_bars_counter']].astype('int')
    df1 = df1.drop(features, axis=1)
    #df1 = df1.dropna()
    return df1, fc._timings

def sample_feature_params(params) -> dict:
        """
        Draws *one* sample of the whole feature-engineering hyper-parameter set.
        Rule of thumb for ranges:
          • lower bound = ‘sane minimum‘ from domain knowledge
          • upper bound = ‘sane maximum’
        Adjust them if you feel the search space is too wide or too narrow.
        """
        # ---- helpers for monotone constraints ----------------------------------
        fast  = params['macd_fast']
        slow  = params['macd_slow']

        slope_lag_min = params['slope_lag_min']
        slope_lag     = params['slope_lag']

        # ---- finally compose the nested dict -----------------------------------
        return {
            'base': {
                'medprice': params['medprice']
            },
            'macd': {
                'fast'      : fast,
                'slow'      : slow,
                'signal'    : params['macd_signal'],
                'macd_roll' : params['macd_roll']
            },
            'overbought': {
                'rsi_p'         : params['rsi_p'],
                'stoch_p'       : params['stoch_p'],
                'oversold_roll' : params['oversold_roll']
            },
            'madist': {
                'span_lenght'   : params['madist_span'],
                'madist_lenght' : params['madist_len']
            },
            'mean_reversion': {
                'window' : params['mr_window']
            },
            'fear_greed': {
                'greed_pct'    : params['fg_greed_pct'],
                'volume_ratio_scr' : params['fg_vol_ratio'],
                'window'       : params['fg_window'],
                'greed_roll'   : params['fg_roll']
            },
            'price_variation': {
                'variation_lenght': params['pv_len']
            },
            'pmax_ma': {
                'pmax_ma_lenght'      : params['pmax_len'],
                'pmax_ma_lenght_roll' : params['pmax_roll']
            },
            'slope': {
                'slope_lag'     : slope_lag,
                'slope_lag_min' : slope_lag_min,
                'sloap_pct'     : params['slope_pct'],
                'sloap_roll'    : params['slope_roll']
            },
            # _trade_duration_features – no params
        }

def build_feature_params(
    flat_params: Dict[str, Any],
    extra_alias: Optional[Dict[str, Tuple[str, str | None]]] = None
) -> Dict[str, Dict[str, Any]]:
    """
    Преобразует «плоский» словарь от Optuna в структуру,
    которую понимает FeatureCalculatorForRegression.
    """

    # 1. Базовая явная таблица соответствий
    alias: Dict[str, Tuple[str, str | None]] = {
        'hp_lamb'          : ('hp_trend'           , 'lamb'),
        'ea_pct'           : ('ema_acceleration'   , 'pct_window'),
        'ea_ema'           : ('ema_acceleration'   , 'ema_window'),
        'mr_window'        : ('Mean_Reversion'     , 'window'),
        'ag_window'        : ('Asset_Growth'       , 'window'),
        'medprice'         : ('MEDPRICE'           , 'medprice'),
        'bb_window'        : ('BB_Width'           , 'bb_window'),
        'macd_fast'        : ('MACD'               , 'fast'),
        'macd_slow'        : ('MACD'               , 'slow'),
        'macd_signal'      : ('MACD'               , 'signal'),
        'fg_window'        : ('Fear_Greed'         , 'window'),
        'atr_window'       : ('ATR_norm'           , 'atr_window'),
        'vr_window'        : ('volume_ratio'       , 'window'),
        'madist_span'      : ('Price_MADist%'      , 'span_lenght'),
        'slope_lag'        : ('slope_trend'        , 'slope_lag'),
        'slope_lag_min'    : ('slope_trend'        , 'slope_lag'),
        'rsi_p'            : ('Overbought_Oversold', 'rsi_p'),
        'stoch_p'          : ('Overbought_Oversold', 'stoch_p'),
        'pmax_len'         : ('pmax_norm'          , 'pmax_ma_length'),
        'pmax_roll'        : ('pmax_norm'          , 'pmax_ma_length_roll'),
        'pc_window'        : ('pmax_norm'          , 'pct_window'),
        'ema_trend_span'   : ('ema_trend'          , 'span'),
        'ema_trend_pct'    : ('ema_trend'          , 'pct_window'),
        'stat_window'      : ('stat_window', None),
        # --- новые алиасы ---
        'roc_window'        : ('ROC'          , 'window'),
        'willr_window'      : ('WILLR'          , 'window'),
        'fractal_short_win' : ('Fractal_Dim', 'short_win'),
        'fractal_long_win'  : ('Fractal_Dim', 'long_win'),
        'peak_price_win'    : ('Peak_Exhaustion_Score', 'price_win'),
        'peak_mom_win'      : ('Peak_Exhaustion_Score', 'mom_win'),
        'peak_vol_win'      : ('Peak_Exhaustion_Score', 'vol_win'),
        'peak_atr_win'      : ('Peak_Exhaustion_Score', 'atr_win'),
        'peak_z_win'        : ('Peak_Exhaustion_Score', 'z_win'),
        'bb_window'         : ('%B_BB', 'window'),
        'bb_std_mult'       : ('%B_BB', 'std_mult'),
        'kurt_window'       : ('Kurtosis_roll', 'window'),
        'obv_window'        : ('OBV_div', 'window'),
        'rsi_slope_rsi_p'   : ('RSI_slope', 'rsi_p'),
        'rsi_diff_win'      : ('RSI_slope', 'diff_win'),
        'voldec_window'     : ('Vol_Decay', 'window'),
        'acceldec_window'   : ('Accel_Decay', 'window'),
        'ent_window'        : ('Entropy_roll', 'window'),
        'wlt_short_win'     : ('Wavelet_Var_Ratio', 'short_win'),
        'wlt_long_win'      : ('Wavelet_Var_Ratio', 'long_win'),
        'acorr_window'      : ('Autocorr_Lag1', 'window'),
        'beta_window'       : ('Beta_Market', 'window'),
        'psc_vel_win'       : ('PSC', 'vel_win'),
        'psc_acc_win'       : ('PSC', 'acc_win'),
        'psc_vol_win'       : ('PSC', 'vol_win'),
        'psc_atr_win'       : ('PSC', 'atr_win'),
        'psc_z_win'         : ('PSC', 'z_win'),
    }

    # 2. Пользовательские переопределения
    if extra_alias:
        alias.update(extra_alias)

    # 3. Автоматический разбор префиксов (fallback)
    prefix_map: Dict[str, str] = {
        'macd'        : 'MACD',
        'hp'          : 'hp_trend',
        'ea'          : 'ema_acceleration',
        'mr'          : 'Mean_Reversion',
        'ag'          : 'Asset_Growth',
        'bb'          : 'BB_Width',
        'fg'          : 'Fear_Greed',
        'atr'         : 'ATR_norm',
        'vr'          : 'volume_ratio',
        'madist'      : 'Price_MADist%',
        'slope'       : 'slope_trend',
        'pmax'        : 'pmax_norm',
        'ema_trend'   : 'ema_trend',
        'rsi'         : 'Overbought_Oversold',
        'stoch'       : 'Overbought_Oversold',
        'fractal'     : 'Fractal_Dim',
        'peak'        : 'Peak_Exhaustion_Score',
        'bb'          : '%B_BB',
        'kurt'        : 'Kurtosis_roll',
        'obv'         : 'OBV_div',
        'rsi_slope'   : 'RSI_slope',
        'voldec'      : 'Vol_Decay',
        'acceldec'    : 'Accel_Decay',
        'entropy'     : 'Entropy_roll',
        'wavelet'     : 'Wavelet_Var_Ratio',
        'acorr'       : 'Autocorr_Lag1',
        'beta'        : 'Beta_Market',
        'psc'         : 'PSC',
    }

    nested: Dict[str, Dict[str, Any]] = defaultdict(dict)

    for key, val in flat_params.items():

        # 3.1 Явное соответствие
        if key in alias:
            prim, arg = alias[key]
            if prim == 'stat_window' or arg is None:
                nested['stat_window'] = val
            else:
                nested[prim][arg] = val
            continue

        # 3.2 Игнорируем вспомогательные ключи вида *_min, *_max, если
        #     они не нужны никакому примитиву.
        if key.endswith('_min') or key.endswith('_max'):
            continue

        # 3.3 Fallback-разбор _
        if '_' in key:
            prefix, arg = key.split('_', 1)
            if prefix in prefix_map:
                nested[prefix_map[prefix]][arg] = val
                continue

        # 3.4 Неизвестный ключ — игнорируем или логируем
        # print(f'Warning: parameter "{key}" was not mapped')

    return {p: d for p, d in nested.items()}

_ORIG_INTERP = F.interpolate


def _collapse_pred_to_bt(y_pred: torch.Tensor) -> torch.Tensor:
    """
    Приводим предсказание к виду (B, T), считая последнюю ось временем (горизонтом).
    Все промежуточные оси (кроме batch=ось 0 и time=последняя ось) усредняем.
    Пример: (B, 1, 4, 10) -> mean по осям (1,2) -> (B,10)
    (B, 10) -> ок
    (B, 1, 10) -> squeeze -> (B,10)
    """
    if not isinstance(y_pred, torch.Tensor):
        raise TypeError(f"y_pred must be a tensor, got {type(y_pred)}")

    # Сначала уберём все единичные оси
    if any(s == 1 for s in y_pred.shape[1:-1]):
        # squeeze не трогает последнюю ось, если она не равна 1
        y_pred = y_pred.squeeze()
        # Если squeeze убрал не только единичные, но и привёл к (B, T) — хорошо.

    if y_pred.dim() == 1:
        # (B,) — интерпретируем как T=1, сделаем (B,1)
        y_pred = y_pred.unsqueeze(-1)
        return y_pred

    if y_pred.dim() == 2:
        # (B, T) — уже как надо
        return y_pred

    # Если размерностей больше 2: считаем last dim = time, batch = 0
    # Все промежуточные оси схлопываем усреднением
    reduce_dims = tuple(range(1, y_pred.dim() - 1))
    if len(reduce_dims) > 0:
        y_pred = y_pred.mean(dim=reduce_dims)
    # На выходе (B, T)
    if y_pred.dim() != 2:
        # На всякий случай добьёмся (B, T)
        y_pred = y_pred.view(y_pred.size(0), -1)
    return y_pred


def _install_safe_interpolate_patch():
    """
    Патч делает F.interpolate детерминированным при включённом torch.use_deterministic_algorithms(True)
    для CUDA и режимов linear/bilinear/bicubic, прогоняя вычисление на CPU.
    Идемпотентен и не меняет сигнатуру.
    """
    if getattr(F.interpolate, "_is_deterministic_wrapper", False):
        return

    _orig_interpolate = F.interpolate

    # Какие режимы считаем потенциально недетерминируемыми на CUDA
    _CUDA_UNSAFE_MODES = {"linear", "bilinear", "bicubic"}  # 1d/2d/2d

    def _needs_cpu_fallback(input, mode):
        if not torch.is_tensor(input):
            return False
        if input.is_cuda and mode in _CUDA_UNSAFE_MODES:
            # upsample_linear1d_backward_out_cuda и др. — недетерминируемы
            return True
        return False

    @functools.wraps(_orig_interpolate)
    def _deterministic_interpolate(
        input: torch.Tensor,
        size=None,
        scale_factor=None,
        mode="nearest",
        align_corners=None,
        recompute_scale_factor=None,
        antialias=False,
    ):
        # Если не нужно, просто вызовем оригинал
        if not _needs_cpu_fallback(input, mode):
            return _orig_interpolate(
                input,
                size=size,
                scale_factor=scale_factor,
                mode=mode,
                align_corners=align_corners,
                recompute_scale_factor=recompute_scale_factor,
                antialias=antialias,
            )

        # CUDA + linear/bilinear/bicubic → CPU fallback
        x = input
        dev = x.device
        orig_dtype = x.dtype

        # Для стабильности переводим в float32 на CPU
        x_cpu = x.detach().to("cpu", dtype=torch.float32).requires_grad_(x.requires_grad)

        y_cpu = _orig_interpolate(
            x_cpu,
            size=size,
            scale_factor=scale_factor,
            mode=mode,
            align_corners=align_corners,
            recompute_scale_factor=recompute_scale_factor,
            antialias=antialias,
        )

        # Возвращаем на исходное устройство и тип
        y = y_cpu.to(dev, dtype=orig_dtype)

        return y

    _deterministic_interpolate._is_deterministic_wrapper = True  # type: ignore[attr-defined]
    F.interpolate = _deterministic_interpolate


_install_safe_interpolate_patch()


def _unpack_pf_batch(batch):
    """
    Унифицированная распаковка батча из TimeSeriesDataSet.to_dataloader(...)
    Возвращает: x (dict), y (Tensor|None), weight (Tensor|None)
    """
    if isinstance(batch, (list, tuple)):
        if len(batch) == 3:
            x, y, weight = batch
        elif len(batch) == 2:
            x, y = batch
            weight = None
        else:
            raise ValueError(f"Unexpected batch tuple length: {len(batch)}")
    elif isinstance(batch, dict):
        # На всякий случай поддержим dict → возьмём таргет из decoder_target, если есть
        x = batch
        y = batch.get("decoder_target", None)
        weight = None
    else:
        raise TypeError(f"Unexpected batch type: {type(batch)}")
    return x, y, weight


# F.interpolate = _deterministic_interpolate


def _extract_pred_tensor(y_pred):
    # извлекаем тензор предикта из любых обёрток
    if isinstance(y_pred, dict):
        for key in ("prediction", "output", "decoder_output"):
            if key in y_pred and torch.is_tensor(y_pred[key]):
                return y_pred[key]
        # если не нашли — попробуем fallback: первый тензор в dict
        for v in y_pred.values():
            if torch.is_tensor(v):
                return v
        raise ValueError("Could not extract prediction tensor from dict y_pred.")

    if isinstance(y_pred, (list, tuple)):
        # обычно y_pred[0] — предсказание
        return y_pred[0]

    if torch.is_tensor(y_pred):
        return y_pred

    raise TypeError(f"Unsupported y_pred type: {type(y_pred)}")


class EventTimeSeriesSplit(BaseCrossValidator):
    """
    Кросс-валидация по событиям (batch), хронологическая, с эмбарго.
    groups: массив той же длины, что и df, со значениями batch
    times:  массив pd.Timestamp (или sortable), та же длина, что и df
    """

    def __init__(self, n_splits: int = 5, embargo_events: int = 1, min_train_events: int = 5):
        self.n_splits = n_splits
        self.embargo_events = embargo_events
        self.min_train_events = min_train_events

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

    def split(self, X, y=None, groups=None, times: Optional[pd.Series] = None) -> Iterator[
        Tuple[np.ndarray, np.ndarray]]:
        if groups is None or times is None:
            raise ValueError("Pass groups=batch and times=time columns")

        groups = np.asarray(groups)
        times = pd.to_datetime(times)

        # порядок событий по старт-времени
        df_tmp = pd.DataFrame({"group": groups, "time": times}).reset_index(names="row_idx")
        first_time = df_tmp.groupby("group")["time"].min().sort_values()
        uniq_groups = first_time.index.to_numpy()

        n_events = len(uniq_groups)
        if n_events < (self.n_splits + self.min_train_events):
            # уменьшаем число сплитов, если событий мало
            eff_splits = max(1, n_events - self.min_train_events)
        else:
            eff_splits = self.n_splits

        # на каждой итерации расширяем train вправо
        for split_idx in range(1, eff_splits + 1):
            # доля событий для валидации
            val_events = max(1, n_events // (eff_splits + 1))
            train_end = n_events - (eff_splits - split_idx + 1) * val_events

            if train_end < self.min_train_events:
                continue

            # эмбарго
            embargoed_end = max(0, train_end - self.embargo_events)

            train_groups = uniq_groups[:embargoed_end]
            val_groups = uniq_groups[train_end: train_end + val_events]

            train_idx = df_tmp.index[df_tmp["group"].isin(train_groups)].to_numpy()
            val_idx = df_tmp.index[df_tmp["group"].isin(val_groups)].to_numpy()

            # индексы исходной X (если это DataFrame — у вас совпадают позиции с row_idx)
            yield (train_idx, val_idx)


class MinimalRichProgressBar(RichProgressBar):
    def on_validation_start(self, trainer, pl_module):
        pass

    def on_validation_batch_start(self, trainer, pl_module, batch, batch_idx):
        pass

    def on_validation_end(self, trainer, pl_module):
        pass


class NoValidationBar(TQDMProgressBar):
    def init_validation_tqdm(self):
        # возвращаем полностью отключённый tqdm для валидации
        return tqdm_class(disable=True)

class CustomTFT(TemporalFusionTransformer):
    def __init__(self, *args, **kwargs):
        self.mask_prob = kwargs.pop("mask_prob", 0.05)
        super().__init__(*args, **kwargs)
        self._val_preds = []
        self._val_trues = []
        self._val_gids = []
        self.scheduled_prob = 0.0

        safe_val = torch.tensor(
            torch.finfo(torch.float16).min,
            dtype=torch.float32
        )

        m = self.multihead_attn.attention
        if hasattr(m, "mask_bias") and not isinstance(m.mask_bias, torch.Tensor):
            delattr(m, "mask_bias")
        m.register_buffer("mask_bias", safe_val)

    def on_epoch_start(self, trainer, pl_module):
        if trainer.max_epochs > 0:
            self.scheduled_prob = min(1.0, trainer.current_epoch / (trainer.max_epochs * 0.8))

    def on_train_epoch_end(self):
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    def training_step(self, batch, batch_idx):
        x, y, weight = _unpack_pf_batch(batch)

        if torch.rand(1).item() < self.mask_prob and "encoder_target" in x:
            enc = x["encoder_target"]
            noise = torch.normal(0, 0.15, size=enc.shape, device=enc.device)
            x = {**x, "encoder_target": noise}
            del enc, noise

        if torch.rand(1).item() < self.scheduled_prob and y is not None:
            with torch.no_grad():
                out = self(x)
                y_pred = self.loss.to_prediction(out)
                y_bt = _collapse_pred_to_bt(y_pred)
                dec_tgt = y_bt if y_bt.dim() == 2 else y_bt.unsqueeze(-1)
                x['decoder_target'] = dec_tgt.detach()
                del out, y_pred, y_bt, dec_tgt

        batch = (x, y, weight) if weight is not None else (x, y)
        result = super().training_step(batch, batch_idx)

        if batch_idx % 50 == 0:  # Rare for speed
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        return result

    def validation_step(self, batch, batch_idx):
        x, y, weight = _unpack_pf_batch(batch)

        with torch.no_grad():
            out_temp = self(x)
            y_pred_temp = self.loss.to_prediction(out_temp)
            y_bt_temp = _collapse_pred_to_bt(y_pred_temp)
            del out_temp, y_pred_temp

        if "decoder_target" in x:
            enc_tgt = x.get("encoder_target")
            if enc_tgt is not None:
                batch_mean = enc_tgt.mean(dim=-1, keepdim=True)
                batch_std = enc_tgt.std(dim=-1, keepdim=True) + 1e-8
                mean_tensor = batch_mean.expand_as(x["decoder_target"])
                std_tensor = (0.1 * batch_std).expand_as(x["decoder_target"])
                noise_fill = torch.normal(mean_tensor, std_tensor)
                del batch_mean, std_tensor, mean_tensor
            else:
                dec_tgt = x["decoder_target"]
                device = dec_tgt.device
                noise_fill = torch.full_like(dec_tgt, self.global_target_mean)
                batch_size = dec_tgt.size(0)
                batch_std = torch.full((batch_size,), self.global_target_std, device=device).unsqueeze(-1)

            if torch.rand(1).item() < self.scheduled_prob:
                decoder_fill = y_bt_temp
            else:
                jitter_size = noise_fill.shape
                additional_jitter = torch.randn(jitter_size, device=noise_fill.device) * (0.05 * batch_std.expand_as(noise_fill))
                decoder_fill = noise_fill + additional_jitter
                del additional_jitter, noise_fill

            decoder_fill = torch.clamp(decoder_fill, -1.0, 1.0)
            x["decoder_target"] = decoder_fill
            del y_bt_temp, batch_std

        out = self(x)
        y_pred_raw = self.loss.to_prediction(out)
        del out

        y_pred_metrics = torch.clamp(y_pred_raw, -1.0, 1.0)
        del y_pred_raw

        y_actual = y if y is not None else x.get("decoder_target")
        if y_actual is None:
            return

        try:
            y_pred_aligned, y_actual_aligned = _align_pred_target(y_pred_metrics, y_actual)
            del y_pred_metrics, y_actual
        except Exception:
            return

        self._val_preds.append(y_pred_aligned.detach())
        self._val_trues.append(y_actual_aligned.detach())
        del y_pred_aligned, y_actual_aligned

        gid = x.get("group_ids")
        if gid is not None and isinstance(gid, torch.Tensor):
            self._val_gids.append(gid.detach())
        else:
            self._val_gids.append(None)

        if batch_idx % 50 == 0:  # Rare for speed
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

    def on_validation_epoch_end(self):
        if len(self._val_preds) == 0:
            return

        yp = torch.cat(self._val_preds, dim=0)
        yt = torch.cat(self._val_trues, dim=0)

        self._val_preds.clear()
        self._val_trues.clear()

        se = (yp - yt) ** 2
        val_mse = float(se.mean().item())
        val_mse_std = float(se.std(unbiased=False).item())
        del yp, yt

        self.log("val_mse", val_mse, prog_bar=True, on_step=False, on_epoch=True)
        self.log("val_mse_std", val_mse_std, prog_bar=False, on_step=False, on_epoch=True)

        has_any_gid = any(g is not None for g in self._val_gids)
        if has_any_gid:
            gid_list = []
            valid = True
            for g in self._val_gids:
                if g is None:
                    valid = False
                    break
                gid_list.append(g)

            if valid and len(gid_list) > 0:
                gid_all = torch.cat(gid_list, dim=0).numpy().ravel()
                se_np = se.numpy().ravel()
                del se

                if gid_all.shape[0] == se_np.shape[0]:
                    uniq = np.unique(gid_all)
                    g_mse = [se_np[gid_all == u].mean() for u in uniq if (gid_all == u).any()]
                    if len(g_mse) > 0:
                        g_mse = np.asarray(g_mse, dtype=float)
                        val_mse_group_mean = float(g_mse.mean())
                        val_mse_group_std = float(g_mse.std(ddof=0))
                        self.log("val_mse_group_mean", val_mse_group_mean, prog_bar=False, on_step=False, on_epoch=True)
                        self.log("val_mse_group_std", val_mse_group_std, prog_bar=True, on_step=False, on_epoch=True)
                    del g_mse, uniq

                del gid_all, se_np

        self._val_gids.clear()
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    def log(self, name, value, *args, **kwargs):
        if value is None:
            return
        super().log(name, value, *args, **kwargs)


def _worker_init_fn(worker_id: int, seed: int):
    """
    Глобальная функция для инициализации worker-а DataLoader-а.
    pickle её «видит» и может передать в подпроцессы.
    """
    set_seeds(seed + worker_id)


def _extract_tensor(x, role="pred"):
    """
    Извлекает torch.Tensor из различных контейнеров/структур.
    - dict: сперва пробуем ключи, характерные для предсказаний/таргета
    - tuple/list: берём первый тензор или первый элемент, приводимый к тензору
    - tensor: возвращаем как есть
    """
    if isinstance(x, torch.Tensor):
        return x

    if isinstance(x, dict):
        # Наиболее типичные ключи в pytorch-forecasting / lightning шагах
        preferred_keys = [
            "prediction", "pred", "output", "y_pred", "yhat", "y", "target"
        ]
        for k in preferred_keys:
            if k in x and isinstance(x[k], torch.Tensor):
                return x[k]
        # Если значения-словари/кортежи — попробуем рекурсивно
        for v in x.values():
            if isinstance(v, torch.Tensor):
                return v
            if isinstance(v, (list, tuple, dict)):
                try:
                    t = _extract_tensor(v, role=role)
                    if isinstance(t, torch.Tensor):
                        return t
                except Exception:
                    pass
        raise TypeError(f"Cannot extract tensor from dict for role={role}. Keys={list(x.keys())}")

    if isinstance(x, (list, tuple)):
        for item in x:
            if isinstance(item, torch.Tensor):
                return item
        # если нет прямого тензора — попробуем рекурсивно
        for item in x:
            if isinstance(item, (list, tuple, dict)):
                try:
                    t = _extract_tensor(item, role=role)
                    if isinstance(t, torch.Tensor):
                        return t
                except Exception:
                    pass
        raise TypeError(f"Cannot extract tensor from {type(x)} for role={role}")

    # Последняя попытка — у объектов некоторых библиотек есть .values или .tensor
    for attr in ("values", "tensor", "data"):
        if hasattr(x, attr):
            v = getattr(x, attr)
            if isinstance(v, torch.Tensor):
                return v

    raise TypeError(f"Unsupported type for tensor extraction (role={role}): {type(x)}")


def _maybe_squeeze_last(x):
    """
    Безопасно убираем последнюю размерность, если она равна 1.
    Если x не тензор — возвращаем как есть.
    """
    if not isinstance(x, torch.Tensor):
        return x
    if x.dim() > 0 and x.size(-1) == 1:
        return x.squeeze(-1)
    return x


def _align_pred_target(y_pred, y_actual):
    """
    Приводим предсказания и таргет к совместимым формам для MSE:
    - Извлекаем тензоры из возможных контейнеров.
    - Сводим предсказание к (B, T_pred) с последней осью как временем.
    - Таргет сводим к (B,) или (B, T_act).
    - Если таргет (B,) — берём последний горизонт из предсказаний.
    - Если таргет (B, T_act) — подгоняем по времени (обрезаем/проверяем равенство).
    """
    # 1) Достаём тензоры
    y_pred = _extract_tensor(y_pred, role="pred")
    y_actual = _extract_tensor(y_actual, role="target")

    # 2) Сжимаем последнюю единичную ось
    y_pred = _maybe_squeeze_last(y_pred)
    y_actual = _maybe_squeeze_last(y_actual)

    # Быстрый путь: формы совпали
    if isinstance(y_pred, torch.Tensor) and isinstance(y_actual, torch.Tensor):
        if y_pred.shape == y_actual.shape:
            return y_pred, y_actual

    # 3) Приводим предсказание к (B, T_pred)
    y_pred_bt = _collapse_pred_to_bt(y_pred)  # (B, T_pred)

    # 4) Приведём таргет к (B,) или (B, T_act)
    if y_actual.dim() == 1:
        # (B,) — ожидаем 1 шаг на таргет → берём последний горизонт из предсказаний
        if y_pred_bt.dim() != 2 or y_pred_bt.size(0) != y_actual.size(0):
            raise ValueError(f"Batch mismatch: pred={tuple(y_pred_bt.shape)} vs target={tuple(y_actual.shape)}")
        y_pred_aligned = y_pred_bt[:, -1]  # последний шаг горизонта
        return y_pred_aligned, y_actual

    if y_actual.dim() == 2:
        # (B, T_act)
        if y_pred_bt.size(0) != y_actual.size(0):
            raise ValueError(f"Batch mismatch: pred={tuple(y_pred_bt.shape)} vs target={tuple(y_actual.shape)}")
        T_pred = y_pred_bt.size(1)
        T_act = y_actual.size(1)
        if T_pred == T_act:
            return y_pred_bt, y_actual
        if T_pred > T_act:
            # Обрежем последние T_act шагов, чтобы соответствовать таргету
            y_pred_bt = y_pred_bt[:, -T_act:]
            return y_pred_bt, y_actual
        # Если предсказаний по времени меньше, чем в таргете — это логическая ошибка настройки
        raise ValueError(
            f"Prediction horizon shorter than target: pred T={T_pred}, target T={T_act} "
            f"(pred shape={tuple(y_pred_bt.shape)}, target shape={tuple(y_actual.shape)})"
        )

    # Случай редкий: если таргет внезапно >2D — пробуем схлопнуть по всем, кроме батча
    if y_actual.dim() > 2:
        # Схлопнём таргет к (B, T_act) по последней оси
        reduce_dims = tuple(range(1, y_actual.dim() - 1))
        if len(reduce_dims) > 0:
            y_actual_bt = y_actual.mean(dim=reduce_dims)
        else:
            y_actual_bt = y_actual
        # Рекурсивно выровняем теперь как (B, ?)
        return _align_pred_target(y_pred_bt, y_actual_bt)

    # Если таргет скалярный (редко, но вдруг), расширим до (B,) повтором
    if y_actual.dim() == 0:
        y_actual = y_actual.expand(y_pred_bt.size(0))
        y_pred_aligned = y_pred_bt[:, -1]
        return y_pred_aligned, y_actual

    # Если сюда дошли — что-то совсем нетипичное
    raise ValueError(
        f"Shapes still mismatch after alignment: pred={tuple(y_pred.shape)} vs target={tuple(y_actual.shape)}"
    )


# Фиксируем seeds для воспроизводимости и стабильности
def set_seeds(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    pl.seed_everything(seed, verbose=False)


class PeakFriendlyHuber(Metric):

    def __init__(
        self,
        delta: float = 0.5,
        peak_thr: float = 0.85,
        peak_weight: float = 1.6,  # Увеличено до 1.6 для stronger поощрения пиков
        contrast_weight: float = 0.02,
        center_band: float = 0.3,
        clip_scale: float = 1.5,  # Новый: scale для soft-clip (tanh * scale, чтобы не сжимать середину сильно)
    ):
        super().__init__()
        self.delta = float(delta)
        self.peak_thr = float(peak_thr)
        self.peak_weight = float(peak_weight)
        self.contrast_weight = float(contrast_weight)
        self.center_band = float(center_band)
        self.clip_scale = float(clip_scale)  # Новый параметр
        self.mse = MeanSquaredError()

    @staticmethod
    def _smooth_l1(diff, delta):
        absd = diff.abs()
        return torch.where(absd < delta, 0.5 * (diff ** 2) / delta, absd - 0.5 * delta)

    def to_prediction(self, y_pred):
        y_pred = _extract_tensor(y_pred, role="pred")
        return super().to_prediction(y_pred)

    def to_quantiles(self, y_pred, quantiles=None, **kwargs):
        y_pred = _extract_tensor(y_pred, role="pred")
        return super().to_quantiles(y_pred, quantiles=quantiles, **kwargs)

    def loss(self, y_pred, y_actual, **kwargs):
        y_pred = _extract_tensor(y_pred, role="pred")
        y_actual = _extract_tensor(y_actual, role="target")
        y_pred, y_actual = _align_pred_target(y_pred, y_actual)

        # Soft-clip (без изменений)
        y_pred = torch.tanh(y_pred * self.clip_scale) / self.clip_scale

        diff = y_pred - y_actual
        base = self._smooth_l1(diff, self.delta)

        # Адаптивный peak_weight: средний по батчу, scale от доли пиков
        if self.peak_weight > 1.0:
            with torch.no_grad():
                peak_mag = torch.relu(y_actual.abs() - self.peak_thr)
                peak_frac = (peak_mag > 0).float().mean()  # Доля пиков в батче
                adaptive_weight = 1.0 + (self.peak_weight - 1.0) * peak_frac  # Больше веса, если много пиков
                w = adaptive_weight * torch.clamp(peak_mag / (1.0 - self.peak_thr + 1e-8), 0.0, 1.0) + 1.0
            huber_term = (base * w).mean()
        else:
            huber_term = base.mean()

        # Лёгкий «anti-flatness» у центра: штрафим чрезмерно малую амплитуду,
        # но только там, где таргет далеко от 0.
        if self.contrast_weight > 0.0:
            with torch.no_grad():
                far_mask = (y_actual.abs() >= self.center_band).float()
                near_mask = (y_actual.abs() < self.center_band).float()

            # Прямая амплитуда предсказания
            far_amp = (y_pred.abs() * far_mask).sum() / (far_mask.sum() + 1e-8)
            near_amp = (y_pred.abs() * near_mask).sum() / (near_mask.sum() + 1e-8)

            # Хотим far_amp >= near_amp + margin; введём небольшой margin
            margin = 0.05
            contrast = torch.relu((near_amp + margin) - far_amp)
            loss = huber_term + self.contrast_weight * contrast
        else:
            loss = huber_term

        return loss

    def __call__(self, y_pred, y_actual, **kwargs):
        return self.loss(y_pred, y_actual, **kwargs)

    def update(self, y_pred, y_actual, **kwargs):
        y_pred = _extract_tensor(y_pred, role="pred")
        y_actual = _extract_tensor(y_actual, role="target")
        y_pred, y_actual = _align_pred_target(y_pred, y_actual)
        self.mse.update(y_pred, y_actual)

    def compute(self):
        return self.mse.compute()

    def reset(self):
        self.mse.reset()

    def name(self):
        return "PeakFriendlyHuber"

class TFTAdapter(pl.LightningModule):
    """
    Обёртка над TemporalFusionTransformer,
    чтобы Trainer воспринимал модель нужного типа
    и наш tft.training_step видел непустой self.trainer.
    """

    def __init__(self, tft: TemporalFusionTransformer):
        super().__init__()
        self.tft = tft

    def on_fit_start(self) -> None:
        # вызовется перед стартом Trainer.fit
        # прикрепляем Trainer к внутреннему tft
        self.tft.trainer = self.trainer
        # и логгеры
        self.tft.log = self.log
        self.tft.log_dict = self.log_dict

    def forward(self, *args, **kwargs):
        return self.tft(*args, **kwargs)

    def training_step(self, batch, batch_idx):
        self.tft.trainer = self.trainer
        result = self.tft.training_step(batch, batch_idx)
        if isinstance(result, dict):
            return result.get("loss")
        elif isinstance(result, tuple) and len(result) >= 2:
            log_dict, out = result[:2]  # Берем первые два элемента
            if isinstance(log_dict, dict):
                return log_dict.get("loss")
        # Если ни один вариант не подошел
        raise ValueError(f"Unexpected return type from tft.training_step: {type(result)}")

    def validation_step(self, batch, batch_idx):
        # обеспечим корректную ссылку на тренер внутри tft (если нужно)
        self.tft.trainer = self.trainer
        self.tft.validation_step(batch, batch_idx)
        return

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        # Просто передаём вызов внутреннему TFT, без dataloader_idx (не нужен для TFT)
        return self.tft.predict_step(batch, batch_idx)

    def predict(self, *args, **kwargs):
        return self.tft.predict(*args, **kwargs)

    def configure_optimizers(self):
        return self.tft.configure_optimizers()


def to_dense(X):
    """Преобразование sparse matrix в dense numpy array"""
    if hasattr(X, 'toarray'):
        return X.toarray()
    return np.asarray(X)

def prepare_data_transformer(df, target_col):
    df = df.dropna(subset=[target_col])
    if 'time' in df.columns and 'time_idx' not in df.columns:
        df = df.rename(columns={'time': 'time_idx'})
    X = df.drop(columns=target_col)
    y = df[target_col].copy()

    exclude = ['time_idx', 'batch']
    numeric_features = [
        c for c in X.select_dtypes(include=['int64', 'float64', 'float32', 'int32']).columns
        if c not in exclude
    ]
    categorical_features = [
        c for c in X.select_dtypes(include=['object', 'category']).columns
        if c not in exclude
    ]

    preprocessing = ColumnTransformer(
        transformers=[
            ('num', Pipeline([
                ('scaler', RobustScaler()),
                ('yeo', PowerTransformer(method='yeo-johnson'))
            ]), numeric_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ],
        remainder='passthrough'  # passthrough → тут окажутся сначала все num→cat, а потом time_idx и batch
    )
    # Возвращаем дополнительные списки для передачи в модель
    return X, y, preprocessing, numeric_features, categorical_features

def split_features_batch_time(X_array, n_transformed):
    """
    X_array: np.ndarray после преобразований shape=(N, n_transformed + 2)
    n_transformed: сколько колонок ушло на num+cat
    возвращает (features, time_raw, batch_raw)
    """
    features = X_array[:, :n_transformed]
    batch_raw = X_array[:, n_transformed].ravel()
    time_raw = X_array[:, n_transformed + 1].ravel()
    return features, batch_raw, time_raw


def tft_output_transformer(x):
    # Больше НЕ клипуем внутри графа. Пусть модель учится выходить за [-1,1],
    # а мы ограничим при расчёте метрик и при возврате пользователю.
    return x


def get_early_stopping_callback(patience=10, min_delta=0.001):
    return EarlyStopping(
        monitor="train_loss",
        patience=patience,
        min_delta=min_delta,
        mode="min",
        verbose=True
    )


class SequenceTransformerRegressor(BaseEstimator, RegressorMixin):
    """
    Temporal Fusion Transformer для последовательностей.
    Интегрируется в Pipeline аналогично LSTM.
    Адаптировано для стабильного обучения и алготрейдинга (реального времени).
    """

    def __init__(self,
                 seq_len: int = 10,
                 pred_len: int = 1,
                 hidden_size: int = 64,  # Увеличено для лучшей емкости
                 hidden_continuous_size: int = 24,
                 epochs: int = 100,  # Увеличено для более долгого обучения
                 batch_size: int = 128,  # Увеличено для стабильности
                 learning_rate: float = 1e-3,  # Увеличено для более быстрого старта
                 patience: int = 15,  # Увеличено для терпимости
                 seed: int = 42,
                 dropout: float = 0.25,  # Уменьшено для меньшей регуляризации
                 weight_decay: float = 1e-4,  # Новый: для регуляризации
                 verbose: int = 2,
                 mask_prob: float = 0.1,  # Уменьшено, чтобы меньше шумить
                 infer_stride: int = 2,
                 ckpt_path=None,
                 preprocessing=None,
                 numeric_features=None,
                 categorical_features=None,
                 remainder_columns=None,
                 min_encoder_length: int = 1):  # Больше логов
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.hidden_size = hidden_size
        self.hidden_continuous_size = hidden_continuous_size
        self.epochs = epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.patience = patience
        self.seed = seed
        self.verbose = verbose
        self._model = None
        self._trainer = None
        self._n_transformed = None
        self._n_feat = None
        self._train_dataset = None
        self._feature_columns = None
        self._dropout = dropout
        self.norm_eps = 1e-6
        self.norm_window = max(10, self.seq_len // 2)
        self.ckpt_path = ckpt_path
        self.weight_decay = weight_decay  # Новый
        self.mask_prob = mask_prob
        self.infer_stride = infer_stride
        self.global_target_mean = 0.0  # Будем вычислять в fit
        self.global_target_std = 1.0  # Fallback global scale
        self.global_target_min = None
        self.global_target_max = None
        self.global_range = None
        self.soft_clip_scale =  None
        self.preprocessing = preprocessing
        self.numeric_features = numeric_features or []
        self.categorical_features = categorical_features or []
        self.remainder_columns = remainder_columns or ['batch', 'time']
        self.use_tanh_post = False
        self.train_q_lo = None
        self.train_q_hi = None
        self.clip_scale = 1.5
        self.future_fill_mode = "repeat"
        self.smooth_window = max(5, self.seq_len // 5)  # For savgol
        self.smooth_poly = 2
        self.ema_alpha = 0.1
        self.infer_batch_size = 512  # Новый: большой батч для inference
        self.infer_stride = infer_stride if infer_stride is not None else 4
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'# Добавьте это, если нужно tanh в предикте
        self.min_encoder_length = max(1, min_encoder_length)  # Не меньше 1

    def fit(self, X, y):
        set_seeds(self.seed)

        X = np.asarray(X)
        if self._n_transformed is None:
            self._n_transformed = X.shape[1] - 2
            self._n_feat = X.shape[1]

        feat, batch_raw, time_raw = split_features_batch_time(X, self._n_transformed)

        df = pd.DataFrame(feat, columns=[f"f{i}" for i in range(feat.shape[1])])
        self._feature_columns = df.columns.tolist()

        df["batch"] = pd.Series(batch_raw).astype("int64")
        df["time_raw"] = pd.to_datetime(time_raw, utc=True, errors="coerce")

        if isinstance(y, (pd.Series, pd.DataFrame)):
            y = y.reset_index(drop=True)
        df["target"] = pd.Series(y, index=df.index).astype(float)

        before = len(df)
        df = df.dropna(subset=["time_raw", "target"]).reset_index(drop=True)
        if self.verbose and len(df) < before:
            print(f"Dropped {before - len(df)} rows with invalid time/target")

        df = df.sort_values(["batch", "time_raw"]).reset_index(drop=True)
        df["time_idx"] = df.groupby("batch").cumcount()

        min_len = max(1, int(self.pred_len))  # Изменено: позволяем короткие батчи (encoder может быть < seq_len)
        gsize = df.groupby("batch").size()
        valid_batches = gsize[gsize >= min_len].index
        if len(valid_batches) == 0:
            raise ValueError(f"No batches with length >= {min_len}. Reduce pred_len.")
        if self.verbose and len(valid_batches) < gsize.index.nunique():
            dropped = sorted(list(set(gsize.index) - set(valid_batches)))
            print(f"Warning: dropped {len(dropped)} short batches: {dropped[:8]}{' ...' if len(dropped) > 8 else ''}")
        df = df[df["batch"].isin(valid_batches)].reset_index(drop=True)

        batch_starts = df.groupby("batch")["time_raw"].min().sort_values()
        uniq_batches = batch_starts.index.to_numpy()
        n_total = len(uniq_batches)
        val_frac = 0.3 if n_total >= 10 else 0.1  # Увеличено для лучшего обобщения
        n_val = max(1, int(round(n_total * val_frac)))

        embargo = 1 if n_total >= 8 else 0

        train_end = max(0, n_total - n_val - embargo)
        train_batches = uniq_batches[:train_end]
        val_batches = uniq_batches[-n_val:]

        if len(train_batches) == 0 and n_total > 1:
            train_batches = uniq_batches[:-1]
            val_batches = uniq_batches[-1:]

        train_df = df[df["batch"].isin(train_batches)].copy()
        val_df = df[df["batch"].isin(val_batches)].copy()

        self.global_target_mean = train_df["target"].mean()
        self.global_target_std = train_df["target"].std() + 1e-8
        self.global_target_min = train_df["target"].min()
        self.global_target_max = train_df["target"].max()
        self.global_range = self.global_target_max - self.global_target_min + 1e-8
        self.soft_clip_scale = max(1.0, self.global_target_std * 1.5)  # Adaptive soft clip for stable [-1,1] without compression

        #self.clip_scale = max(1.0, self.global_target_std * 1.2)   # Adaptive to train variance

        train_targets = train_df["target"].values
        #if len(train_targets) > 0:
        #   self.train_q_lo, self.train_q_hi = np.quantile(train_targets, [0.01, 0.99])

        if self.verbose:
            print(f"Train batches: {len(np.unique(train_batches))}")
            print(f"Val batches: {len(np.unique(val_batches))}")
            print(f"Train rows: {len(train_df)}, Val rows: {len(val_df)}")

        full_dataset = TimeSeriesDataSet(
            df,
            time_idx="time_idx",
            target="target",
            group_ids=["batch"],
            max_encoder_length=int(self.seq_len),
            min_encoder_length=0,#int(self.min_encoder_length),  # Новый: позволяем короткие encoder
            max_prediction_length=int(self.pred_len),
            time_varying_unknown_reals=self._feature_columns,
            target_normalizer=None,
            allow_missing_timesteps=True,
            add_relative_time_idx=True,
            add_target_scales=False,
            add_encoder_length=True,
            min_prediction_length=1,
        )

        train_dataset = TimeSeriesDataSet.from_dataset(
            full_dataset, train_df, predict=False, stop_randomization=True
        )
        val_dataset = TimeSeriesDataSet.from_dataset(
            full_dataset, val_df, predict=False, stop_randomization=True
        ) if len(val_df) > 0 else None

        train_dl = train_dataset.to_dataloader(
            train=True,
            batch_size=int(self.batch_size),
            shuffle=True,
            num_workers=0,
            worker_init_fn=None,
            drop_last=False,
            persistent_workers=False,
        )
        val_dl = None
        if val_dataset is not None and len(val_dataset) > 0:
            val_dl = val_dataset.to_dataloader(
                train=False,
                batch_size=int(self.batch_size),
                shuffle=False,
                num_workers=0,
                worker_init_fn=None,
                drop_last=False,
                persistent_workers=False,
            )

        tft = CustomTFT.from_dataset(
            train_dataset,
            hidden_size=int(self.hidden_size),
            output_size=1,
            loss=PeakFriendlyHuber(
                delta=0.5,
                peak_thr=0.85,  # можно затем подвинуть 0.8..0.9
                peak_weight=1.3,  # аккуратно: 1.3..1.6
                contrast_weight=0.03,  # очень маленькая добавка
                center_band=0.3,  # что считать «центром»
                clip_scale=1.5
            ),
            optimizer="adam",
            learning_rate=float(self.learning_rate),  # оставьте тот, на котором MSE был лучше (у вас 1e-4 давал ~0.186)
            lstm_layers=3,
            hidden_continuous_size=self.hidden_continuous_size,
            attention_head_size=4,
            dropout=float(self._dropout),
            reduce_on_plateau_patience=5,
            reduce_on_plateau_min_lr=1e-6,
            weight_decay=float(self.weight_decay),
            mask_prob=float(self.mask_prob),
            output_transformer=tft_output_transformer,
        )

        class GCCallback(pl.Callback):
            def __init__(self):
                super().__init__()
                self.last_mem = psutil.Process().memory_info().rss / 1e6

            def _check_gc(self):
                current_mem = psutil.Process().memory_info().rss / 1e6
                if current_mem - self.last_mem > 50:
                    gc.collect()
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                self.last_mem = current_mem

            def on_train_epoch_end(self, trainer, pl_module):
                self._check_gc()

            def on_validation_epoch_end(self, trainer, pl_module):
                self._check_gc()

            def on_train_batch_end(self, trainer, pl_module, outputs, batch, batch_idx):
                if batch_idx % 10 == 0:
                    self._check_gc()

            def on_validation_batch_end(self, trainer, pl_module, outputs, batch, batch_idx):
                if batch_idx % 10 == 0:
                    self._check_gc()

        callbacks = []
        if val_dl is not None:
            callbacks.append(
                get_early_stopping_callback(patience=self.patience, min_delta=1e-3))  # Новый: больше patience
            checkpoint_callback = ModelCheckpoint(monitor="train_loss", mode="min", save_top_k=1, verbose=True)
            callbacks.append(checkpoint_callback)
        else:
            checkpoint_callback = None

        logger = TensorBoardLogger(save_dir="lightning_logs/", name="my_model") if self.verbose > 0 else False
        if self.verbose > 0:
            callbacks.append(LearningRateMonitor(logging_interval="step"))
            callbacks.append(NoValidationBar(refresh_rate=20))
        callbacks.append(GCCallback())

        self._model = TFTAdapter(tft)
        self._trainer = pl.Trainer(
            max_epochs=int(self.epochs),
            enable_checkpointing=(checkpoint_callback is not None),
            callbacks=callbacks,
            logger=logger,
            enable_model_summary=True,
            gradient_clip_val=1.0,
            gradient_clip_algorithm="norm",
            deterministic=True,
            benchmark=False,
            accelerator="gpu" if torch.cuda.is_available() else "cpu",
            precision=32,
            limit_val_batches=1.0 if val_dl is not None else 0.0,
            enable_progress_bar=self.verbose > 0,
            log_every_n_steps=50,
            num_sanity_val_steps=0,
        )

        self._trainer.fit(
            self._model,
            train_dataloaders=train_dl,
            val_dataloaders=val_dl if val_dl is not None else None,
            ckpt_path=self.ckpt_path if self.ckpt_path and self.epochs > 0 else None,
        )

        if checkpoint_callback is not None and checkpoint_callback.best_model_path:
            best_path = checkpoint_callback.best_model_path
            if self.verbose:
                print(f"Loaded best model from {best_path} with val_loss={checkpoint_callback.best_model_score}")
            self._model = TFTAdapter.load_from_checkpoint(best_path, tft=tft)

        self._train_dataset = full_dataset

        gc.collect()

        #return self
        try:
            y_train = train_df["target"].to_numpy(dtype=float)
            # robust percentiles — перестрахуемся от выносов: 1% и 99%
            self._cal_p_low = float(np.nanpercentile(y_train, 1))
            self._cal_p_high = float(np.nanpercentile(y_train, 99))
            # амплитуды «типичных пиков»
            top_mask = y_train >= self._cal_p_high
            bot_mask = y_train <= self._cal_p_low
            self._cal_mean_top = float(np.nanmean(y_train[top_mask])) if np.any(top_mask) else float(self.global_target_max)
            self._cal_mean_bot = float(np.nanmean(y_train[bot_mask])) if np.any(bot_mask) else float(self.global_target_min)
            # защита от вырождения
            if not np.isfinite(self._cal_mean_top): self._cal_mean_top = float(self.global_target_max)
            if not np.isfinite(self._cal_mean_bot): self._cal_mean_bot = float(self.global_target_min)
        except Exception:
            # безопасные фолбэки
            self._cal_p_low, self._cal_p_high = self.global_target_min, self.global_target_max
            self._cal_mean_top, self._cal_mean_bot = self.global_target_max, self.global_target_min
        # ----------------------------------------------
        return self

    def predict(self, X):
        if self._train_dataset is None or self._model is None:
            raise RuntimeError("Model is not fitted yet")

        # Full suppress context (no logs/output, including PL/Torch/PF/Seed/GPU/TPU/HPU messages)
        @contextlib.contextmanager
        def suppress_all():
            with open(os.devnull, "w") as devnull, contextlib.redirect_stdout(devnull), contextlib.redirect_stderr(devnull):
                # Подавление всех возможных логгеров
                root_logger = logging.getLogger()
                old_level = root_logger.level
                root_logger.setLevel(logging.CRITICAL + 1)  # Выше CRITICAL, чтобы ничего не логировалось

                # Специфические логгеры (расширенный список)
                for logger_name in [
                    "pytorch_lightning", "lightning.pytorch", "lightning",
                    "torch", "pytorch_forecasting", "optuna",
                    "sklearn", "joblib", "numpy", "pandas",
                    "scipy", "matplotlib", "seaborn", "plotly",
                    "shap", "statsmodels", "torchmetrics",
                    "lightning.pytorch.utilities.migration.utils",  # Для Attribute 'loss' warnings
                    "lightning.pytorch.utilities.migration",
                    "lightning.pytorch.utilities",
                    "lightning.pytorch"
                ]:
                    logging.getLogger(logger_name).setLevel(logging.CRITICAL + 1)

                # Подавление всех предупреждений (расширенный список)
                warnings.filterwarnings("ignore")
                warnings.filterwarnings("ignore", category=DeprecationWarning)
                warnings.filterwarnings("ignore", category=UserWarning)
                warnings.filterwarnings("ignore", category=RuntimeWarning)
                warnings.filterwarnings("ignore", category=FutureWarning)
                warnings.filterwarnings("ignore", message=".*Attribute 'loss' is an instance of nn.Module.*")
                warnings.filterwarnings("ignore", message=".*Attribute 'logging_metrics' is an instance of nn.Module.*")
                warnings.filterwarnings("ignore", message=".*GPU available.*")
                warnings.filterwarnings("ignore", message=".*TPU available.*")
                warnings.filterwarnings("ignore", message=".*HPU available.*")
                warnings.filterwarnings("ignore", message=".*This Pipeline instance is not fitted yet.*")
                warnings.filterwarnings("ignore", message=".*Using an existing study with name.*")
                warnings.filterwarnings("ignore", message=".*A value is trying to be set on a copy of a slice.*")
                warnings.filterwarnings("ignore", message=".*The behavior of DataFrame concatenation.*")
                warnings.filterwarnings("ignore", message=".*torch.utils.checkpoint: the use_reentrant parameter.*")

                # Окружение (расширенное)
                os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
                os.environ['LITMODELS_DISABLE_TIP'] = '1'
                os.environ['HYDRA_FULL_ERROR'] = '1'
                os.environ['TQDM_DISABLE'] = '0'  # Не подавлять tqdm (если вызван снаружи)
                os.environ['CUDA_LAUNCH_BLOCKING'] = '1'  # Для синхронизации CUDA, но без логов
                os.environ['TORCH_USE_DETERMINISTIC_ALGORITHMS'] = '1'  # Без логов

                try:
                    yield
                finally:
                    root_logger.setLevel(old_level)
                    warnings.resetwarnings()

        with suppress_all():
            set_seeds(self.seed)
            pl.seed_everything(self.seed, verbose=False, workers=True)

            if self.preprocessing is not None and not isinstance(X, np.ndarray):
                X = self.preprocessing.transform(X)

            X = np.asarray(X)
            N = X.shape[0]
            if N < self.min_encoder_length:
                return [float(np.nan)] * N

            feat, _, time_raw = split_features_batch_time(X, self._n_transformed)
            df = pd.DataFrame(feat, columns=self._feature_columns)
            df["__row_id"] = np.arange(N)
            df["time_raw"] = pd.to_datetime(time_raw, utc=True, errors="coerce")
            df["batch"] = np.int64(0)
            df["target"] = self.global_target_mean

            df = df.dropna(subset=["time_raw"]).reset_index(drop=True)
            df_sorted = df.sort_values("time_raw").reset_index(drop=True)
            df_sorted["time_idx"] = np.arange(len(df_sorted), dtype=np.int64)

            eff_N = len(df_sorted)
            if eff_N < self.min_encoder_length:
                out = np.full(N, np.nan, dtype=float)
                return out.tolist()

            step_ns = int(60 * 1e9)
            if eff_N >= 2:
                diffs = df_sorted["time_raw"].view("int64").astype("int64").to_numpy()
                diffs = np.diff(diffs)
                step_ns = int(np.nan_to_num(np.median(diffs), nan=step_ns))
                if step_ns <= 0:
                    step_ns = int(60 * 1e9)

            out_raw = np.full(eff_N, np.nan, dtype=float)

            tft = self._model.tft
            orig_log = tft.log
            tft.log = lambda *args, **kwargs: None

            try:
                eff_encoder_len = min(self.seq_len, eff_N)
                M = max(0, eff_N - eff_encoder_len + 1)
                stride = self.pred_len  # Keep original stride to preserve logic
                window_starts = np.arange(0, M, stride)
                K = len(window_starts)

                if K == 0:
                    K = 1
                    window_starts = np.array([0])
                    eff_encoder_len = eff_N

                num_feat_cols = len(self._feature_columns)

                # Vectorized construction of all_feats
                enc_indices = window_starts[:, np.newaxis] + np.arange(eff_encoder_len)
                all_enc_feats = feat[enc_indices]  # (K, eff_encoder_len, num_feat_cols)
                last_enc_feats = all_enc_feats[:, -1, :]
                all_fut_feats = np.repeat(last_enc_feats[:, np.newaxis, :], self.pred_len, axis=1)  # (K, pred_len, num_feat_cols)
                all_feats = np.concatenate([all_enc_feats, all_fut_feats], axis=1).reshape(-1, num_feat_cols)

                # Vectorized all_time_idx
                orig_time_idx = df_sorted["time_idx"].values
                all_enc_time_idx = orig_time_idx[enc_indices]  # (K, eff_encoder_len)
                last_time_idx = all_enc_time_idx[:, -1]
                fut_offsets = np.arange(1, self.pred_len + 1)
                all_fut_time_idx = last_time_idx[:, np.newaxis] + fut_offsets  # (K, pred_len)
                all_time_idx = np.concatenate([all_enc_time_idx, all_fut_time_idx], axis=1).reshape(-1)

                # Vectorized all_batch
                batch_per_window = np.arange(K)[:, np.newaxis]
                all_enc_batch = np.repeat(batch_per_window, eff_encoder_len, axis=1)  # (K, eff_encoder_len)
                all_fut_batch = np.repeat(batch_per_window, self.pred_len, axis=1)  # (K, pred_len)
                all_batch = np.concatenate([all_enc_batch, all_fut_batch], axis=1).reshape(-1)

                # all_target (constant)
                all_target = np.full(K * (eff_encoder_len + self.pred_len), self.global_target_mean, dtype=np.float32)

                # Vectorized all_time_raw using int64 ns
                orig_time_raw_int = df_sorted["time_raw"].view("int64").values
                all_enc_time_int = orig_time_raw_int[enc_indices]  # (K, eff_encoder_len)
                last_time_int = all_enc_time_int[:, -1]
                fut_offsets_ns = fut_offsets * step_ns
                all_fut_time_int = last_time_int[:, np.newaxis] + fut_offsets_ns  # (K, pred_len)
                all_time_int_flat = np.concatenate([all_enc_time_int.reshape(-1), all_fut_time_int.reshape(-1)])
                all_time_raw = pd.to_datetime(all_time_int_flat, unit='ns', utc=True).values  # object array of Timestamp

                pred_df = pd.DataFrame(all_feats, columns=self._feature_columns)
                pred_df["time_idx"] = all_time_idx
                pred_df["batch"] = all_batch
                pred_df["target"] = all_target
                pred_df["time_raw"] = all_time_raw

                pred_df["batch"] = pred_df["batch"].astype("int64")
                pred_df["time_idx"] = pred_df["time_idx"].astype("int64")
                pred_df["target"] = pred_df["target"].astype("float32")

                sliding_dataset = TimeSeriesDataSet.from_dataset(
                    self._train_dataset,
                    pred_df,
                    predict=True,
                    stop_randomization=True,
                    min_prediction_length=self.pred_len,
                    max_prediction_length=self.pred_len,
                )

                test_dl = sliding_dataset.to_dataloader(
                    train=False,
                    batch_size=K,  # Full batch for max speed (K small due to stride=pred_len)
                    num_workers=0,
                    persistent_workers=False,
                    pin_memory=False,
                )

                preds = tft.predict(
                    test_dl,
                    mode="prediction",
                    return_x=False,
                    trainer_kwargs={
                        "logger": False,
                        "enable_progress_bar": False,
                        "enable_model_summary": False,
                        "enable_checkpointing": False,
                        "accelerator": "gpu" if self.device == "cuda" else "cpu"
                    },
                )

                if isinstance(preds, torch.Tensor):
                    preds = preds.detach().cpu()
                    if preds.dim() == 3 and preds.size(-1) == 1:
                        preds = preds.squeeze(-1)
                    if preds.dim() == 2:
                        yhat_stride = preds[:, 0].numpy()  # First step per window
                    elif preds.dim() == 1:
                        yhat_stride = preds.numpy()
                    elif preds.dim() == 0:
                        yhat_stride = np.array([preds.item()])
                    else:
                        yhat_stride = np.full(K, self.global_target_mean)
                else:
                    yhat_stride = np.full(K, self.global_target_mean)

                # Fill out_raw at window ends
                for j, i in enumerate(window_starts):
                    out_raw[i + eff_encoder_len - 1] = yhat_stride[j]

                # Causal interpolation for missed points (linear from past, no future look)
                s = pd.Series(out_raw)
                s = s.interpolate(method='linear', limit_direction='forward')  # Only forward for causality
                s = s.ffill()  # Fill initial with first pred
                out_raw = s.values

                # Global map to [-1,1] (consistent scale, preserves relative peaks)
                out = (out_raw - self.global_target_min) / self.global_range * 2 - 1
                out = np.tanh(out * self.soft_clip_scale) / np.tanh(self.soft_clip_scale)
                out = np.clip(out, -1.0, 1.0)

                # Fill any remaining nans (unlikely) with mapped mean
                nan_mask = np.isnan(out)
                mapped_mean = (self.global_target_mean - self.global_target_min) / self.global_range * 2 - 1
                out[nan_mask] = mapped_mean

            finally:
                tft.log = orig_log
                gc.collect()
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()

            original_pos = df_sorted["__row_id"].to_numpy()
            out_final = np.full(N, np.nan, dtype=float)
            out_final[original_pos] = out

            return out_final.tolist()

    def get_params(self, deep=True):
        # Чтобы sklearn корректно передавал параметры в Pipeline
        return {
            "seq_len": self.seq_len,
            "pred_len": self.pred_len,
            "hidden_size": self.hidden_size,
            "epochs": self.epochs,
            "batch_size": self.batch_size,
            "learning_rate": self.learning_rate,
            "patience": self.patience,
            "seed": self.seed,
            "dropout": self._dropout,
            "weight_decay": self.weight_decay,
            "verbose": self.verbose,
            "ckpt_path": self.ckpt_path
        }

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

def _tensor_state_dict_to(dtype: torch.dtype, state: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
    out = {}
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            # важно: сохраняем в CPU и ровно в dtype (float32 для идентичности)
            out[k] = v.detach().to('cpu', dtype=dtype)
        else:
            out[k] = v
    return out


def _save_bytes_compressed_zip(file_path: str, bytes_map: Dict[str, bytes], compression=zipfile.ZIP_DEFLATED, compresslevel=9):
    with zipfile.ZipFile(file_path, mode='w', compression=compression, compresslevel=compresslevel) as zf:
        for name, b in bytes_map.items():
            zf.writestr(name, b)


def _load_bytes_from_zip(file_path: str) -> Dict[str, bytes]:
    out = {}
    with zipfile.ZipFile(file_path, mode='r') as zf:
        for name in zf.namelist():
            out[name] = zf.read(name)
    return out


def _safe_json_dump(obj: Dict[str, Any]) -> bytes:
    def to_builtin(x):
        import numpy as _np
        import pandas as _pd
        if isinstance(x, (_np.integer,)):
            return int(x)
        if isinstance(x, (_np.floating,)):
            return float(x)
        if isinstance(x, (_np.ndarray,)):
            return x.tolist()
        if isinstance(x, (_pd.Timestamp,)):
            return x.isoformat()
        return x

    def convert(v):
        if isinstance(v, dict):
            return {k: convert(val) for k, val in v.items()}
        if isinstance(v, (list, tuple)):
            return [convert(i) for i in v]
        return to_builtin(v)

    return json.dumps(convert(obj), ensure_ascii=False, separators=(",", ":")).encode("utf-8")


def _safe_json_load(b: bytes) -> Dict[str, Any]:
    return json.loads(b.decode("utf-8"))


def _extract_tsd_feature_lists(tds) -> Dict[str, Any]:
    def g(name, default=None):
        return getattr(tds, name, default)

    lists = {}
    for name in [
        "time_varying_known_reals",
        "time_varying_unknown_reals",
        "static_reals",
        "time_varying_known_categoricals",
        "time_varying_unknown_categoricals",
        "static_categoricals",
        "target_categoricals",
        "known_reals",
        "unknown_reals",
        "known_categoricals",
        "unknown_categoricals",
        "reals",
        "categoricals",
    ]:
        val = g(name, None)
        if val is not None:
            try:
                lists[name] = list(val)
            except Exception:
                pass
    return lists


def save_transformer(
    pipeline: Pipeline,
    path: str,
    *,
    # ЖЁСТКО: float32 по умолчанию для идентичности. Не выставляйте True, пока не сравните предикты.
    float16_weights: bool = False,
    preprocessing_filename: str = "preprocessing.joblib.lzma",
    model_zip_filename: str = "model_weights.zip",
    meta_json_filename: str = "meta.json"
):
    os.makedirs(path, exist_ok=True)

    if not isinstance(pipeline, Pipeline):
        raise TypeError("Expected sklearn Pipeline")

    steps_dict = dict(pipeline.named_steps)
    if "preprocessing" not in steps_dict or "model" not in steps_dict:
        raise ValueError("Pipeline must have 'preprocessing' and 'model' steps")

    preprocessing = steps_dict["preprocessing"]
    model: SequenceTransformerRegressor = steps_dict["model"]

    if model._model is None or model._train_dataset is None:
        raise RuntimeError("Model must be fitted before saving")

    # 1) preprocessing
    prep_path = os.path.join(path, preprocessing_filename)
    with lzma.open(prep_path, "wb", preset=9) as f:
        joblib.dump(preprocessing, f)

    # 2) параметры TDS
    tds = model._train_dataset
    def g(name, default=None):
        return getattr(tds, name, default)

    train_dataset_params = {
        "time_idx": g("time_idx", "time_idx"),
        "target": g("target", "target"),
        "group_ids": list(g("group_ids", ["batch"])),
        "max_encoder_length": int(g("max_encoder_length", model.seq_len)),
        "max_prediction_length": int(g("max_prediction_length", model.pred_len)),
        "target_normalizer": None,
        "allow_missing_timesteps": bool(g("allow_missing_timesteps", True)),
        "add_relative_time_idx": bool(g("add_relative_time_idx", True)),
        "add_target_scales": bool(g("add_target_scales", False)),
        "add_encoder_length": bool(g("add_encoder_length", True)),
        "min_prediction_length": int(g("min_prediction_length", 1)),
        "min_encoder_length": int(g("min_encoder_length", 0)),
    }
    feature_lists = _extract_tsd_feature_lists(tds)

    # 3) веса TFT — строго в float32 (если float16_weights=False)
    tft = model._model.tft
    state = tft.state_dict()
    dtype = torch.float16 if float16_weights else torch.float32
    state = _tensor_state_dict_to(dtype, state)
    buf = io.BytesIO()
    torch.save(state, buf, _use_new_zipfile_serialization=True)
    buf.seek(0)
    weights_zip_path = os.path.join(path, model_zip_filename)
    _save_bytes_compressed_zip(weights_zip_path, {"tft_state_dict.pt": buf.getvalue()})

    # 4) метаданные
    meta = {
        "class": "SequenceTransformerRegressor",
        "params": {
            "seq_len": model.seq_len,
            "pred_len": model.pred_len,
            "hidden_size": model.hidden_size,
            "hidden_continuous_size": model.hidden_continuous_size,
            "epochs": model.epochs,
            "batch_size": model.batch_size,
            "learning_rate": model.learning_rate,
            "patience": model.patience,
            "seed": model.seed,
            "dropout": model._dropout,
            "weight_decay": model.weight_decay,
            "verbose": model.verbose,
            "mask_prob": model.mask_prob,
            "infer_stride": model.infer_stride,
            "ckpt_path": None,
            "min_encoder_length": model.min_encoder_length,
            "lstm_layers": 3,
            "attention_head_size": 4,
            "output_size": 1,
        },
        "feature_columns": model._feature_columns,
        "n_transformed": model._n_transformed,
        "n_feat": model._n_feat,
        "global_stats": {
            "global_target_mean": model.global_target_mean,
            "global_target_std": model.global_target_std,
            "global_target_min": model.global_target_min,
            "global_target_max": model.global_target_max,
            "global_range": model.global_range,
            "soft_clip_scale": model.soft_clip_scale,
            "clip_scale": model.clip_scale,
        },
        "train_dataset_params": train_dataset_params,
        "feature_lists": feature_lists,
        "torch_dtype": "float16" if float16_weights else "float32",
        "preproc_feature_names_out": list(getattr(preprocessing, "get_feature_names_out", lambda: [])()),
    }

    meta_path = os.path.join(path, meta_json_filename)
    with open(meta_path, "wb") as f:
        f.write(_safe_json_dump(meta))

    weights_size_mb = os.path.getsize(weights_zip_path) / (1024 * 1024)
    prep_size_mb = os.path.getsize(prep_path) / (1024 * 1024)
    meta_size_kb = os.path.getsize(meta_path) / 1024.0
    print(f"Saved: weights={weights_size_mb:.2f} MB, preprocessing={prep_size_mb:.2f} MB, meta={meta_size_kb:.1f} KB → dir={path}")

# Требуется наличие в окружении:
# - SequenceTransformerRegressor
# - TimeSeriesDataSet
# - CustomTFT, TFTAdapter
# - tft_output_transformer
# - PeakFriendlyHuber
# - to_dense
# - вспомогательные функции _load_bytes_from_zip, _safe_json_load из предыдущей версии


def load_transformer_exact(
    path: str,
    *,
    preprocessing_filename: str = "preprocessing.joblib.lzma",
    model_zip_filename: str = "model_weights.zip",
    meta_json_filename: str = "meta.json",
    device: Optional[str] = None,
) -> Pipeline:
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'

    warnings.filterwarnings("ignore", message=".*Attribute 'loss' is an instance of `nn.Module`.*")
    warnings.filterwarnings("ignore", message=".*Attribute 'logging_metrics' is an instance of `nn.Module`.*")

    # 1) preprocessing
    prep_path = os.path.join(path, preprocessing_filename)
    with lzma.open(prep_path, "rb") as f:
        preprocessing = joblib.load(f)

    # 2) meta
    meta_path = os.path.join(path, meta_json_filename)
    if not os.path.isfile(meta_path):
        raise FileNotFoundError(meta_path)
    with open(meta_path, "rb") as f:
        meta = json.loads(f.read().decode("utf-8"))

    params = meta["params"]
    feature_columns = meta["feature_columns"]
    n_transformed = meta["n_transformed"]
    n_feat = meta["n_feat"]
    global_stats = meta["global_stats"]
    tds_params = meta["train_dataset_params"]
    feature_lists = meta.get("feature_lists", {})
    saved_torch_dtype = meta.get("torch_dtype", "float32")

    # 3) регрессор
    model = SequenceTransformerRegressor(
        seq_len=params["seq_len"],
        pred_len=params["pred_len"],
        hidden_size=params["hidden_size"],
        hidden_continuous_size=params["hidden_continuous_size"],
        epochs=params["epochs"],
        batch_size=params["batch_size"],
        learning_rate=params["learning_rate"],
        patience=params["patience"],
        seed=params["seed"],
        dropout=params["dropout"],
        weight_decay=params["weight_decay"],
        verbose=params["verbose"],
        mask_prob=params["mask_prob"],
        infer_stride=params["infer_stride"],
        ckpt_path=None,
        preprocessing=preprocessing,
        min_encoder_length=params.get("min_encoder_length", 1),
    )

    model._feature_columns = feature_columns
    model._n_transformed = n_transformed
    model._n_feat = n_feat
    model.global_target_mean = float(global_stats["global_target_mean"])
    model.global_target_std = float(global_stats["global_target_std"])
    model.global_target_min = float(global_stats["global_target_min"])
    model.global_target_max = float(global_stats["global_target_max"])
    model.global_range = float(global_stats["global_range"])
    model.soft_clip_scale = float(global_stats["soft_clip_scale"])
    model.clip_scale = float(global_stats.get("clip_scale", 1.5))
    model.device = device

    # 4) синтетический df
    max_enc = int(tds_params["max_encoder_length"])
    max_pred = int(tds_params["max_prediction_length"])
    num_feat_cols = len(feature_columns)
    rows = max_enc + max_pred

    synth_df_base = pd.DataFrame(
        np.zeros((rows, num_feat_cols), dtype=np.float32),
        columns=feature_columns,
    )
    synth_df_base["time_idx"] = np.arange(rows, dtype=np.int64)
    synth_df_base["target"] = np.zeros(rows, dtype=np.float32)
    synth_df_base["batch"] = 0

    # Попытка №1: «обычная» конструкция TDS как при обучении
    tds_kwargs = dict(
        time_idx=tds_params["time_idx"],
        target=tds_params["target"],
        group_ids=tds_params["group_ids"],
        max_encoder_length=tds_params["max_encoder_length"],
        max_prediction_length=tds_params["max_prediction_length"],
        target_normalizer=None,
        allow_missing_timesteps=tds_params["allow_missing_timesteps"],
        add_relative_time_idx=tds_params["add_relative_time_idx"],
        add_target_scales=tds_params["add_target_scales"],
        add_encoder_length=tds_params["add_encoder_length"],
        min_prediction_length=tds_params["min_prediction_length"],
        min_encoder_length=tds_params.get("min_encoder_length", 0),
    )

    if "time_varying_unknown_reals" in feature_lists:
        tds_kwargs["time_varying_unknown_reals"] = list(feature_lists["time_varying_unknown_reals"])
    else:
        tds_kwargs["time_varying_unknown_reals"] = list(feature_columns)

    for key in [
        "time_varying_known_reals",
        "static_reals",
        "time_varying_known_categoricals",
        "time_varying_unknown_categoricals",
        "static_categoricals",
        "known_reals",
        "unknown_reals",
        "known_categoricals",
        "unknown_categoricals",
    ]:
        if key in feature_lists:
            tds_kwargs[key] = list(feature_lists[key])

    dataset = TimeSeriesDataSet(synth_df_base.copy(), **tds_kwargs)

    # Проверяем порядок reals
    saved_reals = feature_lists.get("reals")
    rebuild_forced = False
    if saved_reals is not None:
        # В PF список dataset.reals может быть кортежами/объектами -- приводим к строкам
        ds_reals = list(getattr(dataset, "reals", []))
        ds_reals = [str(x) for x in ds_reals]
        saved_reals_str = [str(x) for x in saved_reals]
        if ds_reals != saved_reals_str:
            rebuild_forced = True

    if rebuild_forced:
        # Попытка №2: Форсируем порядок каналов reals.
        # Для этого отключим автоматические добавления и сами сгенерируем служебные колонки
        # encoder_length и relative_time_idx в synth_df.
        synth_df = synth_df_base.copy()
        # relative_time_idx: от 0 до rows-1
        synth_df["relative_time_idx"] = np.arange(rows, dtype=np.int64)
        # encoder_length: длина encoder для каждой позиции; для синтетики можно установить константу max_enc
        # PF ожидает целочисленную encoder_length, соответствующую длине энкодера на каждом шаге.
        # Для инициализации архитектуры достаточно положить валидные числа.
        enc_len = np.zeros(rows, dtype=np.int64)
        enc_len[:max_enc] = np.arange(1, max_enc + 1, dtype=np.int64)
        enc_len[max_enc:] = max_enc
        synth_df["encoder_length"] = enc_len

        # Теперь задаём TDS без add_* фичей, и передаём time_varying_unknown_reals в порядке saved_reals.
        # saved_reals начинается с ["encoder_length","relative_time_idx", ... f0..fN]
        tds_kwargs_forced = dict(
            time_idx=tds_params["time_idx"],
            target=tds_params["target"],
            group_ids=tds_params["group_ids"],
            max_encoder_length=tds_params["max_encoder_length"],
            max_prediction_length=tds_params["max_prediction_length"],
            target_normalizer=None,
            allow_missing_timesteps=tds_params["allow_missing_timesteps"],
            add_relative_time_idx=False,
            add_target_scales=tds_params["add_target_scales"],
            add_encoder_length=False,
            min_prediction_length=tds_params["min_prediction_length"],
            min_encoder_length=tds_params.get("min_encoder_length", 0),
            time_varying_unknown_reals=list(saved_reals),  # порядок каналов фиксируем здесь
            # Не задаём known/unknown cats/reals дополнительно, чтобы не нарушить порядок
        )
        dataset = TimeSeriesDataSet(synth_df, **tds_kwargs_forced)

        # Контроль: проверим снова порядок
        ds_reals2 = [str(x) for x in list(getattr(dataset, "reals", []))]
        if ds_reals2 != [str(x) for x in saved_reals]:
            raise RuntimeError(
                f"Failed to force reals order. Got {ds_reals2[:8]}..., expected {list(saved_reals)[:8]}..."
            )

    # 5) TFT 1-в-1
    tft = CustomTFT.from_dataset(
        dataset,
        hidden_size=int(model.hidden_size),
        output_size=int(params.get("output_size", 1)),
        loss=PeakFriendlyHuber(
            delta=0.5,
            peak_thr=0.85,
            peak_weight=1.3,
            contrast_weight=0.03,
            center_band=0.3,
            clip_scale=1.5,
        ),
        optimizer="adam",
        learning_rate=float(model.learning_rate),
        lstm_layers=int(params.get("lstm_layers", 3)),
        hidden_continuous_size=int(model.hidden_continuous_size),
        attention_head_size=int(params.get("attention_head_size", 4)),
        dropout=float(model._dropout),
        reduce_on_plateau_patience=5,
        reduce_on_plateau_min_lr=1e-6,
        weight_decay=float(model.weight_decay),
        mask_prob=float(model.mask_prob),
        output_transformer=tft_output_transformer,
    )

    model._model = TFTAdapter(tft)
    model._train_dataset = dataset

    # 6) загрузка весов
    weights_zip_path = os.path.join(path, model_zip_filename)
    with zipfile.ZipFile(weights_zip_path, mode='r') as zf:
        if "tft_state_dict.pt" not in zf.namelist():
            raise RuntimeError("tft_state_dict.pt not found in weights zip")
        state = torch.load(io.BytesIO(zf.read("tft_state_dict.pt")), map_location="cpu")

    missing, unexpected = tft.load_state_dict(state, strict=False)
    if missing or unexpected:
        warnings.warn(f"load_state_dict: missing={missing}, unexpected={unexpected}")

    # 7) eval + dropout off
    tft.eval()
    for m in tft.modules():
        if isinstance(m, torch.nn.Dropout):
            m.p = 0.0

    model._model.to(device)

    restored = Pipeline([
        ("preprocessing", preprocessing),
        ("to_dense", FunctionTransformer(to_dense)),
        ("model", model),
    ])
    return restored

def apply_linear_calibration(y_pred: np.ndarray, calib: dict) -> np.ndarray:
    a, b = calib.get('a', 1.0), calib.get('b', 0.0)
    return a * np.ravel(y_pred) + b

def ranker_postprocess_minus1_1(y_pred_ranker: np.ndarray) -> np.ndarray:
    yp = np.ravel(y_pred_ranker)
    return 2.0 * ((yp + 15.0) / (5.0 + 15.0)) - 1.0

def load_optimized_pipeline(out_dir, model_type='regressor'):
    """
    Загружает оптимизированный pipeline.
    Возвращает полный pipe.
    """
    # Загружаем preproc
    preproc_file = f'preproc_{model_type}.pkl' if model_type == 'ranker' else 'preproc.pkl'
    preproc = joblib.load(os.path.join(out_dir, preproc_file))

    # Booster из gz
    gz_file = os.path.join(out_dir, f'{model_type}.txt.gz')
    temp_file = os.path.join(out_dir, f'temp_{model_type}.txt')
    with gzip.open(gz_file, 'rb') as f_in:
        with open(temp_file, 'wb') as f_out:
            f_out.write(f_in.read())
    booster = lgb.Booster(model_file=temp_file)
    os.remove(temp_file)

    # Реконструируем модель (LGBM wrapper)
    step_name = 'model' if model_type == 'regressor' else 'ranker'
    if model_type == 'regressor':
        model = lgb.LGBMRegressor()
    elif model_type == 'ranker':
        model = lgb.LGBMRanker()
    else:
        raise ValueError("Unknown model_type")

    model._Booster = booster
    model.fitted_ = True  # Отметить как обученную
    # Устанавливаем n_features_in_ из booster
    model.n_features_in_ = booster.num_feature()

    # Полный pipe
    pipe = Pipeline([
        ('preprocessing', preproc),
        ('to_dense', FunctionTransformer(to_dense, feature_names_out="one-to-one")),
        (step_name, model),
    ])

    return pipe

def load_model_for_ticker(ticker, models_dir=r'C:\Users\aleksandrovva1\Desktop\data science\0-trade\t\models'):
    """
    Загружает модель(и) для тикера из директории models//.
    Возвращает dict с: pipe_reg, pipe_rank (если combined), meta, threshold.
    """
    out_dir = os.path.join(models_dir, ticker)
    if not os.path.exists(out_dir):
        raise FileNotFoundError(f"Directory for {ticker} not found: {out_dir}")

    # Загружаем meta
    with open(os.path.join(out_dir, 'meta.json'), 'r') as f:
        meta = json.load(f)

    best_method = meta['best_method']
    thresh = meta.get('sell_threshold', 0.0)  # Fallback

    pipe_reg = None
    pipe_rank = None

    if best_method == 'regressor':
        pipe_reg = load_optimized_pipeline(out_dir, 'regressor')

    elif best_method == 'ranker':
        pipe_rank = load_optimized_pipeline(out_dir, 'ranker')

    elif best_method == 'combined':
        pipe_reg = load_optimized_pipeline(out_dir, 'regressor')
        pipe_rank = load_optimized_pipeline(out_dir, 'ranker')

    else:
        raise ValueError(f"Unknown best_method: {best_method}")

    return {
        'best_method': best_method,
        'pipe_reg': pipe_reg,
        'pipe_rank': pipe_rank,
        'meta': meta,
        'threshold': thresh,
        'calib_reg': meta.get('reg_calibration', {'a': 1.0, 'b': 0.0}),
        'w_reg': meta.get('best_w_reg', 0.5) if best_method == 'combined' else None
    }


def calculate_mif(ticker_symbol, benchmark_symbol,
                  window=20, volatility_ratio_weight=0.6):
    """
    MIF для российских акций с учетом особенностей рынка.

    :param ticker_symbol: Тикер акции (например: 'SBER.ME', 'GAZP.ME')
    :param benchmark_symbol: Эталонный индекс (по умолчанию IMOEX.ME)
    """
    # Загрузка данных с учетом постсуффиксов Московской биржи
    data = pd.DataFrame(
        {'ticker_symbol': close_price[ticker_symbol], 'benchmark_symbol': close_price[benchmark_symbol]})

    # Фильтрация некорректных данных
    data = data.dropna(how='all').ffill().bfill()

    # Рассчет доходностей
    returns = data[ticker_symbol].pct_change().dropna()
    bench_returns = data[benchmark_symbol].pct_change().dropna()

    # Выравнивание индексов
    aligned_returns, aligned_bench = returns.align(bench_returns, join='inner')

    # Расчет компонентов
    rolling_corr = aligned_returns.rolling(window).corr(aligned_bench)
    volatility_ratio = aligned_bench.rolling(window).std() / aligned_returns.rolling(window).std()

    # Нормализация
    norm_corr = (rolling_corr + 1) / 2
    norm_vol_ratio = np.clip(volatility_ratio, 0, 2) / 2

    # Итоговый индекс
    mif = (volatility_ratio_weight * norm_vol_ratio +
           (1 - volatility_ratio_weight) * norm_corr)

    return mif.dropna()


def send_buy_signal_to_telegram(ticker, price):
    """Отправка сигнала покупки в Telegram."""
    if not tg_channels or 'a1_screen' not in tg_channels:
        return

    message = f"Сигнал покупки: {ticker} на уровне {price}"
    bot.send_message(tg_channels['a1_screen'], message)


def send_sell_signal_to_telegram(ticker, price):
    """Отправка сигнала продажи в Telegram."""
    if not tg_channels or 'a1_screen' not in tg_channels:
        return

    message = f"Сигнал продажи: {ticker} на уровне {price}"
    bot.send_message(tg_channels['a1_screen'], message)


def send_error_to_telegram(error_message):
    """Отправка сообщения об ошибке в Telegram."""
    if not tg_channels or 'error' not in tg_channels:
        return

    bot.send_message(tg_channels['error'], f"Ошибка: {error_message}")


def calculate_window_sums(arr, window_size=9):
    """Вычисление сумм в окне."""
    if not arr:
        return []
    return [sum(arr[max(0, i - window_size + 1):i + 1]) for i in range(len(arr))]


def get_formatted_time(timestamp):
    dt_object = datetime.datetime.fromtimestamp(timestamp / 1000.0)
    time_str = dt_object.strftime("%Y-%m-%d %H:%M:%S.%f")
    return time_str


########################################################################################


def prepare_regime_params(optuna_params):
    """
    Преобразует параметры из формата Optuna в два словаря: базовые параметры режимов и параметры расчета.

    Args:
        optuna_params (dict): Словарь с параметрами из Optuna

    Returns:
        dict: Словарь с двумя ключами: 'base_params' (параметры режимов) и 'calc_params' (остальные параметры)
    """
    # Инициализируем словари для базовых параметров и параметров расчета
    start_params = {}
    base_params = {}
    calc_params = {}

    # Сначала обрабатываем параметры режимов (0-4)

    start_params['moving_average_length'] = optuna_params.get('moving_average_length', 14)
    start_params['atr_period'] = optuna_params.get('atr_period', 10)
    for regime in range(5):
        regime_key = f'regime_{regime}_'
        regime_params = {}

        # Основные параметры режима
        regime_params['average_type'] = optuna_params.get(f'{regime_key}average_type', 'SMA')
        regime_params['moving_average_length'] = optuna_params.get(f'{regime_key}ma_length', 50)
        regime_params['atr_period'] = optuna_params.get(f'{regime_key}atr_period', 14)
        regime_params['atr_multiplier'] = optuna_params.get(f'{regime_key}atr_multiplier', 3.0)

        # Параметры AMA, если они есть
        ama_atr_period = optuna_params.get(f'{regime_key}ama_atr_period')
        ama_min_period = optuna_params.get(f'{regime_key}ama_min_period')
        ama_max_period = optuna_params.get(f'{regime_key}ama_max_period')

        if regime_params['average_type'] == 'AMA' and all(p is not None for p in [ama_atr_period, ama_min_period, ama_max_period]):
            regime_params['ama_params'] = {
                'atr_period': int(ama_atr_period),
                'min_period': int(ama_min_period),
                'max_period': int(ama_max_period)
            }

        base_params[regime] = regime_params

    # Теперь собираем все остальные параметры в calc_params
    other_params = [
        'rsi_length', 'use_smoothing', 'smoothing_length', 'smoothing_type',
        'alma_sigma', 'rsi_overbought', 'rsi_oversold', 'use_knn',
        'knn_neighbors', 'knn_lookback', 'knn_weight', 'feature_count',
        'use_filter', 'filter_method', 'filter_strength', 'sma_length',
        'ema_length', 'rsi_helbuth'
    ]

    for param in other_params:
        if param in optuna_params:
            calc_params[param] = optuna_params[param]

    return {
        'start_params': start_params,
        'base_params': base_params,
        'calc_params': calc_params
    }

params_for_classifier = {
    'ppo_short_window': 27,
    'cmo_window': 40,
    'ppo_long_window': 127,
    'macd_fast_period': [91],
    'macd_slow_period': [76],
    'macd_signal_period': [39],
    'rsi_period': 197,
    'williams_r_window': 33,
    'overbought_rsi_period': 124,
    'overbought_stochastic_period': 200,
    'fear_greed_window': 121,
    'bollinger_window': 155,
    'atr_window': 159,
    'asset_growth_window': 5,
    'volume_window': 149,
    'volume_quantile': 0.8591071502449321
}

class ClassifierFeatureCalculator:
    def __init__(self, df, window=14):
        """
        Инициализация класса для расчета технических индикаторов.
        :param df: DataFrame с исходными данными
        :param window: окно для скользящих расчетов
        :param ticker: тикер актива
        """
        self.df = df.copy()
        self.WINDOW = window
        # self.ticker = ticker
        self._prepare_data()

    def _prepare_data(self):
        """
        Подготовка данных: проверка индекса, сортировка и базовые преобразования.
        """
        self.df = self.df.reset_index(drop=True)
        # Убедимся, что данные имеют правильный формат
        self.df = self.df[['time', 'open', 'close', 'high', 'low', 'volume', 'buy_signal', 'target']].apply(
            pd.to_numeric, errors='coerce'
        )
        # self.df['ticker'] = self.ticker

    def calculate_cmo(self, cmo_window=14):
        """
        Расчет Chande Momentum Oscillator (CMO) для всего DataFrame.
        """
        close = self.df['close']

        # Расчет суммы высоких и низких цен за период
        # Используем векторизованные операции вместо apply
        max_window = close.rolling(cmo_window).max()
        min_window = close.rolling(cmo_window).min()

        # Расчет количества максимумов и минимумов в окне
        # Используем broadcast для сравнения
        is_max = close.rolling(cmo_window).apply(lambda x: (x == x.max()).sum(), raw=True)
        is_min = close.rolling(cmo_window).apply(lambda x: (x == x.min()).sum(), raw=True)

        sum_high = is_max * max_window
        sum_low = is_min * min_window

        # Расчет CMO
        cmo = 100 * (sum_high - sum_low) / (sum_high + sum_low + 1e-10)
        self.df['CMO'] = cmo
        self.df['ago20_CMO'] = self.df['CMO'].shift(20)
        self.df['ago50_CMO'] = self.df['CMO'].shift(50)

        return self.df

    def calculate_williams_r(self, williams_r_window=14):
        """
        Расчет Williams %R для всего DataFrame.
        """
        high = self.df['high']
        low = self.df['low']
        close = self.df['close']

        highest_high = high.rolling(williams_r_window).max()
        lowest_low = low.rolling(williams_r_window).min()

        williams_r = ((highest_high - close) / (highest_high - lowest_low)) * (-100)
        self.df['WILLR'] = williams_r
        self.df['ago50_WILLR'] = self.df['WILLR'].shift(50)
        self.df['ago20_WILLR'] = self.df['WILLR'].shift(20)
        self.df['ago3_WILLR'] = self.df['WILLR'].shift(3)

    def calculate_macd(self, macd_fast_periods=[12], macd_slow_periods=[26], macd_signal_periods=[9]):
        """
        Быстрый расчет нормализованного MACD с использованием векторизованных операций
        """
        close = self.df['close']

        # Создаем множества для уникальных периодов
        unique_fast = set(macd_fast_periods)
        unique_slow = set(macd_slow_periods)

        # Предварительно вычисляем все необходимые EMA и скользящие средние
        ema_cache = {}
        rolling_cache = {}

        # Кешируем быстрые EMA
        for fp in unique_fast:
            ema_cache[f'ema_{fp}'] = close.ewm(span=fp, adjust=False).mean()

        # Кешируем медленные EMA и скользящие средние
        for sp in unique_slow:
            ema_cache[f'ema_{sp}'] = close.ewm(span=sp, adjust=False).mean()
            rolling_cache[f'rolling_{sp}'] = close.rolling(window=sp).mean()

        # Основной цикл вычислений
        for fp in macd_fast_periods:
            ema_fast = ema_cache[f'ema_{fp}']
            for sp in macd_slow_periods:
                ema_slow = ema_cache[f'ema_{sp}']
                rolling_mean = rolling_cache[f'rolling_{sp}']

                # Вычисляем MACD и нормализацию
                macd = ema_fast - ema_slow
                macd_norm = macd / rolling_mean

                # Сохраняем MACD только один раз для комбинации fp/sp
                self.df[f'MACD'] = macd_norm

                # Обрабатываем сигнальные периоды
                for sig in macd_signal_periods:
                    # Вычисляем сигнальную линию
                    signal = macd.ewm(span=sig, adjust=False).mean()
                    signal_norm = signal / rolling_mean

                    # Сохраняем результаты
                    self.df[f'MACD_Signal'] = signal_norm
                    self.df[f'MACD_Hist'] = macd_norm - signal_norm
        return self.df

    def calculate_rsi(self, rsi_period=14):
        """
        Расчет RSI и его сдвиги.
        """
        close = self.df['close']
        delta = close.diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(rsi_period).mean()
        avg_loss = loss.rolling(rsi_period).mean()
        rs = avg_gain / (avg_loss + 1e-10)
        rsi = 100 - (100 / (1 + rs))

        self.df['RSI'] = rsi
        self.df['ago10_RSI'] = self.df['RSI'].shift(10)
        self.df['ago3_RSI'] = self.df['RSI'].shift(3)
        self.df['ago50_RSI'] = self.df['RSI'].shift(50)
        self.df['ago1_RSI'] = self.df['RSI'].shift(1)
        self.df['ago5_RSI'] = self.df['RSI'].shift(5)

    def calculate_overbought_oversold_index(self, rsi_window=14, stoch_window=14):
        """
        Расчет индекса перекупленности/перепроданности.
        """
        close = self.df['close']
        low = self.df['low']
        high = self.df['high']

        # RSI
        delta = close.diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(rsi_window).mean()
        avg_loss = loss.rolling(rsi_window).mean()
        rs = avg_gain / (avg_loss + 1e-10)
        rsi = 100 - (100 / (1 + rs))

        # Stochastic
        lowest_low = low.rolling(stoch_window).min()
        highest_high = high.rolling(stoch_window).max()
        stochastic = 100 * (close - lowest_low) / (highest_high - lowest_low + 1e-10)

        # Индекс перекупленности/перепроданности
        overbought_oversold = (rsi + stochastic) / 2
        self.df['Overbought_Oversold_Index'] = overbought_oversold
        self.df['ago50_Overbought_Oversold_Index'] = self.df['Overbought_Oversold_Index'].shift(50)
        self.df['ago10_Overbought_Oversold_Index'] = self.df['Overbought_Oversold_Index'].shift(10)

    def calculate_fear_greed_index(self, fear_greed_window=25):
        """
        Расчет индекса страха и жадности и сдвигов.
        """
        close = self.df['close']
        volume = self.df['volume']

        # Пример расчета страха/жадности
        volatility = close.pct_change().rolling(window=fear_greed_window).std()
        volume_change = volume.pct_change().rolling(window=fear_greed_window).mean()
        price_trend = close / close.rolling(window=fear_greed_window).mean()

        fear_greed = (volatility + volume_change + price_trend) / 3 * 100
        self.df['Fear_Greed_Index'] = fear_greed
        self.df['ago50_Fear_Greed_Index'] = self.df['Fear_Greed_Index'].shift(50)
        self.df['ago30_Fear_Greed_Index'] = self.df['Fear_Greed_Index'].shift(30)

    def calculate_bollinger_bands(self, bollinger_window=20):
        """
        Расчет Bollinger Bands (ширины полос) и сдвигов.
        """
        close = self.df['close']
        ma = close.rolling(bollinger_window).mean()
        std = close.rolling(bollinger_window).std()
        bb_width = (2 * std) / ma

        self.df['BB_Width'] = bb_width
        self.df['ago30_BB_Width'] = self.df['BB_Width'].shift(30)

    def calc_price_features(self):
        # Добавляем защиту от деления на ноль
        epsilon = 1e-10

        self.df['perc_var_open_close'] = ((self.df['close'] - self.df['open']) / (self.df['open'] + epsilon)) * 100
        self.df['candle_range_perc'] = ((self.df['high'] - self.df['low']) / (self.df['open'] + epsilon)) * 100

        self.df['ago3_perc_var_open_close'] = self.df['perc_var_open_close'].shift(3)
        self.df['ago5_perc_var_open_close'] = self.df['perc_var_open_close'].shift(5)
        self.df['ago30_perc_var_open_close'] = self.df['perc_var_open_close'].shift(30)

        self.df['ago30_candle_range_perc'] = self.df['candle_range_perc'].shift(30)
        self.df['ago5_candle_range_perc'] = self.df['candle_range_perc'].shift(5)
        self.df['ago3_candle_range_perc'] = self.df['candle_range_perc'].shift(3)

    def calculate_overbought_oversold_index(self, overbought_rsi_period=25, overbought_stochastic_period=50):
        """
        Расчет индекса перекупленности/перепроданности для нескольких окон с оптимизацией.
        :param rsi_periods: Список периодов для RSI.
        :param stochastic_periods: Список периодов для стохастического осциллятора.
        """
        df = self.df

        # Предварительные вычисления для RSI
        delta = df['close'].diff()
        gain = delta.where(delta > 0, 0)
        loss = (-delta).where(delta < 0, 0)

        avg_gain = gain.rolling(window=overbought_rsi_period).mean()
        avg_loss = loss.rolling(window=overbought_rsi_period).mean()
        rs = avg_gain / (avg_loss + 1e-10)
        rsi = 100 - (100 / (1 + rs))

        # Предварительные вычисления для Stochastic
        stochastic_dict = {}
        low_min = df['low'].rolling(window=overbought_stochastic_period).min()
        high_max = df['high'].rolling(window=overbought_stochastic_period).max()
        stochastic = 100 * (df['close'] - low_min) / (high_max - low_min + 1e-10)

        # Генерация всех комбинаций и расчет индекса
        combined_index = (rsi + stochastic) / 2
        self.df[f'Overbought_Oversold_Index'] = combined_index
        self.df[f'ago10_Overbought_Oversold_Index'] = self.df[f'Overbought_Oversold_Index'].shift(10)
        self.df[f'ago50_Overbought_Oversold_Index'] = self.df[f'Overbought_Oversold_Index'].shift(50)

    def calculate_atr(self, atr_window=14):
        """
        Расчет ATR и его сдвигов.
        """
        high = self.df['high']
        low = self.df['low']
        close = self.df['close']

        tr1 = high - low
        tr2 = np.abs(high - close.shift(1))
        tr3 = np.abs(low - close.shift(1))
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        atr = tr.rolling(atr_window).mean()

        self.df['ATR'] = atr
        self.df['ago10_ATR'] = self.df['ATR'].shift(10)

    def calculate_ppo(self, short_window=12, long_window=26):
        """
        Расчет Percentage Price Oscillator (PPO) для всего DataFrame.
        """
        close = self.df['close']

        # Расчет скользящих средних
        short_ma = close.rolling(short_window).mean()
        long_ma = close.rolling(long_window).mean()

        # Расчет PPO
        ppo = ((short_ma - long_ma) / long_ma) * 100
        self.df['PPO'] = ppo
        self.df['ago50_PPO'] = self.df['PPO'].shift(50)

    def calculate_asset_growth(self, asset_growth_window=3):
        """
        Расчет Asset Growth и его сдвигов.
        """
        asset = self.df['close']
        asset_growth = asset.pct_change(asset_growth_window).fillna(0) * 100
        self.df['Asset_Growth'] = asset_growth
        self.df['ago50_Asset_Growth'] = self.df['Asset_Growth'].shift(50)
        self.df['ago5_Asset_Growth'] = self.df['Asset_Growth'].shift(5)
        self.df['ago30_Asset_Growth'] = self.df['Asset_Growth'].shift(30)

    def calculate_asset_to_equity_ratio(self):
        """
        Расчет Asset to Equity Ratio и его сдвигов.
        """
        asset = self.df['close']
        equity = self.df['low']
        ratio = asset / (equity + 1e-10)

        self.df['Asset_To_Equity_Ratio'] = ratio
        self.df['ago50_Asset_To_Equity_Ratio'] = self.df['Asset_To_Equity_Ratio'].shift(50)
        self.df['ago5_Asset_To_Equity_Ratio'] = self.df['Asset_To_Equity_Ratio'].shift(5)
        self.df['ago30_Asset_To_Equity_Ratio'] = self.df['Asset_To_Equity_Ratio'].shift(30)
        self.df['ago3_Asset_To_Equity_Ratio'] = self.df['Asset_To_Equity_Ratio'].shift(3)

    def calculate_volume_quantile_deviation(self, volume_window=100, volume_quantile=0.65):
        volume = self.df['volume']
        quantile_value = volume.rolling(window=volume_window).quantile(volume_quantile)
        quantile_str = f"{volume_quantile:.2f}".replace('.', '_')  # Преобразование в строку
        self.df[f"Volume_Quantile_Deviation_%"] = ((volume - quantile_value) / quantile_value) * 100
        self.df[f"ago3_Volume_Quantile_Deviation_%"] = self.df[f"Volume_Quantile_Deviation_%"].shift(3)

    def calculate_anomaly_score(self):
        """
        Расчет аномалий через изолированный лес.
        """
        self.df['price_change'] = self.df['close'].pct_change().fillna(0)
        self.df['anomaly_score'] = IsolationForest(contamination=0.05).fit_predict(
            self.df[['price_change', 'volume']]
        )
        self.df['ago1_anomaly_score'] = self.df['anomaly_score'].shift(1)
        self.df['ago20_anomaly_score'] = self.df['anomaly_score'].shift(20)
        self.df['ago30_anomaly_score'] = self.df['anomaly_score'].shift(30)

        self.df['ago50_price_change30'] = self.df['close'].pct_change(30).fillna(0).shift(50)
        self.df['ago30_price_change50'] = self.df['close'].pct_change(50).fillna(0).shift(30)
        self.df['ago50_price_change'] = self.df['price_change'].shift(50)
        self.df['ago10_price_change50'] = self.df['close'].pct_change(50).fillna(0).shift(10)
        self.df['ago30_price_change60'] = self.df['close'].pct_change(60).fillna(0).shift(30)
        self.df['ago30_price_change'] = self.df['price_change'].shift(30)
        self.df['ago10_price_change70'] = self.df['close'].pct_change(70).fillna(0).shift(10)

    def calculate_perc_var_open_close(self):
        """
        Расчет процентного изменения открытия и закрытия.
        """
        self.df['perc_var_open_close'] = (
                                                 (self.df['close'] - self.df['open']) / (self.df['open'] + 1e-10)
                                         ) * 100
        self.df['ago50_perc_var_open_close'] = self.df['perc_var_open_close'].shift(50)

    def calculate_trend(self, pct_window=6, window=464):
        # Рассчитываем returns с заполнением вперед
        returns = self.df['close'].pct_change(pct_window).fillna(method='ffill').fillna(0)
        # Убедимся, что данных достаточно для STL
        if len(returns) < 2 * window:
            raise ValueError(f"Need at least {2 * window} data points for window={window}")

        res = STL(returns, period=window).fit()

        # Сбрасываем индекс для корректного shift()
        # df = self.df.reset_index(drop=True)

        self.df['trend'] = res.trend
        self.df['ago50_trend'] = self.df['trend'].shift(50)

        self.df['seasonal'] = res.seasonal
        self.df['ago1_seasonal'] = self.df['seasonal'].shift(1)

    def calculate_features(self, **params):
        self.calculate_cmo(params.get('cmo_window', 14))
        self.calculate_ppo(params.get('ppo_short_window', 12), params.get('ppo_long_window', 26))
        self.calculate_macd(params.get('macd_fast_period', [12]),
                            params.get('macd_slow_period', [26]),
                            params.get('macd_signal_period', [9]))
        self.calculate_rsi(params.get('rsi_period', 14))
        self.calculate_williams_r(params.get('williams_r_window', 14))
        self.calculate_overbought_oversold_index(params.get('overbought_rsi_period', 14),
                                                 params.get('overbought_stochastic_period', 14))
        self.calculate_fear_greed_index(params.get('fear_greed_window', 25))
        self.calculate_bollinger_bands(params.get('bollinger_window', 20))
        self.calculate_atr(params.get('atr_window', 14))
        self.calculate_asset_growth(params.get('asset_growth_window', 3))
        self.calculate_anomaly_score()
        self.calculate_volume_quantile_deviation(params.get('volume_window', 100),
                                                 params.get('volume_quantile', 0.65))
        self.calculate_asset_to_equity_ratio()
        self.calculate_trend()
        self.calculate_perc_var_open_close()
        self.calc_price_features()

        numeric_cols = self.df.select_dtypes(include=[np.float64]).columns
        self.df[numeric_cols] = self.df[numeric_cols].astype(np.float32)

        return self.df.iloc[[-1]].copy()


####################################################################

def find_klines(ticker, lookback=40):
    candles = []
    try:
        # Получаем временные метки свечей
        time_list = time_last_kline_end[ticker]  # Формат: ['2025-06-03T14:30:00+00:00', ...]

        # Получаем время покупки
        buy_time_str = open_trades[ticker]['buy_time']  # Может быть в любом формате

        # 1. Универсальное преобразование времени покупки в naive datetime
        if isinstance(buy_time_str, str):
            # Удаляем временную зону если она есть
            if '+' in buy_time_str:
                buy_time_str = buy_time_str.split('+')[0]
            # Удаляем микросекунды если они есть
            if '.' in buy_time_str:
                buy_time_str = buy_time_str.split('.')[0]

            # Парсим как naive datetime
            buy_time = dt.strptime(buy_time_str, "%Y-%m-%dT%H:%M:%S")
        else:
            buy_time = buy_time_str
            if buy_time.tzinfo is not None:  # Если есть временная зона
                buy_time = buy_time.replace(tzinfo=None)  # Удаляем временную зону

        # Добавляем 3 часа если нужно
        buy_time += timedelta(hours=3)
        buy_time = buy_time.replace(second=0, microsecond=0)

        # 2. Преобразуем список временных меток в datetime
        time_series = []
        for t in time_list:
            # Обрабатываем каждую временную метку аналогично buy_time
            if isinstance(t, str):
                if '+' in t:
                    t = t.split('+')[0]
                if '.' in t:
                    t = t.split('.')[0]
                t_dt = dt.strptime(t, "%Y-%m-%dT%H:%M:%S")
            else:
                t_dt = t
                if t_dt.tzinfo is not None:
                    t_dt = t_dt.replace(tzinfo=None)
            time_series.append(t_dt)

        # 3. Находим ближайшую временную метку
        closest_idx = 0
        min_diff = float('inf')
        for i, t in enumerate(time_series):
            diff = abs((t - buy_time).total_seconds())
            if diff < min_diff:
                min_diff = diff
                closest_idx = i

        # 4. Проверяем что разница не слишком большая (не более 30 минут)
        if min_diff > 1800:  # 30 минут в секундах
            print(f"Предупреждение: для {ticker} большая разница во времени: {min_diff/60:.1f} минут")

        # 5. Определяем диапазон свечей
        start_idx = max(0, closest_idx - lookback)
        end_idx = len(time_series)  # Берем все свечи до конца

        # 6. Формируем данные свечей
        candles.append({
            "time": time_last_kline_end[ticker][start_idx:end_idx],
            "open": open_price[ticker][start_idx:end_idx],
            "close": close_price[ticker][start_idx:end_idx],
            "high": high_price[ticker][start_idx:end_idx],
            "low": low_price[ticker][start_idx:end_idx],
            "volume": volume[ticker][start_idx:end_idx],
            "pmax": pmax[ticker][start_idx:end_idx],
            "ma": ma[ticker][start_idx:end_idx],
        })

    except Exception as e:
        bot.send_message(
                        self.error_tg,
                        f"{ticker} ошибка в find_klines"
                    )
        raise  # Пробрасываем исключение дальше

    return candles


async def fetch_tinkoff_candles(client, figi, start_time, end_time, interval):
    """Получение исторических данных из Tinkoff API (асинхронная версия)"""
    candles = []
    current_time = start_time

    while current_time < end_time:
        try:
            # Используем асинхронный генератор
            async for candle in client.get_all_candles(
                    figi=figi,
                    from_=current_time,
                    to=end_time,
                    interval=interval):
                candles.append({
                    "time": candle.time + timedelta(hours=3),
                    "open": float(candle.open.units + candle.open.nano * 1e-9),
                    "close": float(candle.close.units + candle.close.nano * 1e-9),
                    "high": float(candle.high.units + candle.high.nano * 1e-9),
                    "low": float(candle.low.units + candle.low.nano * 1e-9),
                    "volume": candle.volume,
                })
            break
        except Exception as e:
            print(f"Ошибка получения данных: {e}")
            break

    return candles


def prepare_tinkoff_data(candles):
    """Подготовка данных из Tinkoff в DataFrame"""
    if not candles:
        return pd.DataFrame()

    df = pd.DataFrame(candles[0])
    df['time'] = pd.to_datetime(df['time'])
    df = df.sort_values('time').drop_duplicates('time').reset_index(drop=True)
    df.set_index('time', inplace=True)
    return df


def calculate_indicators(df):
    """Расчет индикаторов для отчета"""
    if df.empty:
        return df

    # Вычисляем VAR (Volatility Adjusted Ratio)
    high_array = df['high'].values
    low_array = df['low'].values
    close_array = df['close'].values

    # Функции для расчета индикаторов из TinkoffInvestDataCollector
    def generate_var(high, low, period=10):
        hl2 = (high + low) / 2
        diff = np.diff(hl2, prepend=hl2[0])
        vud1 = np.where(diff > 0, diff, 0)
        vdd1 = np.where(diff < 0, -diff, 0)

        vUD = np.convolve(vud1, np.ones(9) / 9, mode='valid')
        vDD = np.convolve(vdd1, np.ones(9) / 9, mode='valid')

        vCMO = (vUD - vDD) / (vUD + vDD + 1e-10)
        vCMO = np.nan_to_num(vCMO, nan=0.0)

        alpha = 2 / (period + 1)
        var = np.zeros_like(hl2)
        var_before = 0.0

        for i in range(len(hl2)):
            if i < len(vCMO):
                var[i] = (alpha * abs(vCMO[i]) * hl2[i]) + (1 - alpha * abs(vCMO[i])) * var_before
            else:
                var[i] = var_before
            var_before = var[i]

        return var

    def generate_pmax(var, close, high, low, atr_period=14, atr_multiplier=3):
        """Расчет PMAX индикатора"""
        tr = np.maximum(high - low,
                        np.maximum(abs(high - np.roll(close, 1)),
                                   abs(low - np.roll(close, 1))))
        atr = pd.Series(tr).rolling(atr_period).mean().values

        pmax = []
        previous_final_upper = 0
        previous_final_lower = 0
        previous_var = 0
        previous_pmax = 0

        for i in range(len(close)):
            atrc = atr[i] if i < len(atr) and not np.isnan(atr[i]) else 0
            varc = var[i] if i < len(var) else 0

            upper = varc + atr_multiplier * atrc
            lower = varc - atr_multiplier * atrc

            final_upper = upper if (
                    upper < previous_final_upper or previous_var > previous_final_upper) else previous_final_upper
            final_lower = lower if (
                    lower > previous_final_lower or previous_var < previous_final_lower) else previous_final_lower

            if previous_pmax == previous_final_upper:
                pmaxc = final_upper if varc <= final_upper else final_lower
            else:
                pmaxc = final_lower if varc >= final_lower else final_upper

            pmax.append(pmaxc)
            previous_var = varc
            previous_final_upper = final_upper
            previous_final_lower = final_lower
            previous_pmax = pmaxc

        return pmax

    # Вычисляем индикаторы
    var = generate_var(high_array, low_array)
    pmax = generate_pmax(var, close_array, high_array, low_array, ATR_PERIOD, ATR_MULTIPLIER)

    # Добавляем в DataFrame
    df['VAR'] = var
    df['PMAX'] = pmax
    df['WMA'] = df['close'].rolling(MOVING_AVERAGE_LENGHT).mean()

    return df

def save_tinkoff_trade_chart(df, file_path, ticker, alert_price, alert_time,
                             exit_time, exit_price, pnl, reason):
    """Сохранение графика для сделки в Tinkoff"""
    if df.empty:
        return

    # Преобразуем цены в проценты относительно alert_price
    price_cols = ['open', 'high', 'low', 'close', 'pmax', 'ma']
    for col in price_cols:
        df[col] = ((df[col] - alert_price) / alert_price) * 100

    alert_price_percent = 0
    exit_price_percent = ((exit_price - alert_price) / alert_price) * 100

    trade_start_index = df.index.get_loc(pd.to_datetime(open_trades[ticker]['buy_time']).replace(second=0, microsecond=0).isoformat())

    # Преобразуем временные метки
    alert_time = pd.to_datetime(alert_time.replace(second=0, microsecond=0))
    exit_time = pd.to_datetime(exit_time.replace(second=0, microsecond=0))

    # Линии и точки на графике
    alert_line = [alert_price_percent] * len(df)

    # Вертикальные линии для времени входа и выхода
    trade_start_marker = np.full(len(df), np.nan)
    trade_start_marker[trade_start_index] = 0

    # Подготовка дополнительных графиков
    add_plot = [
        mpf.make_addplot(alert_line, color='gray', linestyle='--', label='Alert price', secondary_y=False),
        mpf.make_addplot(trade_start_marker, type='scatter', marker='v', markersize=100,
        color='orange', label='Trade Start'),
        mpf.make_addplot(df['pmax'], color='blue', label='PMAX', secondary_y=False),
        mpf.make_addplot(df['ma'], color='green', label=f'MA', secondary_y=False)
    ]

    '''if not reason:
         exit_scatter = np.full(len(df), np.nan)
         exit_idx = df.index.get_loc(exit_time)
         exit_scatter[exit_idx] = exit_price_percent
         add_plot.append(
            mpf.make_addplot(exit_scatter, type='scatter', marker='^', markersize=50, color='red', label='Exit', alpha=0.7))'''

    title = (f'\n\n\nTinkoff Trade Report; {ticker}; Alert time: {alert_time}; Alert price: {alert_price:.2f};\n'
             f'Exit time: {exit_time}; Exit price: {exit_price:.2f};\n'
             f'PNL: {pnl:.2f}%')

    # Сохранение графика
    mpf.plot(df, type='candle', style='charles', addplot=add_plot, title=title,
            ylabel='Price Change (%)', savefig=file_path, figsize=(15, 10))

def save_and_send_tinkoff_report(ticker, alert_price, trade_start_time, exit_time,
                                 exit_price, pnl, reason, link, tg_channels):
    """Сохранение и отправка отчета по сделке в Tinkoff"""
    try:
        # Конвертация времени
        #trade_start = dt.utcfromtimestamp(trade_start_time / 1000)
        #exit_dt = dt.utcfromtimestamp(exit_time / 1000)

        # Формирование данных результата
        results = {
            'symbol': ticker,
            'alert_time': trade_start_time,
            'alert_price': alert_price,
            'exit_time': exit_time,
            'exit_price': exit_price,
            'pnl': pnl,
            'reason': reason,
            'link': link
        }

        # Сохранение в индивидуальный файл
        df = pd.DataFrame([results])
        filename = f"tinkoff_trade_{ticker}_{trade_start_time.strftime('%Y%m%d_%H%M%S')}.xlsx"
        file_path = f'{path_to_save}reports/{filename}'
        os.makedirs(f'{path_to_save}reports', exist_ok=True)
        df.to_excel(file_path, index=False)

        # Работа с итоговым файлом за неделю
        week_number = trade_start_time.isocalendar()[1]
        year = trade_start_time.year
        summary_filename = f"tinkoff_trades_week_{week_number}_{year}.xlsx"
        summary_path = f'{path_to_save}reports/weekly/{summary_filename}'
        os.makedirs(f'{path_to_save}reports/weekly', exist_ok=True)

        if os.path.exists(summary_path):
            existing_df = pd.read_excel(summary_path)
            updated_df = pd.concat([existing_df, df], ignore_index=True)
        else:
            updated_df = df

        updated_df.to_excel(summary_path, index=False)

        # Отправка в Telegram (замените на вашу реализацию)
        if 'bot' in globals():
            with open(file_path, 'rb') as file:
                bot.send_document(tg_channels['sells'], file)

    except Exception as e:
        if 'bot' in globals():
            bot.send_message(tg_channels['error'], f"Ошибка в отчете Tinkoff: {e}")

class FastRollingMode:
    def __init__(self, window_size):
        self.window = deque(maxlen=window_size)
        self.counts = {}

    def update(self, new_val):
        if len(self.window) == self.window.maxlen:
            old_val = self.window.popleft()
            self.counts[old_val] -= 1
            if self.counts[old_val] == 0:
                del self.counts[old_val]

        self.window.append(new_val)
        self.counts[new_val] = self.counts.get(new_val, 0) + 1
        return max(self.counts.items(), key=lambda x: x[1])[0]


def extract_features(df: pd.DataFrame, window: int = 126):
    """
    Вычисляет устойчивые признаки для кластеризации рыночных режимов.
    """

    def calculate_macd(df, macd_fast_periods=[12], macd_slow_periods=[26], macd_signal_periods=[9]):
        """
        Быстрый расчет нормализованного MACD с использованием векторизованных операций
        """
        close = df['close']

        # Создаем множества для уникальных периодов
        unique_fast = set(macd_fast_periods)
        unique_slow = set(macd_slow_periods)


        # Предварительно вычисляем все необходимые EMA и скользящие средние
        ema_cache = {}
        rolling_cache = {}

        # Кешируем быстрые EMA
        for fp in unique_fast:
            ema_cache[f'ema_{fp}'] = close.ewm(span=fp, adjust=False).mean()

        # Кешируем медленные EMA и скользящие средние
        for sp in unique_slow:
            ema_cache[f'ema_{sp}'] = close.ewm(span=sp, adjust=False).mean()
            rolling_cache[f'rolling_{sp}'] = close.rolling(window=sp).mean()

        # Основной цикл вычислений
        for fp in macd_fast_periods:
            ema_fast = ema_cache[f'ema_{fp}']
            for sp in macd_slow_periods:
                ema_slow = ema_cache[f'ema_{sp}']
                rolling_mean = rolling_cache[f'rolling_{sp}']

                # Вычисляем MACD и нормализацию
                macd = ema_fast - ema_slow
                macd_norm = macd / rolling_mean

                # Сохраняем MACD только один раз для комбинации fp/sp

                # Обрабатываем сигнальные периоды
                for sig in macd_signal_periods:
                    # Вычисляем сигнальную линию
                    signal = macd.ewm(span=sig, adjust=False).mean()
                    signal_norm = signal / rolling_mean

        return pd.DataFrame([macd_norm, signal_norm, macd_norm - signal_norm]).T.fillna(0)

    def calculate_atr(df, atr_window=14):
        """
        Расчет ATR и его сдвигов.
        """
        high = df['high']
        low = df['low']
        close = df['close']

        tr1 = high - low
        tr2 = np.abs(high - close.shift(1))
        tr3 = np.abs(low - close.shift(1))
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        atr = tr.rolling(atr_window).mean()

        return pd.Series(atr).fillna(0)

    def calculate_rsi(df, rsi_period=14):
        """
        Расчет RSI и его сдвиги.
        """
        close = df['close']
        delta = close.diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(rsi_period).mean()
        avg_loss = loss.rolling(rsi_period).mean()
        rs = avg_gain / (avg_loss + 1e-10)
        rsi = 100 - (100 / (1 + rs))

        return pd.Series(rsi).fillna(0)

    def calculate_bollinger_bands(df, bollinger_window=20):
        """
        Расчет Bollinger Bands (ширины полос) и сдвигов.
        """
        close = df['close']
        ma = close.rolling(bollinger_window).mean()
        std = close.rolling(bollinger_window).std()
        bb_width = (2 * std) / ma

        return pd.Series(bb_width).fillna(0)

    macd_trend = calculate_macd(df, macd_slow_periods=[window], macd_fast_periods=[window//3],
                                 macd_signal_periods=[window//6])
    atr = calculate_atr(df, atr_window=window)
    rel_volatility = atr / df["close"]
    rsi_ind = calculate_rsi(df, rsi_period=window//2)
    volume_ratio = df['volume'].rolling(window).apply(
        lambda x: x[-1]/x.mean(), raw=True
    ).fillna(1).values

    features = np.column_stack([
        macd_trend,
        rel_volatility,
        rsi_ind,
        volume_ratio
    ])

    return features




####################################################################

class MachineLearningRSI:
    def __init__(self,
                 rsi_length=300,
                 use_smoothing=True,
                 smoothing_length=268,
                 smoothing_type='ALMA',
                 alma_sigma=6,
                 rsi_overbought=70,
                 rsi_oversold=30,
                 use_knn=True,
                 knn_neighbors=7,
                 knn_lookback=500,
                 knn_weight=0.6,
                 feature_count=5,
                 use_filter=True,
                 filter_method='Kalman',
                 filter_strength=0.7,
                 sma_length=20 + 7*24*4*3,
                 ema_length=21 + 7*24*4*3
                 ):

        # Базовые параметры
        self.rsi_length = rsi_length
        self.use_smoothing = use_smoothing
        self.smoothing_length = smoothing_length
        self.smoothing_type = smoothing_type
        self.alma_sigma = alma_sigma

        # Пороговые уровни
        self.rsi_overbought = rsi_overbought
        self.rsi_oversold = rsi_oversold

        # Параметры KNN
        self.use_knn = use_knn
        self.knn_neighbors = knn_neighbors
        self.knn_lookback = knn_lookback
        self.knn_weight = knn_weight
        self.feature_count = feature_count

        # Фильтрация
        self.use_filter = use_filter
        self.filter_method = filter_method
        self.filter_strength = filter_strength

        self.sma_length = sma_length
        self.ema_length = ema_length

    def calculate_rsi(self, close: pd.Series, length: int) -> pd.Series:
        """Расчет RSI через RMA аналогично PineScript ta.rsi"""
        delta = close.diff()
        gain = delta.clip(lower=0)
        loss = -delta.clip(upper=0)
        avg_gain = gain.ewm(alpha=1/length, min_periods=length, adjust=False).mean()
        avg_loss = loss.ewm(alpha=1/length, min_periods=length, adjust=False).mean()
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        return rsi

    def smooth(self, series: pd.Series) -> pd.Series:
        """Корректный ALMA"""
        if self.smoothing_type == 'SMA':
            return series.rolling(self.smoothing_length).mean()
        elif self.smoothing_type == 'EMA':
            return series.ewm(span=self.smoothing_length, adjust=False).mean()
        elif self.smoothing_type == 'ALMA':
            m = self.smoothing_length
            offset = 0.85
            sigma = self.alma_sigma

            def alma(series):
                window = np.arange(m)
                weights = np.exp(-((window - offset * (m-1))**2) / (2*(sigma**2)))
                weights /= weights.sum()
                return np.convolve(series, weights, mode='valid')

            def alma_causal(series: pd.Series, length: int = 9, offset: float = 0.85, sigma: float = 6) -> pd.Series:
                """
                Казуальная реализация ALMA (Arnaud Legoux Moving Average)
                Использует только прошлые и текущие значения, без lookahead bias.
                """
                if length > len(series):
                    return pd.Series(np.nan, index=series.index)

                # Предвычисление весов ALMA
                window = np.arange(length)
                m = offset * (length - 1)
                s = length / sigma
                weights = np.exp(-((window - m) ** 2) / (2 * s ** 2))
                weights /= weights.sum()

                # Применяем ALMA казуально (rolling + dot product)
                alma_vals = []
                for i in range(length - 1, len(series)):
                    window_data = series.iloc[i - length + 1:i + 1]
                    if window_data.isnull().any():
                        alma_vals.append(np.nan)
                    else:
                        alma_vals.append(np.dot(weights, window_data.values))

                # Паддинг NaN в начало, чтобы сохранить индекс
                alma_series = pd.Series([np.nan] * (length - 1) + alma_vals, index=series.index)

                return alma_series

            alma_series = alma_causal(series.fillna(method='ffill'), m, offset, sigma)#, index=series.index[pad:-pad])
            alma_series = alma_series.reindex(series.index, method='nearest')
            return alma_series
        else:
            return series

    def feature_extraction(self, close: pd.Series, rsi: pd.Series) -> pd.DataFrame:
        """Извлечение признаков для KNN"""
        features = pd.DataFrame(index=close.index)
        features['rsi'] = self.normalize(rsi, self.knn_lookback)

        if self.feature_count >= 2:
            features['momentum_rsi'] = self.normalize(rsi.diff(3), self.knn_lookback)
        if self.feature_count >= 3:
            features['volatility_rsi'] = self.normalize(rsi.rolling(10).std(), self.knn_lookback)
        if self.feature_count >= 4:
            features['slope_rsi'] = self.normalize(self.get_slope(rsi, 5), self.knn_lookback)
        if self.feature_count >= 5:
            features['momentum_price'] = self.normalize(close.diff(5), self.knn_lookback)

        return features.dropna()

    def normalize(self, series: pd.Series, period: int) -> pd.Series:
        """Мин-макс нормализация"""
        min_val = series.rolling(period).min()
        max_val = series.rolling(period).max()
        norm = (series - min_val) / (max_val - min_val)
        return norm.clip(0, 1)

    def get_slope(self, series: pd.Series, window: int) -> pd.Series:
        """Расчет наклона линейной регрессии"""
        idx = np.arange(window)
        def linreg(x):
            A = np.vstack([idx, np.ones(len(idx))]).T
            m, c = np.linalg.lstsq(A, x, rcond=None)[0]
            return m
        return series.rolling(window).apply(linreg, raw=True)

    def apply_knn(self, features: pd.DataFrame, rsi: pd.Series) -> pd.Series:
        """
        Оптимизированная, но идентичная оригиналу версия KNN.
        Сохраняет точную математику оригинального apply_knn_b с ускорением через BallTree.
        """
        # Сохраняем структуру как в оригинале
        full_index = rsi.index
        common_index = features.index.intersection(rsi.index)
        features = features.loc[common_index]
        rsi = rsi.loc[common_index]

        enhanced_rsi = pd.Series(index=full_index, data=np.nan)
        enhanced_rsi.loc[rsi.index] = rsi

        if len(features) < self.knn_lookback:
            return enhanced_rsi

        feature_array = features.values
        rsi_array = rsi.values

        # Основное изменение: BallTree строится на скользящем окне
        for t in range(self.knn_lookback, len(feature_array)):
            window_start = t - self.knn_lookback
            window_end = t
            X_window = feature_array[window_start:window_end]
            y_window = rsi_array[window_start:window_end]

            # Строим дерево только на текущем окне
            tree = BallTree(X_window, metric='euclidean')
            distances, indices = tree.query(feature_array[t].reshape(1, -1),
                                          k=self.knn_neighbors)

            # Точное воспроизведение оригинальной логики взвешивания
            weights = np.where(distances[0] < 1e-6, 1.0, 1.0 / distances[0])
            prediction = np.average(y_window[indices[0]], weights=weights)

            idx = common_index[t]
            enhanced_rsi.loc[idx] = (1 - self.knn_weight) * rsi.loc[idx] + self.knn_weight * prediction

        return enhanced_rsi

    def kalman_filter(self, series: pd.Series) -> pd.Series:
        """Калман-фильтр с параметрами ближе к PineScript"""
        n = len(series)
        xhat = np.full(n, np.nan)
        P = np.zeros(n)
        R = self.filter_strength * 0.1  # Очень маленький measurement noise
        Q = self.filter_strength * 0.01  # Очень маленький process noise

        first_valid_idx = series.first_valid_index()
        if first_valid_idx is None:
            return pd.Series(xhat, index=series.index)

        first_idx = series.index.get_loc(first_valid_idx)
        xhat[first_idx] = series.iloc[first_idx]
        P[first_idx] = 1.0

        for k in range(first_idx + 1, n):
            if np.isnan(series.iloc[k]):
                xhat[k] = xhat[k - 1]
                P[k] = P[k - 1] + Q
            else:
                xhatminus = xhat[k-1]
                Pminus = P[k-1] + Q
                K = Pminus / (Pminus + R)
                xhat[k] = xhatminus + K * (series.iloc[k] - xhatminus)
                P[k] = (1 - K) * Pminus

        return pd.Series(xhat, index=series.index)

    def filter_series(self, series: pd.Series) -> pd.Series:
        """Применение фильтрации к финальному RSI"""
        if self.filter_method == 'None':
            return series
        elif self.filter_method == 'Kalman':
            return self.kalman_filter(series)
        elif self.filter_method == 'DoubleEMA':
            ema1 = series.ewm(span=int(self.filter_strength * 10)).mean()
            ema2 = ema1.ewm(span=int(self.filter_strength * 5)).mean()
            return ema2
        elif self.filter_method == 'ALMA':
            return self.smooth(series)
        else:
            return series

    def week_level(self, close):
        sma_length = self.sma_length
        ema_length = self.ema_length

        # Вычисление 20-недельной SMA
        SMA_20w = close.rolling(window=sma_length, min_periods=1).mean()

        # Вычисление 21-недельной EMA
        MA_21w = close.ewm(span=ema_length, adjust=False).mean()

        return SMA_20w, MA_21w

    def fit(self, close: pd.Series) -> pd.Series:
        """Основная функция расчёта"""
        rsi = self.calculate_rsi(close, self.rsi_length)
        if self.use_smoothing:
            rsi = self.smooth(rsi)
        if self.use_knn:
            features = self.feature_extraction(close, rsi)

            rsi = self.apply_knn(features, rsi)

        if self.use_filter:
            rsi = self.filter_series(rsi)

        sma, ma = self.week_level(close)

        return rsi.clip(0, 100).iloc[-1], sma.iloc[-1], ma.iloc[-1]


####################################################################

class TinkoffInvestDataCollector:
    def __init__(self, token, path_to_save, sector, country, tg_channels, period, index):
        self.token = token
        self.path_to_save = path_to_save
        self.sector = sector
        self.country = country

        #self.adaptive_system = AdaptiveTradingSystem(regime_params)

        # Telegram channels
        self.error_tg = tg_channels['error']
        self.a1_screen_tg = tg_channels['a1_screen']
        self.memmory_id = tg_channels['memory']
        self.stat_tg = tg_channels['stat']
        self.index = index
        self.indexes = {}

        # Данные
        self.tickers = {}
        self.ticker_to_figi = {}
        self.figi_to_ticker = {}
        self.active_tickers = {}

        #Метаданные
        self.stream_last_activity = {}  # ключ: str(figi_chunk_hash), значение: datetime
        self.stream_tasks = {}

        # Списки данных по тикерам
        self.running = True

        if period == 5:
            self.candleintervall = SubscriptionInterval.SUBSCRIPTION_INTERVAL_FIVE_MINUTES
            self.timestop = 5
            self.update_interval = timedelta(minutes=5)
        if period == 1:
            self.candleintervall = SubscriptionInterval.SUBSCRIPTION_INTERVAL_ONE_MINUTE
            self.timestop = 1
            self.update_interval = timedelta(minutes=1)
        if period == 15:
            self.candleintervall = SubscriptionInterval.SUBSCRIPTION_INTERVAL_FIFTEEN_MINUTES
            self.timestop = 15
            self.update_interval = timedelta(minutes=15)
        if period == 60:
            self.candleintervall = SubscriptionInterval.SUBSCRIPTION_INTERVAL_ONE_HOUR
            self.timestop = 60
            self.update_interval = timedelta(minutes=60)

        self._smoothers       = {}    # ticker → FastRollingMode
        self._regime_history  = {}    # ticker → deque[int]
        self._adaptive_params = {}
        self._regime_hist    = {}

    def _get_candle_interval_enum(self):
        """Определяет enum-интервал для вызова REST API Tinkoff"""
        if self.timestop == 1:
            return CandleInterval.CANDLE_INTERVAL_1_MIN
        elif self.timestop == 5:
            return CandleInterval.CANDLE_INTERVAL_5_MIN
        elif self.timestop == 15:
            return CandleInterval.CANDLE_INTERVAL_15_MIN
        elif self.timestop == 60:
            return CandleInterval.CANDLE_INTERVAL_HOUR
        else:
            raise ValueError("Неподдерживаемый интервал")

    def _clean_data(self, candles):
        """Очистка и преобразование данных"""
        if not candles:
            return pd.DataFrame()

        df = pd.DataFrame(candles)
        df["time"] = pd.to_datetime(df["time"])
        return df.sort_values("time").drop_duplicates("time").reset_index(drop=True)

    async def retrospective_signal_handler(self, ticker: str, periods_passed: int):
        """
        Обработка сигналов на основании пропущенных свечей.
        :param ticker: тикер
        :param periods_passed: количество новых свечей, загруженных из backfill
        """
        # Получаем последние N свечей (включая новые)
        closes = close_price[ticker][-periods_passed:]
        end_times = time_last_kline_end[ticker][-periods_passed:]


        for i in range(periods_passed,0,-1):
            signal = self.generate_signal(ma[ticker][-i-1:][:2] , pmax[ticker][-i-1:][:2])
            if signal == 'buy':
                await self._handle_buy_signal(ticker, closes[-i], end_times[-i])
                print(f'[BUY] {ticker}, за время пропуска был сигнал на покупку в {end_times[-i]} по цене {closes[-i]}')
            elif signal == 'sell':
                await self._handle_sell_signal(ticker, closes[-i], end_times[-i])
                print(f'[SELL] {ticker}, за время пропуска был сигнал на продажу в {end_times[-i]} по цене {closes[-i]}')
            else:
                pass

    async def _handle_buy_signal(self, ticker, price, time_str):
        """
        Обработка сигнала на покупку для тикера.
        """
        if ticker in open_trades and open_trades[ticker].get('status') == 'open':
            return  # Уже открыта сделка — пропускаем


        way = phase_ticker_params[ticker]['way']
        ticker_parametrs = prepare_regime_params(phase_ticker_params[ticker]['params'])['calc_params']
        stop_loss_y_n = phase_ticker_params[ticker]['change_tp']
        stop_loss = phase_ticker_params[ticker]['stop_loss_pst_tp']

        #ticker_parametrs = ticker_params[ticker]['params']
        #way = ticker_params[ticker]['way']
        #stop_loss_y_n = ticker_params[ticker]['change']
        #stop_loss = ticker_params[ticker]['stop_loss_pst']

        should_open = False

        if way == 'Базовый':
            should_open = True

        elif way == 'RSI':
            rsi_length = ticker_parametrs['rsi_length']
            use_smoothing = ticker_parametrs['use_smoothing']
            smoothing_length = ticker_parametrs['smoothing_length']
            smoothing_type = ticker_parametrs['smoothing_type']
            alma_sigma = ticker_parametrs['alma_sigma']
            rsi_overbought = ticker_parametrs['rsi_overbought']
            rsi_oversold = ticker_parametrs['rsi_oversold']
            use_knn = ticker_parametrs['use_knn']
            knn_neighbors = ticker_parametrs['knn_neighbors']
            knn_lookback = ticker_parametrs['knn_lookback']
            knn_weight = ticker_parametrs['knn_weight']
            feature_count = ticker_parametrs['feature_count']
            use_filter = ticker_parametrs['use_filter']
            filter_method = ticker_parametrs['filter_method']
            filter_strength = ticker_parametrs['filter_strength']
            sma_length = ticker_parametrs['sma_length']
            ema_length = ticker_parametrs['ema_length']
            rsi_theasold = ticker_parametrs['rsi_helbuth']

            ml_rsi = MachineLearningRSI(
                rsi_length=int(rsi_length),
                use_smoothing=use_smoothing,
                smoothing_length=int(smoothing_length),
                smoothing_type=smoothing_type,
                alma_sigma=int(alma_sigma),
                rsi_overbought=int(rsi_overbought),
                rsi_oversold=int(rsi_oversold),
                use_knn=use_knn,
                knn_neighbors=int(knn_neighbors),
                knn_lookback=int(knn_lookback),
                knn_weight=knn_weight,
                feature_count=int(feature_count),
                use_filter=use_filter,
                filter_method=filter_method,
                filter_strength=filter_strength,
                sma_length=int(sma_length),
                ema_length=int(ema_length)
            )

            rsi, _, _ = ml_rsi.fit(pd.Series(close_price[ticker]))
            if rsi >= rsi_theasold:
                should_open = True

        elif way == 'RSI_SMA_EMA':
            rsi_length = ticker_parametrs['rsi_length']
            use_smoothing = ticker_parametrs['use_smoothing']
            smoothing_length = ticker_parametrs['smoothing_length']
            smoothing_type = ticker_parametrs['smoothing_type']
            alma_sigma = ticker_parametrs['alma_sigma']
            rsi_overbought = ticker_parametrs['rsi_overbought']
            rsi_oversold = ticker_parametrs['rsi_oversold']
            use_knn = ticker_parametrs['use_knn']
            knn_neighbors = ticker_parametrs['knn_neighbors']
            knn_lookback = ticker_parametrs['knn_lookback']
            knn_weight = ticker_parametrs['knn_weight']
            feature_count = ticker_parametrs['feature_count']
            use_filter = ticker_parametrs['use_filter']
            filter_method = ticker_parametrs['filter_method']
            filter_strength = ticker_parametrs['filter_strength']
            sma_length = ticker_parametrs['sma_length']
            ema_length = ticker_parametrs['ema_length']
            rsi_theasold = ticker_parametrs['rsi_helbuth']

            ml_rsi = MachineLearningRSI(
                rsi_length=int(rsi_length),
                use_smoothing=use_smoothing,
                smoothing_length=int(smoothing_length),
                smoothing_type=smoothing_type,
                alma_sigma=int(alma_sigma),
                rsi_overbought=int(rsi_overbought),
                rsi_oversold=int(rsi_oversold),
                use_knn=use_knn,
                knn_neighbors=int(knn_neighbors),
                knn_lookback=int(knn_lookback),
                knn_weight=knn_weight,
                feature_count=int(feature_count),
                use_filter=use_filter,
                filter_method=filter_method,
                filter_strength=filter_strength,
                sma_length=int(sma_length),
                ema_length=int(ema_length)
            )
            rsi, sma, ma_sma = ml_rsi.fit(pd.Series(close_price[ticker]))
            if rsi > rsi_theasold and sma > price and ma_sma > price:
                should_open = True

        if should_open:
            trade_obj = {
                'buy_price': price,
                'buy_time': pd.to_datetime(time_str).isoformat(),
                'max_price': price,
                'status': 'open',
                'reason': 'BASE'
            }
            if stop_loss_y_n != 'No':
                trade_obj['stop_loss'] = stop_loss
            open_trades[ticker] = trade_obj

            bot.send_message(self.stat_tg, f"[TRADE OPENED] {ticker} по {price} в {time_str}")
        else:
            bot.send_message(self.stat_tg, f"[TRADE NOT OPENED] {ticker} по {price} в {time_str} — не прошёл фильтр RSI")

    async def _handle_sell_signal(self, ticker, price, time_str):
        """
        Обработка сигнала на продажу.
        """
        if (ticker in open_trades
            and open_trades[ticker].get('status') == 'open'):
            reason = open_trades[ticker].get('reason', 'UNKNOWN')
            await self.close_trade(ticker, price, open_trades[ticker]['max_price'], reason)
            bot.send_message(self.stat_tg, f"[TRADE CLOSED] {ticker} по {price} в {time_str}")

    async def backfill_missing_candles(self):
        """
        Ускоренный метод дозагрузки пропущенных свечей с учетом временных зон.
        """
        #print("[BACKFILL] Запуск дозагрузки пропущенных свечей...")

        # Создаем пустые структуры для тикеров без открытых сделок
        for ticker in self.active_tickers:
            if ticker not in open_trades or type(open_trades[ticker]) is list:
                open_trades[ticker] = {}

        now_utc = dt.utcnow().replace(second=0, microsecond=0, tzinfo=pytz.UTC)
        #print(f"[TIME SYNC] Текущее UTC время сервера: {now_utc}")

        async def process_ticker(ticker):
            try:
                figi = self.ticker_to_figi[ticker]
                now_utc = dt.utcnow().replace(second=0, microsecond=0, tzinfo=pytz.UTC)

                # Проверяем, есть ли данные о последней свече
                if not time_last_kline_end.get(ticker):
                    #print(f"[SKIP] {ticker}: нет исторических данных")
                    return

                # Последнее сохранённое время
                last_time_local = pd.to_datetime(time_last_kline_end[ticker][-1])
                last_time_utc = last_time_local - timedelta(hours=3)

                #print(f"[SYNC] {ticker}: последняя свеча в UTC: {last_time_utc}")

                # Список сохранённых временных меток
                saved_candle_times_set = set(
                    (pd.to_datetime(t) - timedelta(hours=3)).strftime('%Y-%m-%dT%H:%M:%S')
                    for t in time_last_kline_start[ticker]
                )

                # Начинаем с последней известной свечи
                from_time = last_time_utc
                max_to_time = now_utc - self.update_interval  # Только закрытые свечи

                while from_time < max_to_time:
                    to_time = min(max_to_time, from_time + timedelta(minutes=1000))
                    #print(f"[API REQ] {ticker}: from {from_time} to {to_time}")

                    try:
                        response = await client.market_data.get_candles(
                            figi=figi,
                            from_=from_time,
                            to=to_time,
                            interval=self._get_candle_interval_enum()
                        )
                    except Exception as api_err:
                        #print(f"[API ERROR] {ticker}: ошибка API: {api_err}")
                        return

                    if not response.candles:
                        #print(f"[WARN] {ticker}: API не вернул свечи")
                        break

                    new_candles = [
                        candle for candle in response.candles
                        if candle.time.strftime('%Y-%m-%dT%H:%M:%S') not in saved_candle_times_set
                        and candle.time.replace(tzinfo=pytz.UTC) < max_to_time  # фильтрация незакрытых
                    ]

                    if not new_candles:
                        #print(f"[SKIP] {ticker}: все свечи уже сохранены или незакрыты")
                        break

                    #print(f"[SAVE] {ticker}: сохраняем {len(new_candles)} новых свечей")

                    # Обновляем данные
                    for candle in new_candles:
                        ts_utc = candle.time.replace(tzinfo=pytz.UTC)
                        ts_local = ts_utc + timedelta(hours=3)
                        tc_local = ts_local + self.update_interval

                        open_price[ticker].append(float(candle.open.units) + candle.open.nano * 1e-9)
                        close_price[ticker].append(float(candle.close.units) + candle.close.nano * 1e-9)
                        high_price[ticker].append(float(candle.high.units) + candle.high.nano * 1e-9)
                        low_price[ticker].append(float(candle.low.units) + candle.low.nano * 1e-9)
                        volume[ticker].append(candle.volume)

                        time_last_kline_start[ticker].append(ts_local.isoformat())
                        time_last_kline_end[ticker].append(tc_local.isoformat())

                    # FIFO — обрезаем до MAX_CANDLES
                    for arr in [
                        open_price, close_price, high_price, low_price,
                        volume, time_last_kline_start, time_last_kline_end
                    ]:
                        arr[ticker] = arr[ticker][-MAX_CANDLES:]

                    await self.calculate_indicators_and_signals(ticker, len(new_candles))

                    await self.retrospective_signal_handler(ticker, len(new_candles))

                    # Продвигаем окно
                    from_time = to_time

            except Exception as e:
                print(f"[ERROR] {ticker}: ошибка обработки: {e}")

        # Параллелизуем обработку тикеров
        async with AsyncClient(self.token) as client:
            await asyncio.gather(*(process_ticker(ticker) for ticker in self.active_tickers))

    async def calculate_indicators_and_signals(self, ticker, periods_passed=None):

        try:
            # 1. Сырые данные
            high_arr = np.array(high_price.get(ticker))
            low_arr = np.array(low_price.get(ticker))
            close_arr = np.array(close_price.get(ticker))
            vol_arr = np.array(volume.get(ticker))
            n = len(close_arr)
            if n < 2:
                return

            # 2. Параметры и инициализация
            params       = phase_ticker_params[ticker]['params']
            # размер окна для признаков
            window_feat  = int(params['moving_average_length'] * 9.5)
            # формируем DataFrame-мини-окно для extract_features
            import pandas as pd
            hist_df = pd.DataFrame({
                'high':   high_arr,
                'low':    low_arr,
                'close':  close_arr,
                'volume': vol_arr
            })
            features = extract_features(hist_df, window=window_feat)  # ваша функция
            # scale + predict
            scaled = scaler_global.transform(features)
            labels = kmeans_global.predict(scaled)

            # сглаживаем
            window_smooth = int(params['atr_period'] * 5.5)
            smoother = FastRollingMode(window_size=window_smooth)
            regimes = np.array([smoother.update(l) for l in labels], dtype=int)

            # 2) Готовим базовые параметры по режиму
            regime_params = prepare_regime_params(params)['base_params']

            # 3) PRE-COMPUTE MA & ATR для каждого режима
            ma_cache  = {}
            atr_cache = {}
            for regime, p in regime_params.items():
                atype = p['average_type']
                L     = p['moving_average_length']
                P     = p['atr_period']

                if atype == 'SMA':
                    ma_data = self.generateSma(high_arr, low_arr, window=L)
                elif atype == 'VAR':
                    ma_data = self.generateVar(high_arr, low_arr, moving_average_length=L)
                elif atype == 'EMA':
                    ma_data = self.generateEma(high_arr, low_arr, moving_average_length=L)
                elif atype == 'AMA':
                    ama_p = p.get('ama_params', {'atr_period':14,'min_period':5,'max_period':50})
                    ma_data = self.generateAma(
                        high_arr, low_arr, close_arr,
                        atr_period=ama_p['atr_period'],
                        min_period=ama_p['min_period'],
                        max_period=ama_p['max_period']
                    )
                else:
                    raise ValueError(f"Unknown MA type {atype!r}")

                ma_cache[regime]  = ma_data
                atr_cache[regime] = self.generateAtr(high_arr, low_arr, close_arr, length=P)

            # 4) MERGE по маске режимов
            var_all = np.empty(n, dtype=float)
            atr_all = np.empty(n, dtype=float)
            mul_all = np.empty(n, dtype=float)

            for regime, p in regime_params.items():
                mask = (regimes == regime)
                var_all[mask] = ma_cache[regime][mask]
                atr_all[mask] = atr_cache[regime][mask]
                mul_all[mask] = p['atr_multiplier']

            # Заполняем NAN в начале var_all
            if np.isnan(var_all[0]):
                first = var_all[~np.isnan(var_all)][0]
                var_all[np.isnan(var_all)] = first

            # 5) PMax state machine
            atr_all = np.nan_to_num(atr_all, nan=0.0)
            first_var = var_all[~np.isnan(var_all)][0]
            var_all = np.nan_to_num(var_all, nan=first_var)
            pmax_all = np.empty(n, dtype=float)
            prev_v   = var_all[0]
            prev_a   = atr_all[0]
            prev_m   = mul_all[0]
            prev_fu  = prev_v + prev_m * prev_a
            prev_fl  = prev_v - prev_m * prev_a
            prev_p   = prev_fl
            pmax_all[0] = prev_p

            for i in range(1, n):
                v = var_all[i]; a = atr_all[i]; m = mul_all[i]
                bu = v + m * a
                bl = v - m * a

                fu = bu if (bu < prev_fu or prev_v > prev_fu) else prev_fu
                fl = bl if (bl > prev_fl or prev_v < prev_fl) else prev_fl

                if prev_p == prev_fu:
                    p = fu if v <= fu else fl
                else:
                    p = fl if v >= fl else fu

                pmax_all[i] = p
                prev_v, prev_fu, prev_fl, prev_p = v, fu, fl, p

            # 6) Обновляем ваши словари ma[] и pmax[]
            if periods_passed is None:
                ma[ticker]   = ma[ticker][1:] + [var_all[-1]]
                pmax[ticker] = pmax[ticker][1:] + [pmax_all[-1]]
            else:
                ma[ticker]   = ma[ticker][periods_passed:] + list(var_all[-periods_passed:])
                pmax[ticker] = pmax[ticker][periods_passed:] + list(pmax_all[-periods_passed:])

            signal = self.generate_signal(ma[ticker][-2:], pmax[ticker][-2:])

            # Сохраняем сигнал
            signals[ticker] = signal

            # 9. Финальный сигнал
            if signal == 'buy':
                send_buy_signal_to_telegram(ticker, close_arr[-1])
            elif signal == 'sell':
                send_sell_signal_to_telegram(ticker, close_arr[-1])
        except Exception as e:
            bot.send_message(
                      self.error_tg,
                      f"Ошибка в calculate_indicators_and_signals {ticker} с ошибкой {e}"
                  )


    async def calculate_indicators_and_signals1(self, ticker, periods_passed = None):
        """
        Вычисление индикаторов и сигналов для каждого тикера.

        """
        # Получаем последние данные для тикера

        #Paramsticker_params
        params = ticker_params[ticker]['params']
        MOVING_AVERAGE_LENGHT = params['moving_average_length']
        ATR_PERIOD = params['atr_period']
        ATR_MULTIPLIER = params['atr_multiplier']
        average_type = params['average_type']
        if average_type == 'AMA':
            ama_params = {
              'atr_period': int(params['ama_atr_period']),
              'min_period': int(params['ama_min_period']),
              'max_period': int(params['ama_max_period'])
            }


        high_data = np.array(high_price.get(ticker)[-3000:])
        low_data = np.array(low_price.get(ticker)[-3000:])
        close_data = np.array(close_price.get(ticker)[-3000:])
        # volume_data = volume.get(ticker)[-1000:]

        #if high_data.size==0 or low_data.size==0 or close_data.size==0:
        #    pass

        if len(high_data) == 0 or len(low_data) == 0 or len(close_data) == 0:
            return

        if average_type == 'SMA':
            ma_data = self.generateSma(high_data, low_data, window=MOVING_AVERAGE_LENGHT)
        elif average_type == 'VAR':
            ma_data = self.generateVar(high_data, low_data, moving_average_length=MOVING_AVERAGE_LENGHT)
        elif average_type == 'AMA':
            if ama_params is None:
                raise ValueError("Для AMA необходимо указать параметры ama_params.")
            ma_data = self.generateAma(high_data, low_data, close_data, **ama_params)
        else:
            raise ValueError("Неподдерживаемый тип скользящего среднего.")

        # ema_data = self.generateEma(high_data, low_data)
        # atr_data = self.generateAtr(high_data, low_data, close_data)
        pmax_data = self.generatePMax(ma_data, close_data, high_data, low_data, atr_period=ATR_PERIOD,
                                      atr_multiplier=ATR_MULTIPLIER)

        # Обновляем словари с индикаторами

        if periods_passed == None:
            ma[ticker] = ma[ticker][1:] + [ma_data[-1]]
            # ema[ticker] = ema[ticker][1:] + [ema_data[-1]]
            # atr[ticker] = atr[ticker][1:] + [atr_data[-1]]
            pmax[ticker] = pmax[ticker][1:] + [pmax_data[-1]]
        else:
            #print(len(ma[ticker][periods_passed:]), len(list(ma_data)[-periods_passed:]))
            ma[ticker] = ma[ticker][periods_passed:] + list(ma_data)[-periods_passed:]
            pmax[ticker] = pmax[ticker][periods_passed:] + list(pmax_data)[-periods_passed:]

        # Вычисляем сигналы
        signal = self.generate_signal(ma[ticker][-2:], pmax[ticker][-2:])

        # Сохраняем сигнал
        signals[ticker] = signal


        # Отправляем сигнал в Telegram если необходимо
        if signal == 'buy':
            send_buy_signal_to_telegram(ticker, close_data[-1])
        elif signal == 'sell':
            send_sell_signal_to_telegram(ticker, close_data[-1])

    def generateAma(self, high_array, low_array, close_array, atr_period=14, min_period=5, max_period=50):
        """
        Генерация адаптивного скользящего среднего на основе волатильности.

        :param high_array: Массив значений high.
        :param low_array: Массив значений low.
        :param close_array: Массив значений close.
        :param atr_period: Период для расчета ATR.
        :param min_period: Минимальный период скользящего среднего.
        :param max_period: Максимальный период скользящего среднего.
        :return: Массив значений адаптивного скользящего среднего.
        """
        # Рассчитываем ATR
        atr = self._calculate_atr(high_array, low_array, close_array, atr_period)

        # Нормализуем ATR для использования в качестве коэффициента
        normalized_atr = (atr - np.min(atr)) / (np.max(atr) - np.min(atr) + 1e-10)

        # Рассчитываем динамический период
        dynamic_period = min_period + (max_period - min_period) * normalized_atr

        # Рассчитываем адаптивное скользящее среднее (гибрид SMA и EMA)
        adaptive_ma = np.zeros_like(close_array)
        for i in range(len(close_array)):
            if i < int(dynamic_period[i]):
                adaptive_ma[i] = np.mean(close_array[:i+1])  # SMA для начальных значений
            else:
                period = int(dynamic_period[i])
                alpha = 2 / (period + 1)
                adaptive_ma[i] = alpha * close_array[i] + (1 - alpha) * adaptive_ma[i-1]  # EMA

        return adaptive_ma

    def _calculate_atr(self, high_array, low_array, close_array, period=14):
        """
        Рассчитывает Average True Range (ATR).

        :param high_array: Массив значений high.
        :param low_array: Массив значений low.
        :param close_array: Массив значений close.
        :param period: Период для расчета ATR.
        :return: Массив значений ATR.
        """
        tr = np.zeros_like(high_array)
        tr[0] = high_array[0] - low_array[0]

        for i in range(1, len(high_array)):
            hl = high_array[i] - low_array[i]
            hc = abs(high_array[i] - close_array[i-1])
            lc = abs(low_array[i] - close_array[i-1])
            tr[i] = max(hl, hc, lc)

        atr = np.zeros_like(tr)
        atr[period-1] = np.mean(tr[:period])

        for i in range(period, len(tr)):
            atr[i] = (atr[i-1] * (period-1) + tr[i]) / period

        return atr

    def generateVar(self, high_array, low_array, moving_average_length=14):
        """
        Генерация VAR (Volatility Adjusted Ratio).

        :param high_array: Массив значений high.
        :param low_array: Массив значений low.
        :param moving_average_length: Период для расчета VAR.
        :return: Массив значений VAR.
        """
        # Константа alpha
        valpha = 2 / (moving_average_length + 1)

        # Вычисляем среднее значение между high и low
        hl2 = (high_array + low_array) / 2

        # Вычисляем разницы для vud1 и vdd1
        diff = np.diff(hl2, prepend=hl2[0])
        vud1 = np.where(diff > 0, diff, 0)
        vdd1 = np.where(diff < 0, -diff, 0)

        # Функция для расчета скользящих сумм
        def calculate_window_sums(arr, window_size=9):
            cumsum = np.cumsum(arr)
            return cumsum - np.concatenate((np.zeros(window_size), cumsum[:-window_size]))

        # Вычисляем vUD и vDD
        vUD = calculate_window_sums(vud1, 9)
        vDD = calculate_window_sums(vdd1, 9)

        # Вычисляем vCMO
        epsilon = 1e-10
        with np.errstate(divide='ignore', invalid='ignore'):
            vCMO = np.divide(vUD - vDD, vUD + vDD + epsilon)
        vCMO = np.nan_to_num(vCMO, nan=0.0)

        # Вычисляем VAR
        cmo_abs = np.abs(vCMO)
        var = np.zeros_like(hl2)
        var_before = 0.0
        for i in range(len(hl2)):
            if i < len(cmo_abs):
                var[i] = (valpha * cmo_abs[i] * hl2[i]) + (1 - valpha * cmo_abs[i]) * var_before
            else:
                var[i] = var_before
            var_before = var[i]
        del valpha, hl2, vud1, vdd1, var_before, vUD, vDD, vCMO
        return var

    def generateEma(self, high_array, low_array, moving_average_length=14):
        """Вычисление EMA."""
        if high_array.size==0 or low_array.size==0:
            return []

        hl2 = [(high + low) / 2 for high, low in zip(high_array, low_array)]
        ema = np.full_like(hl2, np.nan)
        alpha = 2 / (moving_average_length + 1)

        if moving_average_length <= 1:
            return hl2

        start_idx = moving_average_length - 1
        sma = np.mean(hl2[:moving_average_length])
        ema[start_idx] = sma

        for i in range(start_idx + 1, len(hl2)):
            ema[i] = alpha * hl2[i] + (1 - alpha) * ema[i - 1]

        del hl2, alpha, start_idx, sma
        return ema

    def generateAtr(self, high_array, low_array, close_array, length=14):
        """Вычисление ATR."""
        if high_array.size==0 or low_array.size==0 or close_array.size==0:
            return []

        n = len(high_array)
        if n == 0 or length > n:
            return []

        tr = np.zeros(n)
        atr = np.full(n, np.nan)

        prev_close = np.roll(close_array, 1)
        prev_close[0] = np.nan

        tr[0] = high_array[0] - low_array[0]

        for i in range(1, n):
            hl = high_array[i] - low_array[i]
            hc = abs(high_array[i] - prev_close[i])
            lc = abs(low_array[i] - prev_close[i])
            tr[i] = max(hl, hc, lc)

        if n >= length:
            atr[length - 1] = np.mean(tr[:length])
            alpha = 1.0 / length
            for i in range(length, n):
                atr[i] = alpha * tr[i] + (1 - alpha) * atr[i - 1]

        del n, tr, prev_close, hl, hc, lc, alpha
        return atr

    def generateSma(self, high_array, low_array, window=14):
        """
        Генерация Simple Moving Average (SMA).

        :param high_array: Массив значений high.
        :param low_array: Массив значений low.
        :param window: Период SMA.
        :return: Массив значений SMA.
        """
        hl2 = (high_array + low_array) * 0.5

        if window <= 1:
            return hl2

        # Создаем массив для результатов с NaN
        sma = np.full_like(hl2, np.nan)

        # Рассчитываем кумулятивную сумму
        cumsum = np.cumsum(hl2)

        # Создаем сдвинутый кумулятивный массив
        shifted_cumsum = np.zeros_like(cumsum)
        shifted_cumsum[window:] = cumsum[:-window]

        # Вычисляем SMA для валидных периодов
        valid = slice(window - 1, None)
        sma[valid] = (cumsum[valid] - shifted_cumsum[valid]) / window

        del valid, shifted_cumsum, cumsum, hl2
        return sma

    def generatePMax(self, var_array, close_array, high_array, low_array, atr_period=14, atr_multiplier=3):
        """Вычисление PMAX."""
        if high_array.size==0 or low_array.size==0 or close_array.size==0 or len(var_array) == 0:
            return []

        atr = self.generateAtr(high_array, low_array, close_array, length=atr_period)
        pmax = []
        previous_final_upperband = 0
        previous_final_lowerband = 0
        previous_var = 0
        previous_pmax = 0

        for i in range(len(close_array)):
            atrc = atr[i] if i < len(atr) and not np.isnan(atr[i]) else 0
            varc = var_array[i] if i < len(var_array) else 0

            basic_upperband = varc + atr_multiplier * atrc
            basic_lowerband = varc - atr_multiplier * atrc

            final_upperband = basic_upperband if (
                    basic_upperband < previous_final_upperband or previous_var > previous_final_upperband) else previous_final_upperband
            final_lowerband = basic_lowerband if (
                    basic_lowerband > previous_final_lowerband or previous_var < previous_final_lowerband) else previous_final_lowerband

            if previous_pmax == previous_final_upperband:
                pmaxc = final_upperband if varc <= final_upperband else final_lowerband
            else:
                pmaxc = final_lowerband if varc >= final_lowerband else final_upperband

            pmax.append(pmaxc)
            previous_var = varc
            previous_final_upperband = final_upperband
            previous_final_lowerband = final_lowerband
            previous_pmax = pmaxc

        del atr, previous_final_upperband, previous_final_lowerband, previous_var, previous_pmax, pmaxc
        return pmax

    def generate_signal(self, var, pmax):
        """Генерация сигнала на основе VAR и PMAX."""

        if len(var)== 0 or not pmax:
            return None

        last_var = var[-1]

        previous_var = var[-2] if len(var) >= 2 else last_var
        last_pmax = pmax[-1]
        previous_pmax = pmax[-2] if len(pmax) >= 2 else last_pmax

        if last_var > last_pmax and previous_var < previous_pmax:
            del last_var, previous_var, last_pmax, previous_pmax
            return 'buy'
        elif last_var < last_pmax and previous_var > previous_pmax:
            del last_var, previous_var, last_pmax, previous_pmax
            return 'sell'
        else:
            del last_var, previous_var, last_pmax, previous_pmax
            return 'hold'

    async def process_buy_signal(self, ticker):
        """Обрабатывает BUY сигнал."""

        try:
            # Проверяем, если тикер уже в активных сделках
            if ticker in open_trades:
                return

            # Формируем данные для классификатора
            ticker_data = {
                'open': open_price[ticker],
                'close': close_price[ticker],
                'high': high_price[ticker],
                'low': low_price[ticker],
                'volume': volume[ticker],
            }
            df = pd.DataFrame(ticker_data)
            feature_calculator = ClassifierFeatureCalculator(df)

            # Вычисляем фичи и предиктим через классификатор
            features = feature_calculator.calculate_features(**params_for_classifier)
            prediction = classifier_model['pipeline'].predict(features)

            # Если модель классификации возвращает 1, открываем сделку
            if prediction[0] == 1:
                print(f'Проверка тикета {ticker} на предмет прохода/непрохода, результат 1 с оценкой {classifier_model["pipeline"].predict_proba(features)}')
                return 1
            else:
                return 0
                print(f'Проверка тикета {ticker} на предмет прохода/непрохода, результат 0 с оценкой {classifier_model["pipeline"].predict_proba(features)}')
        except Exception as e:
            bot.send_message(
                      self.error_tg,
                      f"Ошибка в process_buy_signal {ticker} с ошибкой {e}"
                  )

    '''async def close_trade(self, ticker, current_price, max_price, reason):
        """
        Закрытие сделки.
        """
        trade_data = open_trades.get(ticker, None)

        if trade_data:
            buy_price = trade_data['buy_price']
            buy_time_str = trade_data['buy_time']

            # Преобразуем строку в datetime, если это необходимо
            if isinstance(buy_time_str, str):
                buy_time = dt.strptime(buy_time_str, "%Y-%m-%dT%H:%M:%S.%f")  # Формат соответствует вашему примеру
            else:
                buy_time = buy_time_str  # Уже datetime

            pnl = ((current_price - buy_price) / buy_price) * 100
            trade_duration = (dt.utcnow() - buy_time).total_seconds() // 60

            open_trades[ticker]['status'] = 'closed'

            bot.send_message(
                self.stat_tg,
                f"[TRADE CLOSED] Сделка для {ticker} закрыта по цене {current_price}, на отрезке максимальная цена была {max_price} "
                f"с прибылью {pnl:.2f}% (продолжительность: {trade_duration:.2f} минут). "
                f"Причина: {reason}."
            )
            self.process_trade(ticker,
                               buy_time,
                               buy_price,
                               dt.utcnow(),
                               current_price,
                               pnl,
                               reason)
        else:bot.send_message(self.stats_tg, f"По тикету {ticker} нет данных об открытых позициях")'''

    async def close_trade(self, ticker, current_price, max_price, reason):
        """
        Закрытие сделки с универсальной обработкой временных меток.
        Обрабатывает форматы:
        - "2025-05-30T16:45:00.805244" (без временной зоны)
        - "2025-05-28T20:30:00+00:00" (с временной зоной)
        """
        trade_data = open_trades.get(ticker, None)

        if trade_data:
            buy_price = trade_data['buy_price']
            buy_time_str = trade_data['buy_time']

            # Универсальное преобразование времени
            if isinstance(buy_time_str, str):
                try:
                    # Удаляем временную зону если она есть
                    if '+' in buy_time_str:
                        buy_time_str = buy_time_str.split('+')[0]
                    # Удаляем микросекунды если они есть
                    if '.' in buy_time_str:
                        buy_time_str = buy_time_str.split('.')[0]

                    # Парсим как naive datetime
                    buy_time = dt.strptime(buy_time_str, "%Y-%m-%dT%H:%M:%S")

                    # Добавляем 3 часа если нужно
                    buy_time += timedelta(hours=3)

                except Exception as e:
                    bot.send_message(self.stat_tg, f"Ошибка преобразования даты для {ticker}: {str(e)}")
                    return
            else:
                buy_time = buy_time_str  # Уже datetime
                if buy_time.tzinfo is not None:  # Если есть временная зона
                    buy_time = buy_time.replace(tzinfo=None)  # Удаляем временную зону
                buy_time += timedelta(hours=3)

            # Вычисляем PnL
            pnl = ((current_price - buy_price) / buy_price) * 100

            # Получаем текущее время как naive datetime
            current_time = dt.utcnow()
            trade_duration = (current_time - buy_time).total_seconds() // 60

            open_trades[ticker]['status'] = 'closed'

            bot.send_message(
                    self.stat_tg,
                    f"[TRADE CLOSED] Сделка для {ticker} закрыта по цене {current_price}, на отрезке максимальная цена была {max_price} "
                    f"с прибылью {pnl:.2f}% (продолжительность: {trade_duration:.2f} минут). "
                    f"Причина: {reason}."
                )

            self.process_trade(ticker,
                        buy_time,
                        buy_price,
                        current_time,
                        current_price,
                        pnl,
                        reason)
        else:
            bot.send_message(self.stat_tg, f"По тикету {ticker} нет данных об открытых позициях")

    def process_trade(self, ticker, alert_time, alert_price, exit_time, exit_price, pnl, reason):
        # Определяем интервал
        trade_duration = (exit_time - alert_time) // 1000
        interval = '15m'  # if trade_duration < 5 * 3600 else '5m'

        # Получаем данные
        klines = find_klines(ticker)
        #klines = fetch_tinkoff_candles(ticker, alert_time, exit_time, interval)
        df = prepare_tinkoff_data(klines)

        # Сохраняем график
        chart_file_path = f"/content/drive/MyDrive/t_ml/data/png/{ticker}_{dt.now().strftime('%Y%m%d_%H%M%S')}.png"
        save_tinkoff_trade_chart(df, chart_file_path, ticker, alert_price, alert_time, exit_time, exit_price,
                                 pnl, reason)

        # Отправляем график
        caption = f"{ticker}\n\nSG Model; PNL: {pnl:.2f}%"
        sent_message = bot.send_photo('-1002619839070', open(chart_file_path, 'rb'), caption=caption)
        message_link = f"/https://t.me/c/2619839070/{sent_message.message_id}"

        # Сохраняем и отправляем отчет
        save_and_send_tinkoff_report(ticker, alert_price, alert_time, exit_time, exit_price, pnl, reason, message_link, tg_channels)

    def _load_saved_data(self):
        try:
            with open(f'{self.path_to_save}open_price.txt', 'r') as f:
                self.open_price = json.loads(f.read())

            with open(f'{self.path_to_save}close_price.txt', 'r') as f:
                self.close_price = json.loads(f.read())

            with open(f'{self.path_to_save}high_price.txt', 'r') as f:
                self.high_price = json.loads(f.read())

            with open(f'{self.path_to_save}low_price.txt', 'r') as f:
                self.low_price = json.loads(f.read())

            with open(f'{self.path_to_save}volume.txt', 'r') as f:
                self.volume = json.loads(f.read())

            with open(f'{self.path_to_save}time_last_kline_start.txt', 'r') as f:
                self.time_last_kline_start = json.loads(f.read())

            with open(f'{self.path_to_save}time_last_kline_end.txt', 'r') as f:
                self.time_last_kline_end = json.loads(f.read())

            #print("Данные успешно загружены.")
        except Exception as e:
            print(f"Ошибка при загрузке данных: {e}")
            self.open_price, self.close_price = {}, {}
            self.high_price, self.low_price = {}, {}
            self.volume = {}
            self.time_last_kline_start, self.time_last_kline_end = {}, {}

    async def save_data(self):
        """Save data to files periodically."""
        while True:
            await asyncio.sleep(30 * self.timestop)
            try:
                with open(f'{self.path_to_save}open_price.txt', 'w') as f:
                    f.write(json.dumps(open_price))

                with open(f'{self.path_to_save}close_price.txt', 'w') as f:
                    f.write(json.dumps(close_price))

                with open(f'{self.path_to_save}high_price.txt', 'w') as f:
                    f.write(json.dumps(high_price))

                with open(f'{self.path_to_save}low_price.txt', 'w') as f:
                    f.write(json.dumps(low_price))

                with open(f'{self.path_to_save}volume.txt', 'w') as f:
                    f.write(json.dumps(volume))

                #print("Данные успешно сохранены.")
            except Exception as e:
                bot.send_message(self.error_tg, f'Error saving 1\n{e}')
                await asyncio.sleep(0.5)

            try:
                with open(f'{self.path_to_save}ma.txt', 'w') as f:
                    f.write(json.dumps(ma))

                with open(f'{self.path_to_save}pmax.txt', 'w') as f:
                    f.write(json.dumps(pmax))

                with open(f'{self.path_to_save}signals.txt', 'w') as f:
                    f.write(json.dumps(signals))

                with open(f'{self.path_to_save}open_trades.txt', 'w') as f:
                    f.write(json.dumps(open_trades))

                with open(f'{self.path_to_save}trading_data.txt', 'w') as f:
                    f.write(json.dumps(trading_data))

                with open(f'{self.path_to_save}time_last_kline_start.txt', 'w') as f:
                    json.dump(time_last_kline_start, f)

                with open(f'{self.path_to_save}time_last_kline_end.txt', 'w') as f:
                    json.dump(time_last_kline_end, f)

            except Exception as e:
                bot.send_message(self.error_tg, f'Error saving 2\n{e}')
                await asyncio.sleep(0.5)

    async def create_tickers(self):
        """Create a list of tickers based on specified sector and country."""
        async with AsyncClient(self.token) as client:
            instruments = await client.instruments.shares()
            for share in instruments.instruments:
                if (share.country_of_risk == self.country):  # and share.sector == self.sector):
                    self.tickers[share.ticker] = [share.figi, share.name, share.sector]

        async with AsyncClient(self.token) as client:
            instruments = await client.instruments.find_instrument(query=self.index)
            for instrument in instruments.instruments:
                self.indexes[instrument.ticker] = [instrument.figi, instrument.name, instrument.instrument_type]

    async def update_trading_statuses(self):
        """Обновление статусов инструментов и поддержка актуального списка"""
        try:
            async with AsyncClient(self.token) as client:
                instruments = await client.instruments.shares()
                new_active = {}

                # Собираем свежие данные
                for share in instruments.instruments:
                    if share.ticker in self.tickers and share.trading_status == 5:
                        new_active[share.ticker] = {
                            'figi': share.figi,
                            'name': share.name,
                            'status': share.trading_status
                        }

                # Удаляем тикеры с изменившимся статусом
                removed_tickers = set(self.active_tickers.keys()) - set(new_active.keys())
                for ticker in removed_tickers:
                    del self.active_tickers[ticker]
                    print(f'Удален тикер {ticker}')

                # Добавляем новые тикеры
                added_tickers = set(new_active.keys()) - set(self.active_tickers.keys())
                for ticker in added_tickers:
                    self.active_tickers[ticker] = new_active[ticker]
                    print(f'Добавлен тикер {ticker}')

            await asyncio.sleep(self.update_interval.total_seconds())

        except Exception as e:
            print(f"Ошибка при обновлении статусов: {e}")
            await asyncio.sleep(60)

     async def compute_model_prediction(self, ticker):
        if ticker not in open_trades or open_trades[ticker]['status'] != 'open':
            return None

        buy_time_str = open_trades[ticker]['buy_time']
        buy_dt = datetime.fromisoformat(buy_time_str)

        times_iso = time_last_kline_start.get(ticker, [])
        dt_scale = [pd.to_datetime(s, utc=True) for s in times_iso]

        entry_idx = bisect_left(dt_scale, buy_dt)

        if entry_idx >= len(dt_scale):
            return None

        # Slice from entry to end
        start_idx = entry_idx
        df = pd.DataFrame({
            'time': [pd.to_datetime(t, utc=True) for t in times_iso[start_idx:]],
            'open': open_price[ticker][start_idx:],
            'high': high_price[ticker][start_idx:],
            'low': low_price[ticker][start_idx:],
            'close': close_price[ticker][start_idx:],
            'volume': volume[ticker][start_idx:],
        })

        if len(df) < 1:
            return None

        # Compute regimes
        window = int(self.phase_df_params[ticker]['params']['moving_average_length'] * 9.5)
        features = extract_features(df, window=window)
        scaled = self.scaler_global.transform(features)
        labels = self.kmeans_global.predict(scaled)
        regime_series = pd.Series(labels, index=df.index)

        window_size = int(self.phase_df_params[ticker]['params']['atr_period'] * 5.5)
        smoother = FastRollingMode(window_size=window_size)
        smoothed = [smoother.update(x) for x in labels]
        smoothed_regime = pd.Series(smoothed, index=df.index)

        # Generate signals
        regime_params = prepare_regime_params(self.phase_df_params[ticker]['params'])
        CV = AdaptiveTradingSystem(regime_params['base_params'])
        df = CV.generate_adaptive_signals(df, regime_series=smoothed_regime)

        # For real-time, set event_time and event_price for the first row (buy)
        df.loc[0, 'event_time'] = df.loc[0, 'time']
        df.loc[0, 'event_price'] = df.loc[0, 'close']
        # No event_sell, but not needed for features

        # No pnl, target

        # Compute indicators
        feats_list = self.final_cols[ticker]
        params = build_feature_params(self.final_params[ticker])  # Assuming build_feature_params defined
        gh, _ = calculate_indicators(df, feats_list, params=params, multy=False)

        gh['batch'] = 0
        gh['regime'] = smoothed_regime.astype('object')

        # Compute predd using transformer
        columns_for_neuro = feats_list + ['batch', 'regime', 'time']
        gh_test = gh[columns_for_neuro]
        X_test, _, _, _, _ = prepare_data_transformer(gh_test, 'normalized_target')  # y None

        with warnings.catch_warnings(record=True):
            warnings.simplefilter("always")
            predd = self.transformers[ticker].predict(X_test)

        gh['predd'] = predd

        # Add derived
        gh['predd_shift_5'] = gh['predd'].shift(5).fillna(0)
        gh['predd_pct'] = gh['predd'].pct_change(3).fillna(0)
        gh['predd_var'] = gh['predd'].rolling(10).var().fillna(0)

        # Prepare X for final predict (last row)
        columns_for_model = feats_list + ['regime', 'predd', 'predd_var', 'predd_shift_5', 'predd_pct']
        X_pred = gh[columns_for_model].iloc[-1:]

        # Predict
        model_data = self.models[ticker]
        best_method = model_data['best_method']

        if best_method == 'regressor':
            pred = model_data['pipe_reg'].predict(X_pred)
            pred = apply_linear_calibration(pred, model_data['calib_reg'])
        elif best_method == 'ranker':
            pred = model_data['pipe_rank'].predict(X_pred)
            pred = ranker_postprocess_minus1_1(pred)
        elif best_method == 'combined':
            pred_reg = model_data['pipe_reg'].predict(X_pred)
            pred_reg = apply_linear_calibration(pred_reg, model_data['calib_reg'])
            pred_rank = model_data['pipe_rank'].predict(X_pred)
            pred_rank = ranker_postprocess_minus1_1(pred_rank)
            w = model_data['w_reg']
            pred = pred_reg * w + pred_rank * (1 - w)

        pred = float(pred[0])

        threshold = model_data['threshold']  # Or 0.75 as example; but use from meta

        if pred > threshold:
            return pred, True  # Sell
        else:
            return pred, False

    async def check_and_update_dicts(self):
        required_tickers = self.ticker_to_figi.keys()
        dicts = [
            open_price,
            close_price,
            high_price,
            low_price,
            volume,
            time_last_kline_start,
            time_last_kline_end,
            ma,
            open_trades,
            trading_data,
            pmax,
            signals
        ]

        for d in dicts:
            for ticker in required_tickers:
                if ticker not in d and d!=open_trades:
                    d[ticker] = []
                elif ticker not in d and d==open_trades:
                    d[ticker] = {}

    async def initialize_tickers_stream(self):
        async with AsyncClient(self.token) as client:
            instruments = await client.instruments.shares()
            for share in instruments.instruments:
                if share.ticker in TICKERS:
                    await asyncio.sleep(0.1)
                    status = await client.market_data.get_trading_status(figi=share.figi)
                    if status.trading_status !=1:
                        self.ticker_to_figi[share.ticker] = share.figi
                        self.figi_to_ticker[share.figi] = share.ticker
                        self.tickers[share.ticker] = [share.figi, share.name, share.sector]
                        self.active_tickers[share.ticker] = {
                            'figi': share.figi,
                            'name': share.name,
                            'status': status.trading_status
                        }
            await self.check_and_update_dicts()
            print(f"[INIT] Активных тикеров: {len(self.ticker_to_figi)}")

    async def process_candle_stream(self, candle: Candle):
        ticker = self.figi_to_ticker.get(candle.figi)
        if not ticker:
            return

        ts = candle.time + timedelta(hours=3)
        tc = ts + self.update_interval
        open_price[ticker] = open_price[ticker][1:] + [float(candle.open.units + candle.open.nano * 1e-9)]
        close_price[ticker] = close_price[ticker][1:] + [float(candle.close.units + candle.close.nano * 1e-9)]
        high_price[ticker] = high_price[ticker][1:] + [float(candle.high.units + candle.high.nano * 1e-9)]
        low_price[ticker] = low_price[ticker][1:] + [float(candle.low.units + candle.low.nano * 1e-9)]
        volume[ticker] = volume[ticker][1:] + [candle.volume]
        time_last_kline_start[ticker] = time_last_kline_start[ticker][1:] + [ts.isoformat()]
        time_last_kline_end[ticker] = time_last_kline_end[ticker][1:] + [tc.isoformat()]

        # Обработка сигналов
        await self.calculate_indicators_and_signals(ticker)

        #print(f"[{ticker}] Time Start: {ts:%Y-%m-%d %H:%M} "
        #      f"O: {open_price[ticker][-1]:.2f} H: {high_price[ticker][-1]:.2f} "
        #      f"L: {low_price[ticker][-1]:.2f} C: {close_price[ticker][-1]:.2f} "
        #      f"Signal: {signals[ticker]} Pmax: {pmax[ticker][-1]:.2f} Ma: {ma[ticker][-1]:.2f} "
        #      f"Time Close: {tc:%Y-%m-%d %H:%M}")


        if signals[ticker] == 'buy':

            way = phase_ticker_params[ticker]['way']
            ticker_parametrs = prepare_regime_params(phase_ticker_params[ticker]['params'])['calc_params']
            stop_loss_y_n = phase_ticker_params[ticker]['change_tp']
            stop_loss = phase_ticker_params[ticker]['stop_loss_pst_tp']

            if way == 'Базовый':

                if stop_loss_y_n == 'No':

                    open_trades[ticker] = {
                          'buy_price': close_price[ticker][-1],
                          'buy_time': (dt.utcnow() + timedelta(hours=3)).isoformat(),
                          'max_price': close_price[ticker][-1],
                          'status': 'open',
                          'reason': 'BASE'
                      }
                else:
                    open_trades[ticker] = {
                          'buy_price': close_price[ticker][-1],
                          'buy_time': (dt.utcnow() + timedelta(hours=3)).isoformat(),
                          'max_price': close_price[ticker][-1],
                          'status': 'open',
                          'reason': 'BASE',
                          'stop_loss': stop_loss
                      }

                bot.send_message(
                    self.stat_tg,
                    f"[TRADE OPENED] {ticker} по {close_price[ticker][-1]} в {dt.utcnow()}"
                )


            elif way == 'RSI':
                rsi_length = ticker_parametrs['rsi_length']
                use_smoothing = ticker_parametrs['use_smoothing']
                smoothing_length = ticker_parametrs['smoothing_length']
                smoothing_type = ticker_parametrs['smoothing_type']
                alma_sigma = ticker_parametrs['alma_sigma']
                rsi_overbought = ticker_parametrs['rsi_overbought']
                rsi_oversold = ticker_parametrs['rsi_oversold']
                use_knn = ticker_parametrs['use_knn']
                knn_neighbors = ticker_parametrs['knn_neighbors']
                knn_lookback = ticker_parametrs['knn_lookback']
                knn_weight = ticker_parametrs['knn_weight']
                feature_count = ticker_parametrs['feature_count']
                use_filter = ticker_parametrs['use_filter']
                filter_method = ticker_parametrs['filter_method']
                filter_strength = ticker_parametrs['filter_strength']
                sma_length = ticker_parametrs['sma_length']
                ema_length = ticker_parametrs['ema_length']
                rsi_theasold = ticker_parametrs['rsi_helbuth']

                ml_rsi = MachineLearningRSI(
                    rsi_length=int(rsi_length),
                    use_smoothing=use_smoothing,
                    smoothing_length=int(smoothing_length),
                    smoothing_type=smoothing_type,
                    alma_sigma=int(alma_sigma),
                    rsi_overbought=int(rsi_overbought),
                    rsi_oversold=int(rsi_oversold),
                    use_knn=use_knn,
                    knn_neighbors=int(knn_neighbors),
                    knn_lookback=int(knn_lookback),
                    knn_weight=knn_weight,
                    feature_count=int(feature_count),
                    use_filter=use_filter,
                    filter_method=filter_method,
                    filter_strength=filter_strength,
                    sma_length=int(sma_length),
                    ema_length=int(ema_length)
                )

                rsi, sma, ma_sma = ml_rsi.fit(pd.Series(close_price[ticker]))

                if rsi>=rsi_theasold:
                  if stop_loss_y_n == 'No':

                      open_trades[ticker] = {
                            'buy_price': close_price[ticker][-1],
                            'buy_time': (dt.utcnow() + timedelta(hours=3)).isoformat(),
                            'max_price': close_price[ticker][-1],
                            'status': 'open',
                            'reason': 'BASE'
                        }
                  else:
                      open_trades[ticker] = {
                            'buy_price': close_price[ticker][-1],
                            'buy_time': (dt.utcnow() + timedelta(hours=3)).isoformat(),
                            'max_price': close_price[ticker][-1],
                            'status': 'open',
                            'reason': 'BASE',
                            'stop_loss': stop_loss
                        }
                  bot.send_message(
                      self.stat_tg,
                      f"[TRADE OPENED] {ticker} по {close_price[ticker][-1]} в {dt.utcnow()}"
                  )
                else:
                  bot.send_message(
                        self.stat_tg,
                        f"[TRADE NOT OPENED] {ticker} по {close_price[ticker][-1]} в {dt.utcnow()} не прошел проверку на трендовую линию"
                    )



            elif way== 'RSI_SMA_EMA':
                  rsi_length = ticker_parametrs['rsi_length']
                  use_smoothing = ticker_parametrs['use_smoothing']
                  smoothing_length = ticker_parametrs['smoothing_length']
                  smoothing_type = ticker_parametrs['smoothing_type']
                  alma_sigma = ticker_parametrs['alma_sigma']
                  rsi_overbought = ticker_parametrs['rsi_overbought']
                  rsi_oversold = ticker_parametrs['rsi_oversold']
                  use_knn = ticker_parametrs['use_knn']
                  knn_neighbors = ticker_parametrs['knn_neighbors']
                  knn_lookback = ticker_parametrs['knn_lookback']
                  knn_weight = ticker_parametrs['knn_weight']
                  feature_count = ticker_parametrs['feature_count']
                  use_filter = ticker_parametrs['use_filter']
                  filter_method = ticker_parametrs['filter_method']
                  filter_strength = ticker_parametrs['filter_strength']
                  sma_length = ticker_parametrs['sma_length']
                  ema_length = ticker_parametrs['ema_length']
                  rsi_theasold = ticker_parametrs['rsi_helbuth']

                  ml_rsi = MachineLearningRSI(
                    rsi_length=int(rsi_length),
                    use_smoothing=use_smoothing,
                    smoothing_length=int(smoothing_length),
                    smoothing_type=smoothing_type,
                    alma_sigma=int(alma_sigma),
                    rsi_overbought=int(rsi_overbought),
                    rsi_oversold=int(rsi_oversold),
                    use_knn=use_knn,
                    knn_neighbors=int(knn_neighbors),
                    knn_lookback=int(knn_lookback),
                    knn_weight=knn_weight,
                    feature_count=int(feature_count),
                    use_filter=use_filter,
                    filter_method=filter_method,
                    filter_strength=filter_strength,
                    sma_length=int(sma_length),
                    ema_length=int(ema_length)
                )

                  rsi, sma, ma_sma = ml_rsi.fit(pd.Series(close_price[ticker]))

                  if rsi>rsi_theasold and sma > close_price[ticker][-1] and ma_sma > close_price[ticker][-1]:

                    if stop_loss_y_n == 'No':

                        open_trades[ticker] = {
                              'buy_price': close_price[ticker][-1],
                              'buy_time': (dt.utcnow() + timedelta(hours=3)).isoformat(),
                              'max_price': close_price[ticker][-1],
                              'status': 'open',
                              'reason': 'BASE'
                          }
                    else:
                        open_trades[ticker] = {
                              'buy_price': close_price[ticker][-1],
                              'buy_time': (dt.utcnow() + timedelta(hours=3)).isoformat(),
                              'max_price': close_price[ticker][-1],
                              'status': 'open',
                              'reason': 'BASE',
                              'stop_loss': stop_loss
                          }

                    bot.send_message(
                        self.stat_tg,
                        f"[TRADE OPENED] {ticker} по {close_price[ticker][-1]} в {dt.utcnow()}"
                    )
                  else:
                    bot.send_message(
                        self.stat_tg,
                        f"[TRADE NOT OPENED] {ticker} по {close_price[ticker][-1]} в {dt.utcnow()} не прошел проверку на Трендовая + недельные скользящие"
                    )

        if ticker in open_trades and open_trades[ticker]['status'] == 'open':
            pred, should_sell = await self.compute_model_prediction(ticker)
            if should_sell:
                signals[ticker] = 'sell'
                open_trades[ticker]['reason'] = 'model'
                # Then the sell logic can trigger in the existing if

        await self.check_open_trades(ticker)

        if signals[ticker] == 'sell' and (isinstance(open_trades, dict)and isinstance(open_trades.get(ticker), dict)and open_trades[ticker].get('status') == 'open'):

            reason = open_trades[ticker]['reason']

            await self.close_trade(ticker, close_price[ticker][-1], open_trades[ticker]['max_price'], reason)

        #check = await self.process_buy_signal(ticker)



    async def check_open_trades(self, ticker):
        """Проверяет состояние открытой позиции по тикеру с защитой от ошибок"""
        # Глобальные переменные (не атрибуты класса!)
        global open_trades, close_price, signals

        # 1. Безопасная проверка открытой позиции
        if not (
            isinstance(open_trades, dict)  # open_trades - это словарь
            and isinstance(open_trades.get(ticker), dict)  # тикер есть в open_trades
            and open_trades[ticker].get('status') == 'open'  # статус 'open'
        ):
            return

        # 2. Безопасный доступ к ценам
        if not (
            isinstance(close_price, dict)  # close_price - словарь
            and isinstance(close_price.get(ticker), list)  # тикер есть в close_price
            and len(close_price[ticker]) > 0  # есть хотя бы одна цена
        ):
            return

        trade_info = open_trades[ticker]
        current_price = close_price[ticker][-1]

        # 3. Обновление максимальной цены
        if 'max_price' in trade_info and current_price > trade_info['max_price']:
            trade_info['max_price'] = current_price

        # 4. Проверка стоп-лосса (с защитой от отсутствия ключей)
        if all(k in trade_info for k in ('stop_loss', 'buy_price')):
            stop_loss_price = trade_info['buy_price'] * (1 - trade_info['stop_loss'])
            if current_price <= stop_loss_price:
                signals[ticker] = 'sell'  # Глобальный словарь signals
                trade_info.update({
                    'reason': 'TAKE_LOSS',
                    'status': 'closed',
                    'close_price': current_price,
                    'close_time': dt.utcnow().isoformat()
                })

    async def check_missing_periodically(self):
        """Проверка пропущенных тикеров с учетом ближайшей свечной 5-минутки"""
        while self.running:
            try:
                now = dt.utcnow()

                # Рассчитываем ближайшую 5-минутку (или кратную self.timestop минуту)
                minutes = (now.minute // self.timestop) * self.timestop
                next_interval = now.replace(minute=minutes, second=0, microsecond=0)

                if next_interval <= now:
                    # Если текущая 5-минутка уже прошла, берем следующую
                    next_interval += timedelta(minutes=self.timestop)

                # Добавляем 10 секунд к расчетному времени
                next_check_time = next_interval + timedelta(seconds=10)

                # Рассчитываем время до следующей проверки
                time_to_wait = (next_check_time - now).total_seconds()

                # Ждем до ближайшей 5-минутки + 10 секунд
                await asyncio.sleep(time_to_wait)

                # Рассчитываем метку времени завершения предыдущего интервала
                expected_end = next_interval
                expected_end_ts = int(expected_end.timestamp() * 1000)

                # Проверяем, какие тикеры не обновились
                missing = []
                for ticker in self.active_tickers:
                    last_ts = time_last_kline_end.get(ticker, 0)

                    # Если last_ts — список или кортеж, берем последний элемент
                    if isinstance(last_ts, (list, tuple)):
                        last_ts = last_ts[-1] if last_ts else 0

                    if last_ts < expected_end_ts:
                        missing.append(ticker)

                # Выводим предупреждение, если есть пропущенные тикеры
                if missing:
                    print(f"\n[WARNING] Missing data for {len(missing)} tickers "
                          f"in period ending {expected_end:%H:%M}:")
                    for t in sorted(missing):
                        print(f"- {t}")
                    print(f"Total missing: {len(missing)}\n")

            except Exception as e:
                print(f"Error in missing tickers check: {e}")
                await asyncio.sleep(10)

    '''async def stream_candles_batch_with_backoff(self, figi_batch, chunk_id):
        backoff = 1
        max_backoff = 60  # секунд

        while True:
            try:
                async with AsyncClient(self.token) as client:
                    async def request_iterator():
                        yield MarketDataRequest(
                            subscribe_candles_request=SubscribeCandlesRequest(
                                subscription_action=SubscriptionAction.SUBSCRIPTION_ACTION_SUBSCRIBE,
                                instruments=[
                                    CandleInstrument(figi=figi, interval=self.candleintervall)
                                    for figi in figi_batch
                                ],
                                waiting_close=True
                            )
                        )
                        while True:
                            await asyncio.sleep(1)

                    #print(f"[Stream Init] Чанк {chunk_id} FIGIs: {len(figi_batch)}")

                    stream = client.market_data_stream.market_data_stream(request_iterator())

                    # Сохраняем время первого запуска
                    self.stream_last_activity[chunk_id] = dt.utcnow()

                    async for response in stream:
                        # Это критически важно — обновить время активности ТОЛЬКО при получении свечи!
                        if response.candle:
                            self.stream_last_activity[chunk_id] = dt.utcnow()
                            await self.process_candle_stream(response.candle)

                        # Пример: получение и логирование метаданных (по возможности)
                        if hasattr(stream, "initial_metadata"):
                            metadata = await stream.initial_metadata()
                            if metadata:
                                print(f"[Meta] Chunk {chunk_id}: {metadata}")

                    # Если цикл завершился — сбрасываем backoff
                    backoff = 1

            except Exception as e:
                print(f"[BACKOFF] Ошибка в чанке {chunk_id}: {e}")
                bot.send_message(self.error_tg, f"[Stream BACKOFF] Chunk {chunk_id} error: {e}")
                await asyncio.sleep(backoff + random.uniform(0, 2))
                backoff = min(max_backoff, backoff * 2)

    async def stream_candles_batch(self, figi_batch: list):
        """Обрабатывает стрим по группе тикеров."""
        async with AsyncClient(self.token) as client:
            async def request_iterator():
                yield MarketDataRequest(
                    subscribe_candles_request=SubscribeCandlesRequest(
                        subscription_action=SubscriptionAction.SUBSCRIPTION_ACTION_SUBSCRIBE,
                        instruments=[
                            CandleInstrument(figi=figi, interval=self.candleintervall)
                            for figi in figi_batch
                        ],
                        waiting_close=True
                    )
                )
                while True:
                    await asyncio.sleep(1)

            try:
                stream = client.market_data_stream.market_data_stream(request_iterator())
                async for response in stream:
                    if response.candle:
                        await self.process_candle_stream(response.candle)
            except Exception as e:
                print(f"[Batch Stream Error] {e}")
                bot.send_message(self.error_tg, f"[Ошибка потока в чанке]: {e}")
                raise e

    async def candle_stream_handler(self):
        figi_list = list(self.figi_to_ticker.keys())
        chunk_size = 20
        figi_chunks = list(split_figis(figi_list, chunk_size))

        self.stream_last_activity = {}
        self.stream_tasks = {}

        for chunk in figi_chunks:
            chunk_id = hashlib.md5(",".join(chunk).encode()).hexdigest()[:8]
            self.stream_last_activity[chunk_id] = dt.utcnow()
            task = asyncio.create_task(self.stream_candles_batch_with_backoff(chunk, chunk_id))
            self.stream_tasks[chunk_id] = task

        # Отдельная задача для watchdog (запускать только один раз!)
        if not hasattr(self, '_watchdog_started'):
            self._watchdog_started = True
            asyncio.create_task(self.watchdog_controller(chunk_size=chunk_size, timeout_minutes=60))

    async def watchdog_controller(self, chunk_size=20, timeout_minutes=3):
        print("[WATCHDOG] Контроллер запущен")
        while True:
            now = dt.utcnow()
            for chunk_id, last_time in list(self.stream_last_activity.items()):
                if (now - last_time) > timedelta(minutes=timeout_minutes):
                    print(f"[WATCHDOG] Чанк завис: {chunk_id}. Перезапуск...")

                    # Завершаем старую задачу
                    task = self.stream_tasks.get(chunk_id)
                    if task:
                        task.cancel()
                        await asyncio.sleep(1)

                    # Восстанавливаем FIGI для чанка
                    all_figi = list(self.figi_to_ticker.keys())
                    for chunk in split_figis(all_figi, chunk_size):
                        test_id = hashlib.md5(",".join(chunk).encode()).hexdigest()[:8]
                        if test_id == chunk_id:
                            new_task = asyncio.create_task(self.stream_candles_batch_with_backoff(chunk, chunk_id))
                            self.stream_tasks[chunk_id] = new_task
                            self.stream_last_activity[chunk_id] = dt.utcnow()
                            print(f"[WATCHDOG] Чанк {chunk_id} перезапущен.")
                            break

            await asyncio.sleep(30)'''

    '''async def candle_stream_handler(self):
        """Инициализирует несколько параллельных стримов по чанкам FIGI."""
        #await self.initialize_tickers_stream()
        #await self.backfill_missing_candles()

        figi_list = list(self.figi_to_ticker.keys())
        chunk_size = 15  # 🔧 Настраиваемый размер чанка
        figi_chunks = list(split_figis(figi_list, chunk_size))

        # Ограничиваем количество параллельных стримов
        max_concurrent_streams = 3  # Не более 3 чанков одновременно
        semaphore = asyncio.Semaphore(max_concurrent_streams)

        async def run_chunk(chunk):
            async with semaphore:
                await self.stream_candles_batch(chunk)

        #tasks = []
        #for chunk in figi_chunks:
        #    task = asyncio.create_task(self.stream_candles_batch(chunk))
        #    tasks.append(task)

        tasks = []
        for chunk in figi_chunks:
            tasks.append(run_chunk(chunk))

        done, pending = await asyncio.wait(tasks, return_when=asyncio.FIRST_EXCEPTION)

        # Ждём завершения хотя бы одного потока (ошибка/завершение)
        #done, pending = await asyncio.wait(tasks, return_when=asyncio.FIRST_EXCEPTION)

        # Обрабатываем завершившиеся задачи
        for d in done:
            if d.exception():
                print(f"[Stream ERROR] Один из потоков вернул ошибку: {d.exception()}")
                # Отменяем все оставшиеся стримы
                for p in pending:
                    p.cancel()
                raise d.exception()'''

    async def candle_stream_handler(self):
        """
        Один устойчивый поток для получения свечей по всем FIGI.
        Без разбивки на чанки. Без параллельных стримов.
        """
        await self.initialize_tickers_stream()
        await self.backfill_missing_candles()

        figi_list = list(self.figi_to_ticker.keys())

        async with AsyncClient(self.token) as client:

            async def request_iterator():
                # Сначала отписываемся от всех FIGI (на случай предыдущих подписок)
                yield MarketDataRequest(
                    subscribe_candles_request=SubscribeCandlesRequest(
                        subscription_action=SubscriptionAction.SUBSCRIPTION_ACTION_UNSUBSCRIBE,
                        instruments=[
                            CandleInstrument(figi=figi, interval=self.candleintervall)
                            for figi in figi_list
                        ],
                        waiting_close=True
                    )
                )
                await asyncio.sleep(0.5)  # Дать API обработать отписку

                # Теперь подписываемся на все FIGI одним запросом
                yield MarketDataRequest(
                    subscribe_candles_request=SubscribeCandlesRequest(
                        subscription_action=SubscriptionAction.SUBSCRIPTION_ACTION_SUBSCRIBE,
                        instruments=[
                            CandleInstrument(figi=figi, interval=self.candleintervall)
                            for figi in figi_list
                        ],
                        waiting_close=True
                    )
                )
                # Поддерживаем жизнь стрима
                while True:
                    await asyncio.sleep(1)

            try:
                stream = client.market_data_stream.market_data_stream(request_iterator())

                async for response in stream:
                    if response.candle:
                        await self.process_candle_stream(response.candle)

            except AioRpcError as e:
                status = e.code()
                description = e.details()
                print(f"[gRPC Error]: {status}, {description}")
                bot.send_message(self.error_tg, f"[gRPC Error]: {status}, {description}")
                raise e  # Позволит внешней логике обработать и перезапустить

            except Exception as e:
                print(f"[Stream Exception]: {e}")
                bot.send_message(self.error_tg, f"[Stream Exception]: {e}")
                raise e

    '''async def candle_stream_handler(self):
        async with AsyncClient(self.token) as client:
            await self.initialize_tickers_stream()
            await self.backfill_missing_candles()

            figi_list = list(self.figi_to_ticker.keys())

            yield MarketDataRequest(
                subscribe_candles_request=SubscribeCandlesRequest(
                    subscription_action=SubscriptionAction.SUBSCRIPTION_ACTION_UNSUBSCRIBE,
                    instruments=[
                        CandleInstrument(figi=figi, interval=self.candleintervall)
                        for figi in figi_list
                    ],
                    waiting_close=True
                )
            )
            await asyncio.sleep(0.5)

            async def request_iterator():
                yield MarketDataRequest(
                    subscribe_candles_request=SubscribeCandlesRequest(
                        subscription_action=SubscriptionAction.SUBSCRIPTION_ACTION_SUBSCRIBE,
                        instruments=[
                            CandleInstrument(
                                figi=figi,
                                interval=self.candleintervall
                            ) for figi in figi_list
                        ],
                        waiting_close=True
                    )
                )
                while True:
                    await asyncio.sleep(1)

            try:
                stream = client.market_data_stream.market_data_stream(request_iterator())
                async for response in stream:
                    if response.candle:
                        await self.process_candle_stream(response.candle)
            except Exception as e:
                print(f"[Stream Error] {e}")
                bot.send_message(self.error_tg, f"[Ошибка потока]: {e}")
                await asyncio.sleep(10)'''


    async def monitor_memory_usage(self):
        """Функция для мониторинга использования памяти."""
        process = psutil.Process()  # Получаем текущий процесс
        while True:
            try:
                # Получаем информацию об использовании памяти
                mem_info = process.memory_info()
                rss_memory = mem_info.rss / 1024 ** 2  # Resident Set Size (в МБ)
                vms_memory = mem_info.vms / 1024 ** 2  # Virtual Memory Size (в МБ)
                memory_usage_message = (
                    f"Использование памяти:\n"
                    f"- RSS (физическая память): {rss_memory:.2f} МБ\n"
                    f"- VMS (виртуальная память): {vms_memory:.2f} МБ"
                )
                # Отправляем сообщение в Telegram
                bot.send_message(self.memmory_id, memory_usage_message)
            except Exception as e:
                # Логируем ошибки в случае проблем
                bot.send_message(self.memmory_id, f"Ошибка мониторинга памяти: {e}")
            # Ждем 60 секунд до следующей проверки
            await asyncio.sleep(5*60)

    async def run(self):
          while True:
              try:
                  await self.create_tickers()  # Инициализация тикеров (подписки и т.д.)

                  # Выполняется один раз при первом запуске
                  #if not getattr(self, "initialized", False):
                  #    await self.initialize_tickers_stream()
                  #    await self.backfill_missing_candles()
                  #    self.initialized = True  # Больше не выполнять

                  # Основные задачи (стрим и watchdog)
                  tasks = [
                      self.candle_stream_handler(),
                      #self.save_data(),
                      self.monitor_memory_usage()
                  ]
                  results = await asyncio.gather(*tasks, return_exceptions=True)

                  for res in results:
                      if isinstance(res, Exception):
                          print(f"[EXCEPTION] Задача вернула исключение: {res}")
                          raise res  # Это заставит выйти из try и сработает перезапуск

                  #await asyncio.gather(*tasks)

              except AioRpcError as e:
                  # Обработка ошибок gRPC-потока (в том числе RST_STREAM code 8)
                  status_code = e.code()
                  description = e.details()

                  if status_code in (StatusCode.CANCELLED, StatusCode.UNAVAILABLE):
                      print(f"[RESTART] gRPC поток отменён: {status_code}, {description}. Перезапуск через 10 секунд...")
                      bot.send_message(self.error_tg, f"[RESTART] gRPC ошибка: {status_code}, {description}")
                      await asyncio.sleep(10)
                      continue

                  else:
                      print(f"[CRITICAL] gRPC критическая ошибка: {status_code} - {description}")
                      bot.send_message(self.error_tg, f"[CRITICAL] Ошибка gRPC: {status_code} — {description}")
                      await asyncio.sleep(30)
                      continue

              except Exception as e:
                  error_message = str(e)

                  if "Remote end closed connection without response" in error_message \
                    or "RST_STREAM" in error_message \
                    or "error code 8" in error_message:
                      print("[RESTART] Обнаружен обрыв соединения. Перезапуск бота через 10 секунд...")
                      bot.send_message(self.error_tg, "[RESTART] Обрыв соединения (RST_STREAM). Перезапуск бота.")
                      await asyncio.sleep(10)
                      continue

                  else:
                      # Все остальные ошибки
                      print(f"[CRITICAL] Необработанная ошибка: {e}")
                      bot.send_message(self.error_tg, f"[CRITICAL] Необработанная ошибка: {e}")
                      await asyncio.sleep(30)
                      continue

def split_figis(figi_list, chunk_size):
    """Разбивает список FIGI на чанки фиксированного размера."""
    for i in range(0, len(figi_list), chunk_size):
        yield figi_list[i:i + chunk_size]


# Main entry point
if __name__ == "__main__":
    token = 't.bWjxPlU6vz77qoqS754OKy0QorLDaZP-CE091dhGl56v7GHrqgF-mQAdWaeRg2kDRJmmxzvaaOwKUTxW6dnOKg'
    path_to_save = path_to_save#'/projects/LONG_ALERT/data/'
    sector = SECTOR
    country = COUNTRY
    period = 15
    index = "IMOEX2"
    tg_channels = {
        'error': '-1002596481496',
        'a1_screen': '-1002555215009',
        'memory': '-1002277392256',
        'stat': '-1002681271607',
        'sells': '-1002619839070'
    }

    collector = TinkoffInvestDataCollector(token, path_to_save, sector, country, tg_channels, period, index)

    # Запускаем асинхронный код
    #asyncio.run(collector.run())
    async def run_at_specific_times():
        while True:
            now = datetime.datetime.now()
            current_minute = now.minute
            current_second = now.second
            target_minutes = [0, 15, 30, 45]

            # Находим следующий 15-минутный интервал
            next_target = None
            for minute in target_minutes:
                if minute > current_minute:
                    next_target = minute
                    break

            if next_target is None:  # Если текущая минута больше 45
                next_target = 0
                wait_minutes = 60 - current_minute
            else:
                wait_minutes = next_target - current_minute

            # Вычисляем оставшиеся секунды до следующего интервала
            remaining_seconds = wait_minutes * 60 - current_second

            # Логика запуска:
            # Если до следующего интервала <= 8 минут (480 секунд) - ждем его наступления
            # Если > 8 минут - запускаем немедленно
            if remaining_seconds <= 60*9:  # 8 минут = 480 секунд
                # Ждем наступления следующего интервала
                sleep_seconds = remaining_seconds
                print(f"До следующего интервала {next_target} минут осталось {remaining_seconds//60} мин {remaining_seconds%60} сек - ждем его наступления")
            else:
                # Запускаем немедленно
                print(f"До следующего интервала {wait_minutes} мин - запускаем немедленно")
                await collector.run()

                # После выполнения вычисляем время до следующего запуска
                now_after = datetime.datetime.now()
                current_minute_after = now_after.minute
                current_second_after = now_after.second

                # Находим следующий интервал
                for minute in target_minutes:
                    if minute > current_minute_after:
                        next_target_after = minute
                        break
                else:
                    next_target_after = 0

                # Вычисляем сколько ждать
                if next_target_after == 0:
                    wait_minutes_after = 60 - current_minute_after
                else:
                    wait_minutes_after = next_target_after - current_minute_after

                remaining_seconds_after = wait_minutes_after * 60 - current_second_after
                sleep_seconds = remaining_seconds_after

            print(f"Следующая проверка через {sleep_seconds//60} мин {sleep_seconds%60} сек")
            await asyncio.sleep(sleep_seconds)

    # Запускаем асинхронный код
    asyncio.run(run_at_specific_times())